<h1 style="text-align:center;">Secret Sharing for Private Information Retrieval</h1> 

## 1. Introduction

Introduced by Chor, Goldreich e Kushilevitz [1,2] in 1995, Private Information Retrieval (PIR) is a cryptographic protocol used to access a specific message in a distributed database replicated in some servers without revealing any information about which message is being accessed to the servers. PIR has many interesting applications such as private research, access to medical documents, bank accounts, market statistics for business investment, video streaming services, among others.

A PIR scheme must be able to access the information desired by the user in such a way that any server is unable to learn any information about what information is being accessed. Thus, a trivial solution to the problem could be the user download the entire database from a server. In this way, the user will have all the messages in the database, which guarantees access to the desired information while ensuring security, because the user shows interest in all the messages in the database, so the server remains unaware of which message is of interest. However, as modern databases have a high amount of information stored, this solution is considered computationally expensive and impractical. Therefore, the user wants to minimize the total communication cost.

An interesting approach to this problem is that we can use Secret Sharing to create PIR schemes. The main idea is to use polynomials over finite fields, since polynomials provide a simple and efficient way of generating and combining secret data such as solving linear systems or taking a polynomial interpolation. In addition, finite fields will provide algebraic and statistical properties that make PIR schemes perfectly secure from the Information Theory perspective.

In this notebook, we will present Private Information Retrieval using Secret Sharing schemes, motivating from the most straightforward scheme to the most elaborate schemes that are optimal for finite parameters. For all possible solutions to the problem, we will present a possible implementation of the algorithm and a demonstration of why the scheme has the properties of retrieving and securing the desired message.

## 2. Finite Fields

In order to present the Secret Sharing and PIR schemes, it will be sufficient to give an overview of a particularly simple and important category of finite fields, the prime fields or the field of remainders in modulo $p$ where $p$ is prime, but first, let us remember the definition of a field.

**Definition 2.1:** A **field** is a set $\mathbb{F}$ with two binary operation called addition $(+)$ and multiplication $(*)$ that satisfies the following properties $\forall a,b,c \in \mathbb{F}$:

1. **Closure:** The addition and multiplication of any elements in $\mathbb{F}$ belongs to $\mathbb{F}$.
2. **Associativity:** $(a+b)+c=a+(b+c)$ and $(a*b)*c=a*(b*c)$.
3. **Identity:** There is an element $0 \in \mathbb{F}$ with respect to the addition and $1 \in \mathbb{F}$ with respect to the multiplication such that $a+0 = 0+a = a$ and $a*1 = 1*a = a$.
4. **Inverse:** There is a elements $a' \in \mathbb{F}$ such that $a+ a' = a'+ a = 0$. There is a element $a'' \in \mathbb{F}$ for $a \neq 0$ such that $a*a'' = a''*a = 1$.
5. **Commutative:** $a*b=b*a$.
6. **Distributive:** $a*(b+c)=(a*b)+(a*c)$.

**Definition 2.2:** Let $p \in \mathbb{Z}$ be a prime number. The **field of remainders in modulo $\mathbf{p}$** is the set 
\begin{align*}
    \mathbb{Z}_p=\{\overline{0},\overline{1},...,\overline{p-1}\}
\end{align*}
where the operations are defined as, for $\overline{a},\overline{b} \in \mathbb{Z}_p$, we do the usual addition of $\mathbb{Z}$, divide $(a+b)$ by $p$ to get the remainder $0\leq r \leq p-1$ and define $\overline{a+b}=\overline{r}$. We define multiplication in the same way.

**Example 2.1:** The operations in the finite field $\mathbb{Z}_{2}= \{\overline{0},\overline{1}\}$ are defined in the tables below:
$$
\begin{aligned}
\begin{array}{|c|c|c|}
\hline \text {+} & \overline{0} & \overline{1} \\
\hline \overline{0} & \overline{0} & \overline{1} \\
\overline{1} & \overline{1} & \overline{0} \\
\hline
\end{array}
\ \ \ \ \ \ \ \ \ \ \ \ \         
\begin{array}{|c|c|c|}
\hline \text {*} & \overline{0} & \overline{1} \\
\hline \overline{0} & \overline{0} & \overline{0} \\
\overline{1} & \overline{0} & \overline{1} \\
\hline
\end{array}
\end{aligned}
$$

**Example 2.2:** The operations in the finite field $\mathbb{Z}_{3}= \{\overline{0},\overline{1},\overline{2}\}$ are defined in the tables below:
$$
\begin{aligned}
\begin{array}{|c|c|c|c|}
\hline \text {+} & \overline{0} & \overline{1} & \overline{2} \\
\hline \overline{0} & \overline{0} & \overline{1} & \overline{2} \\
\overline{1} & \overline{1} & \overline{2} & \overline{0} \\
\overline{2} & \overline{2} & \overline{0} & \overline{1} \\
\hline
\end{array}
\ \ \ \ \ \ \ \ \ \ \ \ \         
\begin{array}{|c|c|c|c|}
\hline \text {*} & \overline{0} & \overline{1} & \overline{2} \\
\hline \overline{0} & \overline{0} & \overline{0} & \overline{0} \\
\overline{1} & \overline{0} & \overline{1} & \overline{2} \\
\overline{2} & \overline{0} & \overline{2} & \overline{1} \\
\hline
\end{array}
\end{aligned}
$$

**Example 2.3:** The operations in the finite field $\mathbb{Z}_{5}= \{\overline{0},\overline{1},\overline{2},\overline{3},\overline{4}\}$ are defined in the tables below:
$$
\begin{aligned}
\begin{array}{|c|c|c|c|c|c|}
\hline \text {+} & \overline{0} & \overline{1} & \overline{2} & \overline{3} & \overline{4} \\
\hline \overline{0} & \overline{0} & \overline{1} & \overline{2} & \overline{3} & \overline{4} \\
\overline{1} & \overline{1} & \overline{2} & \overline{3} & \overline{4} & \overline{0} \\
\overline{2} & \overline{2} & \overline{3} & \overline{4} & \overline{0} & \overline{1} \\
\overline{3} & \overline{3} & \overline{4} & \overline{0} & \overline{1} & \overline{2} \\
\overline{4} & \overline{4} & \overline{0} & \overline{1} & \overline{2} & \overline{3} \\
\hline
\end{array}
\ \ \ \ \ \ \ \ \ \ \ \ \         
\begin{array}{|c|c|c|c|c|c|}
\hline \text {*} & \overline{0} & \overline{1} & \overline{2} & \overline{3} & \overline{4} \\
\hline \overline{0} & \overline{0} & \overline{0} & \overline{0} & \overline{0} & \overline{0} \\
\overline{1} & \overline{0} & \overline{1} & \overline{2} & \overline{3} & \overline{4} \\
\overline{2} & \overline{0} & \overline{2} & \overline{4} & \overline{1} & \overline{3} \\
\overline{3} & \overline{0} & \overline{3} & \overline{1} & \overline{4} & \overline{2} \\
\overline{4} & \overline{0} & \overline{4} & \overline{3} & \overline{2} & \overline{1} \\
\hline
\end{array}
\end{aligned}
$$

There are two other generic notations for finite fields, $\text{GF}(q)$ and $\mathbb{F}_q$, as there are other types of finite fields. For our purposes, we will use the notation $\text{GF}(q)$ where $\text{GF}(p)=\mathbb{Z}_p$ if $p$ is prime. We adopt this notation because we will use a python library called "galois" to implement the algorithms. Furthermore, to avoid confusion with the library notation, we will omit the slashes above the elements, denoting $\overline{x} \in \text{GF}(p)$ by $x$. 

**Remark 2.1:** The name "galois" comes from the famous mathematician Évariste Galois, known for his fundamental contributions to group theory, which has a significant importance in the study of finite fields. "$\text{GF}$" is equivalent to Galois Field.

The library provides many interesting things for implementing codes using finite fields arithmetic, learning about finite field theory (for example, the non-prime finite fields), provide error correction codes, etc. We can install galois from PyPI running "pip install galois" in the terminal. So, let us consider some examples that will be useful for learning some functions which we will use later.

**Example 2.4:** A finite field is defined using the command "galois.GF($q$)" where $q$ is the number of elements in your field.

In [1]:
import galois

GF2 = galois.GF(2)
print(GF2)

GF3 = galois.GF(3)
print(GF3)

GF5 = galois.GF(5)
print(GF5)

<class 'galois.GF(2)'>
<class 'galois.GF(3)'>
<class 'galois.GF(5)'>


**Example 2.5:** An element $x$ of a finite field $\text{GF}(q)$ for $0 \leq x \leq q-1$ can be represented as follows:

In [2]:
GF5 = galois.GF(5)
x = GF5(4)

print(x)
print(type(x))
print(type(4))

4
<class 'galois.GF(5)'>
<class 'int'>


**Example 2.6:** The operations of addition and multiplication are accepted if the elements belong to the same finite field:

In [3]:
GF3 = galois.GF(3)

x = GF3(2)
y = GF3(1)

print('The addition result is', x+y)
print('The multiplication result is', x*y)

The addition result is 0
The multiplication result is 2


The galois library also offers perfect integration with the NumPy library, which provides functions for performing operations with arrays, polynomial interpolations, solving linear systems, etc. (NumPy can be installed by running "pip install numpy" in the terminal)

**Example 2.7:** Any numpy array can be converted to a array over finite fields if the entries are elements $x \in \text{GF}(q)$:

In [4]:
import numpy as np

GF5 = galois.GF(5)

v = np.array([2, 1, 3])
print(type(v))

v = GF5(v)
print(type(v))
print(v)

<class 'numpy.ndarray'>
<class 'galois.GF(5)'>
[2 1 3]


**Example 2.8:** We can generate a random vector $v \in \text{GF}^n(q)$ or a random matrix $M \in \text{GF}^{n \times m}(q)$ as follows:

In [5]:
GF11 = galois.GF(11)
n = 4
m = 3

v = GF11.Random((n))
M = GF11.Random((n, m))

print(v)
print('')
print(M)


[8 9 5 1]

[[ 2  8  0]
 [ 0  6  7]
 [ 2 10  6]
 [ 9  6  5]]


**Example 2.9:** We can solve a linear system $Ax = b$ over finite fields as follows:

In [6]:
GF5 = galois.GF(5)

A = GF5(np.array([[1,1,1],[1,2,4],[1,3,4]]))
b = GF5(np.array([0, 2, 2]))

x = np.linalg.solve(A, b)
print('The solution for the matrix equation Ax = b is x =', x)
print('The type of x is', type(x))

The solution for the matrix equation Ax = b is x = [1 0 4]
The type of x is <class 'galois.GF(5)'>


**Example 2.10:** We can compute the determinant of a square matrix $M \in \text{GF}^{n \times n}(q)$ as follows:

In [7]:
GF17 = galois.GF(17)
n = 5

M = GF17(np.array([[1,1,1,1],[1,2,4,8],[1,3,9,10], [1,4,16,14]]))

det_M = np.linalg.det(M)
print('The matrix M is')
print(M)
print('and its determinant is', det_M)

The matrix M is
[[ 1  1  1  1]
 [ 1  2  4  8]
 [ 1  3  9 10]
 [ 1  4 16 14]]
and its determinant is 14


**Example 2.11:** We can compute the inner product of two vectors $v,u \in \text{GF}^{n}(q)$ as follows:

In [8]:
GF17 = galois.GF(17)

v = GF17(np.array([1, 1, 1]))
u = GF17(np.array([13, 1, 16]))

inner_prod = np.dot(v,u)
print('The inner product of v and u is', inner_prod)

The inner product of v and u is 13


**Example 2.12:** We can a vector $v \in \text{GF}^{n}(q)$ with 1 in a entry $i \in \{1,2,...,n\}$ and zero all the other as follows:

In [9]:
i = 1
n = 6

v = GF3(np.eye(n, dtype=int)[i-1])
print(v)

[1 0 0 0 0 0]


More information about the library galois can be found in https://pypi.org/project/galois/ and NumPy in https://numpy.org.

## 3. Definitions and Notations

**Definition 3.1:** The canonical basis of $\text{GF}^{n}(q)$ is a set $\{e_1,e_2,...,e_n\}$ where $e_i \in \text{GF}^{n}(q)$ is the vector with a $1$ in the $i$-th entry and zero in all other entries.

**Definition 3.2:** Given two vectors $v,u \in \text{GF}^{n}(q)$, written explicitly as
\begin{align*}
    v=[v_1,v_2,...,v_n]^t \text{ and }u=[u_1,u_2,...,u_n]^t,
\end{align*}
the inner product is a function $\langle \cdot, \cdot \rangle: \text{GF}^{n}(q)\times \text{GF}^{n}(q) \rightarrow \text{GF}^{n}(q)$ such that
\begin{align*}
    \langle v, u \rangle=v^tu=v_1u_1+v_2u_2+...+v_nu_n
\end{align*}

**Definition 3.3:** Given a positive integer $Z$, the set denoted by $[Z] \subset \mathbb{N}$ is defined by
\begin{align*}
    [Z] = \{1, 2, · · · , Z\}.
\end{align*}

**Example 3.1:** The set $\{1,2,3,4,5\} = [5]$.

**Definition 3.4:** Let $S$ be set. The Cardinality of $S$, denoted by $|S|$, is the number of elements in $S$. 

**Example 3.2:** The cardinality of the finite field $\text{GF}(q)$ is $|\text{GF}(q)|=q$.

**Definition 3.5:** Let $P$ be any statement. The Inverson Bracket of $P$, denoted by $[P]$, is function that maps $P$ into 1 if $P$ is true and 0 if $P$ is false, i.e., 
\begin{align*}
    [P]=
    \begin{cases}
    1, \text{if $P$ is true.}\\
    0, \text{if $P$ is false.}
    \end{cases}
\end{align*}

**Example 3.3:** The Iverson bracket of the statement $x=y$, i.e., $[x=y]$ is 1 if $x=y$ and 0 if $x\neq y$.

**Example 3.4:** Let $X$ be a uniform random variable in $\text{GF}(2)=\{0,1\}$, then
\begin{align*}
    \text{Pr}[X=x]=\frac{1}{|\text{GF}(2)|} \cdot [x \in \text{GF}(2)] =\frac{1}{2} \cdot [x \in \text{GF}(2)]
\end{align*}
In others words, $X$ only have non-zero probability when $x$ is a element of $\text{GF}(2)=\{0,1\}$, because $X$ is defined in $\text{GF}(2)=\{0,1\}$.

## 4. Private Information Retrieval

In private information retrieval (PIR), a user wants private access a specific message $D_i \in \text{GF}(q)$ stored in entry $i \in [M]$ of a distributed database $D \in \text{GF}^{M}(q)$ replicated in $N$ servers without revealing any information about which message is being accessed to any server. Note that all servers know what information is stored in the database, so our goal is to keep the value of the desired index $i$ secure. The most straightforward solution discussed in the introduction involves the user downloading the entire database. In this way, the user would be able to access the desired message $D_i$ while keeping the desired index $i$ secure, because the user declares interest in all entries at the same time, i.e. the servers cannot tell which index the user is interested in. However, since the modern databases have a high amount of information stored, this solution is considered computationally expensive and impractical. Then, the user has an interest in minimizing the computational costs of communication, i.e., Upload and Download.

So let us define what Upload and Donwload communication costs are to consider a simplest non-trivial PIR scheme.

**Definition 4.1:** Upload is the act of transferring data, as symbols in $\text{GF}(q)$, from a local device (typically the source device) to a remote server or another device.

**Definition 4.2:** Download is the act of transferring data, as symbols in $\text{GF}(q)$, from a remote server or another device to a local device (typically the destination device).

**Remark 4.1:** We will quantify the Upload and Download communication costs through the number of symbols/elements of $\text{GF}(q)$ that are involved in the respective processes, as we will see in the example below.

**Example 4.1:** Suppose that there are $N = 2$ servers that do not collude and each has a copy of database $D=[D_1,...,D_M ]^t \in \text{GF}^{M}(q)$. Furthermore, suppose a situation where the user needs to access the $i$-th entry of a database $D$, i.e., the secret is not $D_i \in \text{GF}(q)$, but the index $i \in [M]$. The user will proceed as follows.

- **Step 1:** Generate a uniform vector $R \in \text{GF}^M(q)$, i.e., $R$ can assume any vector in $\text{GF}^M(q)$ with the same probability.
- **Step 2:** The user transmits the query $Q_1=R$ to Server $1$ and $Q_2=e_i+R$ to Server $2$.
- **Step 3:** Each server $j$ computes the inner product $A_j=\langle Q_j, D \rangle$ and send it to the user.
- **Step 4:** The user decodes $D_i$ by doing $A_2-A_1=\langle e_i+R, D \rangle-\langle R, D \rangle=\langle e_i+R-R, D \rangle=\langle e_i, D \rangle=D_i$.

The scheme is secure due to the fact that each query $Q_j \in \text{GF}^M(q)$ is statistically independent of the index $i$, because
\begin{align*}
    &\text{Pr}[Q_1=r]=\text{Pr}[R=r]=\frac{1}{q^M},\\
    &\text{Pr}[Q_2=r]=\text{Pr}[R=r-e_i]=\frac{1}{q^M}.
\end{align*}
for $r \in \text{GF}^M(q)$.

In addition, for $m \in [M]$ and $r \in \text{GF}^M(q)$, the conditional probability of $Q_j$ given $i$ is given by
\begin{align*}
    &\text{Pr}[Q_1=r|i=m]=\text{Pr}[R=r|i=m]=\text{Pr}[R=r]=\frac{1}{q^M},\\
    &\text{Pr}[Q_2=r|i=m]=\text{Pr}[e_i+R=r|i=m]=\text{Pr}[R=r-e_m]=\frac{1}{q^M}.
\end{align*}

Then, by Bayes Theorem, we conclude that
\begin{align*}
    \text{Pr}[i=m|Q_j=r]&=\frac{\text{Pr}[Q_j=r|i=m]\text{Pr}[i=m]}{\text{Pr}[Q_j=r]}\\
    &=\frac{\frac{1}{q^M}\ \text{Pr}[i=m]}{\frac{1}{q^M}}\\
    &=\text{Pr}[i=m]
\end{align*}
In other words, the fact that each server $j$ has the query $Q_j$ does not change the probability of the desired index value, which implies that they are independent and the server cannot deduce anything about $i \in [M]$.

Considering, for example, $M=8$ and $q=3$, we can define a database as below:

In [10]:
GF3 = galois.GF(3)

D = GF3(np.array([2, 1, 0, 0, 2, 1, 1, 0]))
print(D)

[2 1 0 0 2 1 1 0]


Thus, for $i=1$, the user wants to access the symbol $D_1=2 \in \text{GF}(3)$ and the problem can be viewed computationally as follows:

- **Step 1:** Generate a uniform vector $R \in \text{GF}^8(3)$.

In [11]:
# Using example 2.8
R = GF3.Random((8))
print(R)

[0 1 1 1 0 1 0 1]


- **Step 2:** The user transmits the query $Q_1=R$ to Server $1$ and $Q_2=e_i+R$ to Server $2$.

In [12]:
e_i = GF3(np.eye(8, dtype=int)[0])
Q_1 = R
Q_2 = e_i + R

print("The Server 1 receives", Q_1)
print("The Server 2 receives", Q_2)

The Server 1 receives [0 1 1 1 0 1 0 1]
The Server 2 receives [1 1 1 1 0 1 0 1]


- **Step 3:** Each server $j$ computes the inner product $A_j=\langle Q_j, D \rangle$ and send it to the user.

In [13]:
A_1 = np.dot(D, Q_1)
A_2 = np.dot(D, Q_2)

print("The Server 1 answers with", A_1)
print("The Server 2 answers with", A_2)

The Server 1 answers with 2
The Server 2 answers with 1


- **Step 4:** The user decodes $D_i$ by doing $A_2-A_1=\langle e_i+R, D \rangle-\langle R, D \rangle=\langle e_i+R-R, D \rangle=\langle e_i, D \rangle=D_i$.

In [14]:
D_i = A_2 - A_1 
print("The desired message is", D_i)

The desired message is 2


The total cost of communication for this scheme is as follows:
- **Upload:** As the user uploads $M$ symbols of $\text{GF}(q)$ for each server, because $Q_j$ is a vector in $\text{GF}^M(q)$, the upload cost will be $2M$.
- **Download:** As the user downloads $1$ symbol of $\text{GF}(q)$ from each server, because $A_j \in \text{GF}(q)$, the download cost will be $2$.

Thus, the total cost of communication will be $2M+2$.

In the previous example each message $D_i$ consists of a single symbol in $\text{GF}(q)$. However, as we discussed at the beginning of this section, messages can have multiple symbols. Let us consider an example.


**Example 4.2:** Suppose that there are $N = 2$ servers that do not collude and each message has $s$ symbols $\text{GF}(q)$, i.e., $D_i \in \text{GF}^{s}(q)$. Each server has a copy of database $D=[D(1),...,D_(s) ]^t \in \text{GF}^{sM}(q)$ where each $D(k) \in \text{GF}^{M}(q)$ is a vector that contains the k-th symbol of each message. The reason for thinking of the database in this way instead of $D=[D_1,D_2,...,D_{sM} ]^t \in \text{GF}^{sM}(q)$ is that the server can reuse the queries $Q_j$ to retrieve all the symbols of the desired message $D_i \in \text{GF}^{s}(q)$. This process is described in detail below:

- **Step 1:** Generate a uniform vector $R \in \text{GF}^M(q)$, i.e., $R$ can assume any vector in $\text{GF}^M(q)$ with the same probability.
- **Step 2:** The user transmits the query $Q_1=R$ to Server $1$ and $Q_2=e_i+R$ to Server $2$.
- **Step 3:** Each server $j$ computes $s$ inner products $A_j^k=\langle Q_j, D(k) \rangle$ and send it to the user.
- **Step 4:** The user decodes $D_i=[A_2^1-A_1^1, A_2^2-A_1^2,...,A_2^s-A_1^s]^t \in \text{GF}^{s}(q)$.

The fact that this scheme is secure follows immediately by applying the demonstration of the previous example to $s$ databases $D(k) \in \text{GF}^{M}(q)$ for $k \in [s]$. In addition, we can verify this scheme computationally in a similar way to the previous example if we define $s$ databases $D(k)$.

The total cost of communication for this scheme is as follows:
- **Upload:** As the user uploads $M$ symbols of $\text{GF}(q)$ for each server, because $Q_j$ is a vector in $\text{GF}^M(q)$, the upload cost will be $2M$.
- **Download:** As the user downloads $s$ symbols of $\text{GF}(q)$ from each server, because $A_j^k \in \text{GF}(q)$, the download cost will be $2s$.

Thus, the total cost of communication will be $2M+2s$.

Comparing the two previous examples, we see that the upload cost is the same because we can reuse the queries $Q_j$ to retrieve all the symbols of the desired message $D_i \in \text{GF}^{s}(q)$. Thus, when messages are arbitrarily large, we should focus on schemes that minimize the download cost. Below, we will define the common metric for evaluating such PIR schemes, called Download Rate.

**Definition 4.3:** Consider a symbol to be an element of $\text{GF}(q)$. Then, the Download Rate $\mathcal{R}$ is defined by 
\begin{align*}
    \mathcal{R}=\frac{\text{number of symbols of the desired message}}{\text{number of symbols downloaded}}.
\end{align*}

**Example 4.3:** The download rate of the schemes in examples **4.1** and **4.2** is, respectively, $\frac{1}{2}$ and $\frac{s}{2s}=\frac{1}{2}$.

In the next section, we will present a generalization of the idea behind the schemes presented in the previous examples, called Secret Sharing. Furthermore, we will see that we can extend Secret Sharing into schemes known as Ramp-Secret Sharing, which will provide us with asymptotically optimal PIR schemes, i.e., schemes in which, as the message size increases, the performance approaches a maximum efficiency that we will determine.

## 5. Secret Sharing

Introduced by Adi Shamir[3] and George Backeley[4] in 1979, Secret Sharing is a cryptographic protocol used to divide a confidential information, referred to as the "secret", into multiple shares and distribute them in group of participants such that a minimum number of shares are needed to reconstruct the secret. 

Let us illustrate this with an example.

**Example 5.1:** A user wants to store a secret $S \in \text{GF}(5)$ with $N=4$ servers where any $T=2$ of them can collude to obtain information about the secret. Each participant receives a secret share and the sharing process must be done in such a way that no server can deduce any information about the secret only with their sharing, but in presence of three or more sharers, the secret can be completely determined. This problem can be solved by the so-called **(4,3)-Secret Sharing Scheme** as shown below:

- **Step 1:** Choose $R_1,R_2 \in \text{GF}(5)$ uniformly at random.
- **Step 2:** Define the polynomial $f(x)=S+R_1x+R_2x^2$ over the finite field $\text{GF}(5)$.
- **Step 3:** Send for each participant $i$ the evaluation $f(i)=S+R_1i+R_2i^2$, $i \in \{1,2,3,4\}$ and $N<q$. 

The scheme is secure because the polynomial $f(x)$ describes a parabola that is completely determined by three points. Therefore, since $T=2$ participants only have two evaluations, there is not a single parabola that passes through these points. Furthermore, since the discrete random variables $R_1,R_2$ are uniformly random, all possible values for $S \in \text{GF}(5)$ are equiprobable. However, if we have the shares of $T+1=2+1=3$ participants, we will have three different points to completely determine the coefficients of the polynomial $f(x)$, i.e., $S$ is decodable. In other words, with three different evaluations, we have, for $\{i_1,i_2,i_3\} \subset \{1,2,3,4\}$, the following system to solve:

$$\begin{cases}
  S+i_1R_1+i_1^2R_2 &= f(i_1) \\
  S+i_1R_1+i_2^2R_2 &= f(i_2) \\
  S+i_3R_1+i_3^2R_2 &= f(i_3)
\end{cases}
.
$$
Equivalently, the system above can be written as follows:
$$
\begin{bmatrix}
    i_1^0 & i_1^1 & i_1^2 \\
    i_2^0 & i_2^1 & i_2^2 \\
    i_3^0 & i_3^1 & i_3^2
\end{bmatrix}

\begin{bmatrix}
    S \\
    R_1 \\
    R_2
\end{bmatrix}
=
\begin{bmatrix}
    f(i_1) \\
    f(i_2) \\
    f(i_3)
\end{bmatrix}
$$

The matrix $3 \times 3$ is called a Vandermonde matrix $\text{V}(i_1,i_2,i_3)$ of coefficients $i_1,i_2,i_3$. A very interesting thing about a Vandermonde matrix is that if all the coefficients are distinct and non-zero, the matrix is invertible. Therefore, the matrix equation above always has a unique solution for the vector $[S, R_1,R_2]^t \in \text{GF}^3(5)$, i.e., $S$ is deterministic and can be written as a function of $f(i_1),f(i_2),f(i_3)$. Thus, in probabilistic terms, for $s, t_1,t_2,t_3 \in \text{GF}(5)$the Decodabilty property can be described as follows:
\begin{align*}
    \text{Pr}[S=s|f(i_1)=t_1,f(i_2)=t_2,f(i_3)=t_3]=
    \begin{cases}
        1, & \text{if} \ s=g(t_1,t_2,t_3). \\
        0, & \text{otherwise}.
        \end{cases}
\end{align*}
where $g(t_1,t_2,t_3)$ is a function that determines the values that $S$ will take on depending on the values of $t_1,t_2,t_3 \in \text{GF}(5)$.

Now, for the Security property, note that two participants observe the following linear system:

$$\begin{cases}
  S+i_1R_1+i_1^2R_2 &= f(i_1) \\
  S+i_1R_1+i_2^2R_2 &= f(i_2)
\end{cases}
.
$$
Equivalently, the system can be written as follows:
$$
\begin{bmatrix}
    i_1^0 & i_1^1 & i_1^2 \\
    i_2^0 & i_2^1 & i_2^2
\end{bmatrix}

\begin{bmatrix}
    S \\
    R_1 \\
    R_2
\end{bmatrix}
=
\begin{bmatrix}
    f(i_1) \\
    f(i_2)
\end{bmatrix}
,
$$
$\{i_1,i_2\} \subset \{1,2,3,4\}$.

Since we have three variables and only two equations, the system is classified as underdetermined, which implies that we have more than one solution for the coefficients $S,R_1,R_2 \in \text{GF}(5)$ that satisfy both equations. In probabilistic terms, for $t_1,t_2 \in \text{GF}(5)$, we observe that
\begin{align*}
    \text{Pr}[f(i_1)&=t_1,f(i_2)=t_2]=\text{Pr}[S+R_1i_1+R_2i_1^2=t_1,S+R_1i_2+R_2i_2^2=t_2]\\
    &=\text{Pr}[R_1=i_1^{-1}(t_1-S-R_2i_1^2),R_2=i_2^{-2}(t_2-S-R_1i_2^1)]\\
    &=\frac{1}{5}\frac{1}{5}=\frac{1}{25},
\end{align*}
because the variables $R_1,R_2$ are uniform at random in $\text{GF}(5)$.

In addition, for $s,t_1,t_2 \in \text{GF}(5)$ the conditional probability of the shares $f(i_1),f(i_2)$ given the secret $S$ is
\begin{align*}
    \text{Pr}[f(i_1)&=t_1,f(i_2)=t_2|S=s]=\text{Pr}[S+R_1i_1+R_2i_1^2=t_1,S+R_1i_2+R_2i_2^2=t_2|S=s]\\
    &=\text{Pr}[s+R_1i_1+R_2i_1^2=t_1,s+R_1i_2+R_2i_2^2=t_2]\\
    &=\text{Pr}[R_1=i_1^{-1}(t_1-s-R_2i_1^2),R_2=i_2^{-2}(t_2-s-R_1i_2^1)]\\
    &=\frac{1}{5}\frac{1}{5}=\frac{1}{25}.
\end{align*}

So, using Bayes theorem, it follows that
\begin{align*}
    \text{Pr}[S=s|f(i_1)=t_1,f(i_2)=t_2]&=\frac{\text{Pr}[f(i_1)=t_1,f(i_2)=t_2|S=s]\text{Pr}[S=s]}{\text{Pr}[f(i_1)=t_1,f(i_2)=t_2]}\\
    &=\frac{\frac{1}{25} \text{Pr}[S=s]}{\frac{1}{25}}\\
    &=\text{Pr}[S=s].
\end{align*}

This implies that given any two valuations $f(i_1),f(i_2)$ the value that $S$ assumes is independent, i.e., the scheme does not reveal any information about the secret $S$.

This scheme can be generalized as follows. If an individual wants to store a secret $S \in \text{GF}(q)$ with $N$ participants where at most $T$ of them $(1\leq T<N<q)$ are allowed to collude, the general $(N,T+1)$-Threshold Secret Sharing scheme that solves the problem is as follows:

- **Step 1:** Choose $R_1,...,R_T \in \text{GF}(q)$ uniformly at random.
- **Step 2:** Define the polynomial $f(x)=S+R_1x+...+R_Tx^T$ over the finite field $\text{GF}(q)$.
- **Step 3:** Send for each participant $i$ the evaluation $f(i)$ for $i \in \{1,2,...,N\}$.

Now, let us revisit the previous example in another way.

**Example 5.2:** In the same configurations of the previous example, consider a scheme that involves the following steps:

- **Step 1:** Choose $R_1,R_2 \in \text{GF}(5)$ uniformly at random.
- **Step 2:** Define the polynomial $f(x)=S+R_1x+R_2x^2$ over the finite field $\text{GF}(5)$.
- **Step 3:** Send $R_1$ and $R_2$ for participant 1 and 2, respectively. And, send $f(1)$ and $f(2)$ for participants 3 and 4, respectively. 

This scheme is curious because we do not send evaluations of $f(x)$ to the first $T=2$ participants, but note that all shares are a linear combination of the variables $S,R_1,R_2$ as we can define below:
\begin{align*}
    Y_j = c_{0j}S + c_{1j}R_1 + c_{2j} R_2
\end{align*}
where $Y_j$ represents the share sent to each participant $j \in \{1,2,3,4\}$. In this way, if we define 
\begin{align*}
    C_j = 
    \begin{bmatrix}
        c_{0j} \\
        c_{1j} \\
        c_{2j}
    \end{bmatrix}
    \in \text{GF}^3(5),
\end{align*}
we have the following vectors for each combination $Y_j$:
\begin{align*}
    C_1 = 
    \begin{bmatrix}
        0 \\
        1 \\
        0
    \end{bmatrix}
    ,
    C_2 = 
    \begin{bmatrix}
        0 \\
        0 \\
        1
    \end{bmatrix}
    ,
    C_3 = 
    \begin{bmatrix}
        1^0 \\
        1^1 \\
        1^2
    \end{bmatrix}
    ,
    C_4 = 
    \begin{bmatrix}
        2^0 \\
        2^1 \\
        2^2
    \end{bmatrix}
    .
\end{align*}

Using the definition of linear independence and the fact that the vector space $\text{GF}^3(5)$ has dimension three, any group of three vectors above are linearly independent. This implies that when we have three shares, the matrix equation
$$
\begin{bmatrix}
    C_{j_1}^t \\
    C_{j_2}^t \\
    C_{j_3}^t
\end{bmatrix}

\begin{bmatrix}
    S \\
    R_1 \\
    R_2
\end{bmatrix}
=
\begin{bmatrix}
    Y_{j_1} \\
    Y_{j_2} \\
    Y_{j_3}
\end{bmatrix}
$$
has a unique solution for $\{j_1,j_2,j_3\} \subset \{1,2,3,4\}$. So, the scheme has the Decodability property.

On the other hand, if we have only $T=2$ evaluations, we observe the following matrix equation:
$$
\begin{bmatrix}
    C_{j_1}^t \\
    C_{j_2}^t
\end{bmatrix}

\begin{bmatrix}
    S \\
    R_1 \\
    R_2
\end{bmatrix}
=
\begin{bmatrix}
    Y_{j_1} \\
    Y_{j_2}
\end{bmatrix}
$$
$\{j_1,j_2\} \subset \{1,2,3,4\}$.

Similarly to **Example 1**, we have two linearly independent combinations $Y_{i_1}$ and $Y_{i_2}$ of the three variables $S,R_1,R_2$, so the system does not have a unique solution. And, since the variables $R_1,R_2$ are uniform at random, the two results below follows:
\begin{align*}
    &\text{Pr}[Y_{i_1}=t_1,Y_{i_2}=t_2]=\frac{1}{25}, \\
    &\text{Pr}[Y_{i_1}=t_1,Y_{i_2}=t_2|S=s]=\frac{1}{25}.
\end{align*}
for $s,t_1,t_2 \in \text{GF}(5)$.

So, using Bayes theorem, it follows that
\begin{align*}
    \text{Pr}[S=s|Y_{i_1}=t_1,Y_{i_2}=t_2]&=\frac{\text{Pr}[Y_{i_1}=t_1,Y_{i_2}=t_2|S=s]\text{Pr}[S=s]}{\text{Pr}[Y_{i_1}=t_1,Y_{i_2}=t_2]}\\
    &=\frac{\frac{1}{25} \text{Pr}[S=s]}{\frac{1}{25}}\\
    &=\text{Pr}[S=s].
\end{align*}

Therefore, the scheme has the Security property. In summary, we have now concluded that this scheme is equivalent to a $(4,3)$-Threshold Secret Sharing scheme. 

In general, we can prove that this schemes are always equivalent. Therefore, another way to generalize a $(N,T+1)$-Threshold Secret Sharing scheme is as follows. If an individual wants to store a secret $S \in \text{GF}(q)$ with $N$ participants where at most $T$ of them $(1\leq T<N\leq q)$ are allowed to collude, the general $(N,T+1)$-Threshold Secret Sharing scheme that solves the problem is as follows:

- **Step 1:** Choose $R_1,...,R_T \in \text{GF}(q)$ uniformly at random.
- **Step 2:** Define the polynomial $f(x)=S+R_1x+...+R_Tx^T$ over the finite field $\text{GF}(q)$.
- **Step 3:** Send $R_j$ to each participant $j$ of the first $T$ participants. And, send $f(j)$ to each participant $T+j$ between $T+1$ and $T+(N-T)=N$.

**Remark 5.1:** Note that in the second version, the number of elements in the finite field can be equal or greater than $N$ instead of just greater, as in the first version. This comes from the fact that we only send $N-T$ evaluations of the polynomial $f(x)$, so we must have at least $N-T$ non-zero elements in $\text{GF}(q)$ ($N-T<q$).

This first part is just a short introduction to the idea of Secret Sharing in its simplest form applied to the problem of secure and distributed data storage. We will not focus on showing implementations or establish the mathematical proofs for this case, as they are presented in detail here: https://github.com/IgorAureliano/git-SSJupyter/blob/main/SSjupyterlab.ipynb. We only focus on showing implementations of Secret Sharing in conjunction with the PIR problem. So, if the reader of this notebook is interested, we highly recommend reading the other project.

In PIR scenario, we can note that the PIR schemes used in examples **4.1** and **4.2** are an application of the $(2,2)$-Threshold Secret Sharing scheme in its vector form, where the secret $S$ is equivalent to $e_i$. In general, for $N$ servers on which any $T$ can communicate, a database $D=[D_1,D_2,...,D_M]^t \in \text{GF}^M(q)$ and a desired index $i \in [M]$, the PIR scheme that follows the direct application of the $(N,T+1)$-Threshold Secret Sharing scheme can be given as follows:

- **Step 1:** Generate a $T$ uniformly at random vectors $R_1,...,R_T \in \text{GF}^M(q)$.
- **Step 2:** The user transmits the query $Q_j=R_j$ for the servers $j \in [T]$ and $Q_{T+k}=e_i+R_1x+R_2x^2+...+R_Tx^T$, $k \in [N-T]$, for server $T+k$.
- **Step 3:** Each server $j$ computes the inner product $A_j=\langle Q_j, D \rangle$ and send it to the user.
- **Step 4:** The user choose $T+1$ answers and interpolates the polynomial $g(x) = \langle e_i, D \rangle + \langle R_1, D \rangle x + \langle R_2, D \rangle x^2+ ...+ \langle R_T, D \rangle x^T$, which arises from applying the inner product properties applied to the answers, to determine $D_i=\langle e_i, D \rangle$.

**Remark 5.2:** We set $D=[D_1,D_2,...,D_M]^t \in \text{GF}^M(q)$ to emphasize the idea that the above scheme can only retrieve one symbol per application, as we showed in examples **4.1** and **4.2**.

Let us consider an example.

**Example 5.3:** Suppose a database $D=[D_1,D_2,...,D_M]^t \in \text{GF}^M(q)$ replicated on $N=4$ servers on which any $T=2$ can collude. The user wants to private access the message in the $i$-th entry. Thus, he proceeds as follows:

- **Step 1:** Generate uniformly at random vectors $R_1,R_2 \in \text{GF}^M(q)$.
- **Step 2:** The user transmits the query $Q_j=R_j$ for the servers $j \in [2]$ and $Q_{2+k}=e_i+R_1x+R_2x^2$, $k \in [2]$, for server $T+k$.
$$
\begin{aligned}
\begin{array}{|c|c|c|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 & \text{Server } 4 \\
\hline R_1 & R_2 & e_i+R_1+R_2 & e_i+2R_1+4R_2 \\
\hline
\end{array}
\end{aligned}
$$
- **Step 3:** Each server $j$ computes the inner product $A_j=\langle Q_j, D \rangle$ and send it to the user.
- **Step 4:** The user choose $T+1=2+1=3$ answers and interpolates the polynomial $g(x) = \langle e_i, D \rangle + \langle R_1, D \rangle x + \langle R_2, D \rangle x^2$ to determine $D_i=\langle e_i, D \rangle$. 

The scheme has the property of Decodability of the message due to the fact that the answers are given by combinations
\begin{align*}
    A_j = c_{0j}\langle e_i, D \rangle + c_{1j}\langle R_1, D \rangle + c_{2j} \langle R_2, D \rangle.
\end{align*}
Thus, if we define $C_j=[c_{0j},c_{1j},c_{2j}]^t \in \text{GF}^3(q)$, it follows that
\begin{align*}
    C_1 = 
    \begin{bmatrix}
        0 \\
        1 \\
        0
    \end{bmatrix}
    ,
    C_2 = 
    \begin{bmatrix}
        0 \\
        0 \\
        1
    \end{bmatrix}
    ,
    C_3 = 
    \begin{bmatrix}
        1 \\
        1 \\
        1
    \end{bmatrix}
    ,
    C_4 = 
    \begin{bmatrix}
        1 \\
        2 \\
        4
    \end{bmatrix}
    .
\end{align*}

using the definition of linear independence and the fact that the vector space $\text{GF}^3(q)$ has dimension three, any group of three vectors above are linearly independent. This implies that the matrix equation
$$
\begin{bmatrix}
    C_{j_1}^t \\
    C_{j_2}^t \\
    C_{j_3}^t
\end{bmatrix}

\begin{bmatrix}
    \langle e_i, D \rangle \\
    \langle R_1, D \rangle \\
    \langle R_2, D \rangle
\end{bmatrix}
=
\begin{bmatrix}
    A_{j_1} \\
    A_{j_2} \\
    A_{j_3}
\end{bmatrix}
$$
has a unique solution for $\{j_1,j_2,j_3\} \subset \{1,2,3,4\}$. So, the scheme can determine $D_i=\langle e_i, D \rangle$.

Now, to prove that the scheme is secure, we need to show that given $T=2$ any queries that $T=2$ servers have are independent of the value $m \in [M]$ that the desired index $i$ assumes, i.e.,
\begin{align*}
    \text{Pr}[i=m|Q_{j_1}=q_1,Q_{j_2}=q_2]=\text{Pr}[i=m],
\end{align*}
$\forall q_1,q_2 \in \text{GF}^M(q)$ and $\forall \{j_1,j_2\} \subset [4]$.

To do this, we have established that all queries are defined using the combination
\begin{align*}
    Q_j = q_{0j}e_i + q_{1j}R_1+ d_{2j}R_2
\end{align*}
where $[q_{0j},d_{1j},d_{2j}]^t \in \text{GF}^3(q)$.

In this way, since $R_1,R_2 \in \text{GF}^M(q)$ are uniformly at random, note that the joint probability of $Q_{j_1},Q_{j_2}$ is
\begin{align*}
    \text{Pr}[Q_{j_1}=r_1,Q_{j_2}=r_2]&=\text{Pr}[q_{0j_1}e_i + q_{1j_1}R_1+ q_{2j_1}R_2=r_1,q_{0j_2}e_i + q_{1j_2}R_1+ q_{2j_2}R_2=r_2]\\
    &=\text{Pr}[R_1=q_{1j_1}^{-1}(r_1-q_{0j_1}e_i-q_{2j_1}R_2),R_2=q_{2j_2}^{-1}(r_2-q_{0j_2}e_i-d_{1j_2}R_1)]\\
    &=\left(\frac{1}{q^M}\right)^2.
\end{align*}
for $r_1,r_2 \in \text{GF}^M(q)$.

Similarly, for $m \in [M]$ and $r_1,r_2 \in \text{GF}^M(q)$, the conditional probability of $Q_{j_1},Q_{j_2}$ given the desired index $i$ is
\begin{align*}
    \text{Pr}[Q_{j_1}=r_1,Q_{j_2}=r_2|i=m]&=\text{Pr}[q_{0j_1}e_i + q_{1j_1}R_1+ q_{2j_1}R_2=r_1,q_{0j_2}e_i + d_{1j_2}R_1+ d_{2j_2}R_2=r_2|i=m]\\
    &=\text{Pr}[q_{0j_1}e_m + q_{1j_1}R_1+ 1_{2j_1}R_2=r_1,1_{0j_2}e_m + q_{1j_2}R_1+ d_{2j_2}R_2=r_2]\\
    &=\text{Pr}[R_1=q_{1j_1}^{-1}(r_1-q_{0j_1}e_m-q_{2j_1}R_2),R_2=q_{2j_2}^{-1}(r_2-q_{0j_2}e_m-d_{1j_2}R_1)]\\
    &=\left(\frac{1}{q^M}\right)^2.
\end{align*}

So, using Bayes theorem, it follows that
\begin{align*}
    \text{Pr}[i=m|Q_{j_1}=r_1,Q_{j_2}=r_2]&=\frac{\text{Pr}[Q_{j_1}=q_1,Q_{j_2}=q_2|i=m]\text{Pr}[i=m]}{\text{Pr}[Q_{j_1}=q_1,Q_{j_2}=q_2]}\\
    &=\frac{\left(\frac{1}{q^M}\right)^2 \text{Pr}[i=m]}{\left(\frac{1}{q^M}\right)^2}\\
    &=\text{Pr}[i=m].
\end{align*}

To simulate this process computationally, consider $M=5$, $i=5$ and $q=7$. In this way, we can define a database $D \in \text{GF}^5(7)$ below:

In [15]:
GF7 = galois.GF(7)

D = GF7(np.array([2, 1, 5, 4, 2]))
print(D)

[2 1 5 4 2]


The user is interested in accessing the last symbol $D_5=2 \in \text{GF}(7)$. Thus, the scheme goes as follows:

- **Step 1:** Generate uniformly at random vectors $R_1,R_2 \in \text{GF}^5(7)$.

In [16]:
R_1 = GF7.Random((5))
R_2 = GF7.Random((5))

print(R_1)
print(R_2)

[6 6 0 2 6]
[1 6 3 1 3]


- **Step 2:** The user transmits the query $Q_j=R_j$ for the servers $j \in [2]$ and $Q_{2+k}=e_i+R_1x+R_2x^2$, $k \in [2]$, for server $T+k$.

In [17]:
e_i = GF7(np.eye(5, dtype=int)[4])
Q_1 = R_1
Q_2 = R_2
Q_3 = e_i + R_1 + R_2
Q_4 = e_i + GF7(2) * R_1 + GF7(4) * R_2



print("The Server 1 receives", Q_1)
print("The Server 2 receives", Q_2)
print("The Server 3 receives", Q_3)
print("The Server 4 receives", Q_4)

The Server 1 receives [6 6 0 2 6]
The Server 2 receives [1 6 3 1 3]
The Server 3 receives [0 5 3 3 3]
The Server 4 receives [2 1 5 1 4]


- **Step 3:** Each server $j$ computes the inner product $A_j=\langle Q_j, D \rangle$ and send it to the user.

In [18]:
A_1 = np.dot(D, Q_1)
A_2 = np.dot(D, Q_2)
A_3 = np.dot(D, Q_3)
A_4 = np.dot(D, Q_4)

print("The Server 1 answers with", A_1)
print("The Server 2 answers with", A_2)
print("The Server 3 answers with", A_3)
print("The Server 4 answers with", A_4)

The Server 1 answers with 3
The Server 2 answers with 5
The Server 3 answers with 3
The Server 4 answers with 0


- **Step 4:** The user choose $T+1=2+1=3$ answers and interpolates the polynomial $g(x) = \langle e_i, D \rangle + \langle R_1, D \rangle x + \langle R_2, D \rangle x^2$ to determine $D_i=\langle e_i, D \rangle$.

In this step, we will show that any three answers can retrieve the desired symbol $D_5=2$. Remember that we are trying to solve the following system
$$
\begin{bmatrix}
    C_{j_1}^t \\
    C_{j_2}^t \\
    C_{j_3}^t
\end{bmatrix}

\begin{bmatrix}
    \langle e_i, D \rangle \\
    \langle R_1, D \rangle \\
    \langle R_2, D \rangle
\end{bmatrix}
=
\begin{bmatrix}
    A_{j_1} \\
    A_{j_2} \\
    A_{j_3}
\end{bmatrix}
$$
for $\{j_1,j_2,j_3\} \subset \{1,2,3,4\}$ and
\begin{align*}
    C_1 = 
    \begin{bmatrix}
        0 \\
        1 \\
        0
    \end{bmatrix}
    ,
    C_2 = 
    \begin{bmatrix}
        0 \\
        0 \\
        1
    \end{bmatrix}
    ,
    C_3 = 
    \begin{bmatrix}
        1 \\
        1 \\
        1
    \end{bmatrix}
    ,
    C_4 = 
    \begin{bmatrix}
        1 \\
        2 \\
        4
    \end{bmatrix}
    .
\end{align*}



The only thing that changes is the definition of the matrix of coefficients that defines the system to interpolate $g(x)$.
- **1)** $A_1,A_2,A_3$:

In [19]:
b1 = GF7(np.array([A_1, A_2, A_3]))
M_coeff = GF7(np.array([[0, 1, 0], [0, 0, 1], [1, 1, 1]]))

x = np.linalg.solve(M_coeff, b1)

print('The desired message is', x[0])

The desired message is 2


- **2)** $A_1,A_2,A_4$:

In [20]:
b1 = GF7(np.array([A_1, A_2, A_4]))
M_coeff = GF7(np.array([[0, 1, 0], [0, 0, 1], [1, 2, 4]]))

x = np.linalg.solve(M_coeff, b1)

print('The desired message is', x[0])

The desired message is 2


- **3)** $A_1,A_3,A_4$:

In [21]:
b1 = GF7(np.array([A_1, A_3, A_4]))
M_coeff = GF7(np.array([[0, 1, 0], [1, 1, 1], [1, 2, 4]]))

x = np.linalg.solve(M_coeff, b1)

print('The desired message is', x[0])

The desired message is 2


- **4)** $A_2,A_3,A_4$:

In [22]:
b1 = GF7(np.array([A_2, A_3, A_4]))
M_coeff = GF7(np.array([[0, 0, 1], [1, 1, 1], [1, 2, 4]]))

x = np.linalg.solve(M_coeff, b1)

print('The desired message is', x[0])

The desired message is 2


The download rate of this scheme is given by
\begin{align*}
    \mathcal{R}=\frac{1}{3}.
\end{align*}
In general, PIR schemes that are a direct application of the $(N,T+1)$-Threshold Secret Sharing schemes have a download rate of 
\begin{align*}
    \mathcal{R}=\frac{1}{T+1},
\end{align*}
because we download $T+1$ answers/symbols to retrieve one symbol $D_i$.

Now, we will introduce the scheme called Ramp Secret Sharing to show how we can improve the download rate above.

Let us return to the Secret Sharing storage problem discussed in examples **5.1** and **5.2**. The $(N,T+1)$-Threshold Secret Sharing scheme has an interesting property that even if $N-T-1$ shares are lost/erased, it will still be possible to recover the secret $S \in \text{GF}(q)$, because the scheme only needs $T+1$ linearly independent combinations to interpolate $f(x)$. In other words, the scheme is resilient against $N-T-1$ erasures. However, suppose we do not want to be resilient to erasures. In this case, we can modify the $(N,T+1)$-Threshold Secret Sharing scheme to share more secrets by increasing the degree of $f(x)$ and placing the secrets as coefficients. This will make us less resilient to erasures, but we will be able to share more secrets securely. This idea can be generalized to the following scheme: 

- There are $N$ servers where $T$ of them collude.
- The secrets are elements $S_1,S_2,...,S_B \in \text{GF}(q)$ ($1\leq T<B<N<q$).
- The minimum threshold required to learn something about the secrets is &T+1$.
- The threshold to uniquely determine all secrets is $B+1$. 
- **Step 1:** Choose $R_1,...,R_T \in \text{GF}(q)$ uniformly at random.
- **Step 2:** Define the polynomial $f(x)=S_1+S_2x+...+S_{B-T+1}x^{B-T}+R_1^{B-T+1}+...+R_Tx^B$ over the finite field $\text{GF}(q)$.
- **Step 3:** Send for each participant $i$ the evaluation $f(i)$. 

This scheme is called $(N,B,T+1)$-Ramp Secret Sharing. An equivalent version of the above scheme would be:

For $N$ servers where $T$ collude ($T<N$), $q \in \mathbb{N}$ such that $N-T<q$ and secrets $S_1,S_2,...,S_B \in \text{GF}(q)$, the general $(N,B,T+1)$-Ramp Secret Sharing is as follows.
- **Step 1:** Choose $R_1,...,R_T \in \text{GF}(q)$ uniformly at random.
- **Step 2:** Define the polynomial $f(x)=S_1+S_2x+...+S_{B-T+1}x^{B-T}+R_1^{B-T+1}+...+R_Tx^B$ over the finite field $\text{GF}(q)$.
- **Step 3:** Send $R_j$ for each participant $j \in [T]$ and send $f(k)$ for each participant $T+k$, $k \in [N-T]$. 

**Remark 5.3:** If $B=T$, then the schemes $(N,T,T+1)$-Ramp Secret Sharing and $(N,T+1)$-Threshold Secret Sharing are equivalent.

The main difference between the schemes $(N,B,T+1)$-Ramp Secret Sharing and $(N,T+1)$-Threshold Secret Sharing is that in $(N,B,T+1)$-Ramp Secret Sharing, if $T+1\leq B$, we need more than $T+1$ linearly independent combinations of the coefficients of $f(x)$. Let us consider an example.

**Example 5.4:** Assume the same configuration as in example **5.1** where there are $N=4$ servers where any $T=2$ of them can collude. However, suppose the user wants to store two secrets $S_1,S_2 \in \text{GF}(5)$. Note that we can define the polynomial $f(x)=S_1+S_2x+R_1x^2+R_2x^3$, because we have $N=4$ servers. So, if each server holds a linearly independent combination of the coefficients of $f(x)$, we can interpolate $f(x)$ to decode $S_1$ and $S_2$. The scheme is as follows.

- **Step 1:** Choose $R_1,R_2 \in \text{GF}(5)$ uniformly at random.
- **Step 2:** Define the polynomial $f(x)=S_1+S_2x+R_1x^2+R_2x^3$ over the finite field $\text{GF}(5)$.
- **Step 3:** Send $D_{j}=R_j$ for each participant $j \in [2]$ and $D_{2+k}=f(k)$ for each participant $2+k$, $k \in [2]$.

$$
\begin{aligned}
\begin{array}{|c|c|c|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 & \text{Server } 4 \\
\hline R_1 & R_2 & S_1+S_2+R_1+R_2 & S_1+2S_2+4R_1+3R_2 \\
\hline
\end{array}
\end{aligned}
$$

To prove that the scheme has the Decodability property, i.e., the property of being able to decode $S_1$ and $S_2$, note that the user observes the following linear system when joining the four shares:

$$
\begin{bmatrix}
    0 & 0 & 1 & 0 \\
    0 & 0 & 0 & 1 \\
    1 & 1 & 1 & 1 \\
    1 & 2 & 4 & 3
\end{bmatrix}

\begin{bmatrix}
    S_1 \\
    S_2 \\
    R_1 \\
    R_2
\end{bmatrix}
=
\begin{bmatrix}
    D_1 \\
    D_2 \\
    D_3 \\
    D_4
\end{bmatrix}
$$

Since the matrix $4 \times 4$ is invertible, it follows that the system has a unique solution. Thus, we can define two deterministic functions $h_1$ and $h_2$ such that $S_1=h_1(D_1,D_2,D_3,D_4)$ and $S_2=h_2(D_1,D_2,D_3,D_4)$. Therefore, the joint distribution of the two secrets given the four shares is 
\begin{align*}
    \text{Pr}[S_1=s_1,S_2=s_2|D_1=d_1,...,D_4=d_4]=
    \begin{cases}
        1, & \text{if} \ s_j=h_j(d_1,d_2,d_3,d_4), \forall j \in [2]. \\
        0, & \text{otherwise}.
        \end{cases}
\end{align*}

The demonstration of the security property is analogous to examples **5.1** and **5.2**, i.e., when any $\{j_1,j_2\}\subset \{1,2,3,4\}$ servers talk to each other, for $s_1,s_2,d_1,d_2 \in \text{GF}(5)$, it follows that 
\begin{align*}
    &\text{Pr}[D_{j_1}=d_1,D_{j_2}=d_2]=\frac{1}{25}, \\
    &\text{Pr}[D_{j_1}=d_1,D_{j_2}=d_2|S_1=s_1,S_2=s_1]=\frac{1}{25},
\end{align*}
because $R_1,R_2 \in \text{GF}(5)$ are uniformly at random.

So, using Bayes theorem, it follows that
\begin{align*}
    \text{Pr}[S_1=s_1,S_2=s_2|D_{j_1}=d_1,D_{j_2}=d_2]&=\frac{\text{Pr}[D_{j_1}=d_1,D_{j_2}=d_2|S_1=s_1,S_2=s_2]\text{Pr}[S_1=s_1,S_2=s_2]}{\text{Pr}[D_{j_1}=d_1,D_{j_2}=d_2]}\\
    &=\frac{\frac{1}{25} \text{Pr}[S_1=s_1,S_2=s_2]}{\frac{1}{25}}\\
    &=\text{Pr}[S_1=s_1,S_2=s_2].
\end{align*}

In other words, any two shares that two servers have are independent and do not reveal any information about the secrets.

Note that the way the shares are defined depends exclusively on the fact that the combinations are linearly independent, so the matrix $4 \times 4$ that generates the shares will be invertible and the system below will have a unique solution.
$$
\begin{bmatrix}
    0 & 0 & 1 & 0 \\
    0 & 0 & 0 & 1 \\
    1 & 1 & 1 & 1 \\
    1 & 2 & 4 & 3
\end{bmatrix}

\begin{bmatrix}
    S_1 \\
    S_2 \\
    R_1 \\
    R_2
\end{bmatrix}
=
\begin{bmatrix}
    D_1 \\
    D_2 \\
    D_3 \\
    D_4
\end{bmatrix}
$$

We can therefore make different versions of share definitions. For example, another alternative way of defining shares would be as in the following table 
$$
\begin{aligned}
\begin{array}{|c|c|c|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 & \text{Server } 4 \\
\hline R_1 & R_2 & S_1+R_1+R_2 & S_2+2R_1+4R_2 \\
\hline
\end{array}
\end{aligned}
$$
because the system that defines the shares in this case is 
$$
\begin{bmatrix}
    0 & 0 & 1 & 0 \\
    0 & 0 & 0 & 1 \\
    1 & 0 & 1 & 1 \\
    0 & 1 & 2 & 4
\end{bmatrix}

\begin{bmatrix}
    S_1 \\
    S_2 \\
    R_1 \\
    R_2
\end{bmatrix}
=
\begin{bmatrix}
    D_1 \\
    D_2 \\
    D_3 \\
    D_4
\end{bmatrix}
$$
where the matrix $4\times 4$ is invertible again.

Now, let us see how this scheme can be applied to the PIR problem.

**Example 5.5:** Suppose a database $D=[D_1,D_2,...,D_M]^t \in \text{GF}^{2M}(q)$ where $D_j=[D_{j_1},D_{j_2}]^t \in \text{GF}^2(q)$ replicated on $N=4$ servers where any $T=2$ can collude. The user wants to private access the message in the $i$-th entry. Thus, he proceeds as follows:

- **Step 1:** Generate uniformly at random vectors $R_1,R_2 \in \text{GF}^{2M}(q)$.
- **Step 2:** The $D_{i_1},D_{i_2}$ are stored in the entries $2i-1,2i$ of the database $D$. So, the user transmits a query $Q_j$ for each server $j \in [4]$ as in the table below:
$$
\begin{aligned}
\begin{array}{|c|c|c|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 & \text{Server } 4 \\
\hline R_1 & R_2 & e_{2i-1}+R_1+R_2 & e_{2i}+2R_1+4R_2 \\
\hline
\end{array}
\end{aligned}
$$
- **Step 3:** Each server $j$ computes the inner product $A_j=\langle Q_j, D \rangle$ and send it to the user.
- **Step 4:** The user use all answers to decode $D_i = [\langle e_{2i-1}, D \rangle, \langle e_{2i}, D \rangle]^t=[D_{i_1},D_{i_2}]^t$.

The scheme has the property of Decodability of the message due to the fact that the user has the answers $A_1=\langle R_1, D \rangle$, $A_2=\langle R_2, D \rangle$ and can decode $D_{i_1}=\langle e_{2i-1}, D \rangle$ and $D_{i_2}=\langle e_{2i}, D \rangle$ doing the following combination:
\begin{align*}
    &A_3-A_1-A_2= \langle e_{2i-1}+R_1+R_2, D \rangle - \langle R_1, D \rangle - \langle R_2, D \rangle=\langle e_{2i-1}, D \rangle=D_{i_1}\\
    &A_4-2A_1-4A_2= \langle e_{2i}+2R_1+4R_2, D \rangle - 2\langle R_1, D \rangle - 4\langle R_2, D \rangle=\langle e_{2i}, D \rangle=D_{i_2}
\end{align*}

Thus, for $d_1,d_2,a_1,...,a_4 \in \text{GF}(q)$, it follows that
\begin{align*}
    \text{Pr}[D_{i_1}=d_1,D_{i_2}=d_2|A_1=a_1,...,A_4=a_4]=
    \begin{cases}
        1, & \text{if} \ d_1=a_3-a_1-a_2 \text{ and }d_2=a_4-2a_1-4a_2. \\
        0, & \text{otherwise}.
        \end{cases}
\end{align*}

To prove that the scheme is secure, it is enough to note that every query can be written as a combination
\begin{align*}
    Q_j = c_1^je_{2i-1}+c_2^je_{2i}+c_3^jR_1+c_4^jR_2,
\end{align*}
where the subindex denotes the coefficients associated with $Q_j$.

As defined in **Step 2**, this combinations are linearly independent. So, for $\{k_1,k_2\} \subset \{1,2,3,4\}$, $q_1,q_2 \in \text{GF}(q)$ and $i \in [M]$, it follows that
\begin{align*}
    &\text{Pr}[Q_{k_1}=q_1,Q_{k_2}=q_2]=\left(\frac{1}{q^{2M}}\right)^2, \\
    &\text{Pr}[Q_{k_1}=q_1,Q_{k_2}=q_2|i=m]=\left(\frac{1}{q^{2M}}\right)^2,
\end{align*}
because $R_1,R_2 \in \text{GF}^{2M}(q)$ are uniformly at random.

So, using Bayes theorem, it follows that
\begin{align*}
    \text{Pr}[i=m|Q_{k_1}=q_1,Q_{k_2}=q_2]&=\frac{\text{Pr}[Q_{k_1}=q_1,Q_{k_2}=q_2|i=m]\text{Pr}[i=m]}{\text{Pr}[Q_{k_1}=q_1,Q_{k_2}=q_2]}\\
    &=\frac{\left(\frac{1}{q^{2M}}\right)^2 \text{Pr}[i=m]}{\left(\frac{1}{q^{2M}}\right)^2}\\
    &=\text{Pr}[i=m].
\end{align*}


Considering $M=4$, $q=7$ and $i=2$, we can define a database

In [23]:
q = 7
i = 2
M = 4
N = 4
T = 2
GF7 = galois.GF(7)

D = GF7(np.array([1,0,4,2,1,6,6,5]))
print(D)

[1 0 4 2 1 6 6 5]


So, the desired message is $D_2=[4, 2]^t$ and we can check the computational process as follows:

- **Step 1:** Generate uniformly at random vectors $R_1,R_2 \in \text{GF}^{2M}(q)$.

In [24]:
R_1 = GF7.Random((2*M))
R_2 = GF7.Random((2*M))

print(R_1)
print(R_2)

[0 4 0 3 0 6 0 4]
[4 1 5 3 4 6 6 6]


- **Step 2:** The $D_{j_1},D_{j_2}$ are stored in the entries $2i-1,2i$ of the database $D$. So, the user transmits a query $Q_j$ for each server $j \in [4]$ as in the table below:
$$
\begin{aligned}
\begin{array}{|c|c|c|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 & \text{Server } 4 \\
\hline R_1 & R_2 & e_{2i-1}+R_1+R_2 & e_{2i}+2R_1+4R_2 \\
\hline
\end{array}
\end{aligned}
$$

In [25]:
e_1 = GF7(np.eye(2*M, dtype=int)[2*(i-1)])
e_2 = GF7(np.eye(2*M, dtype=int)[2*i-1])
Q_1 = R_1
Q_2 = R_2
Q_3 = e_1 + R_1 + R_2
Q_4 = e_2 + GF7(2) * R_1 + GF7(4) * R_2



print("The Server 1 receives", Q_1)
print("The Server 2 receives", Q_2)
print("The Server 3 receives", Q_3)
print("The Server 4 receives", Q_4)

The Server 1 receives [0 4 0 3 0 6 0 4]
The Server 2 receives [4 1 5 3 4 6 6 6]
The Server 3 receives [4 5 6 6 4 5 6 3]
The Server 4 receives [2 5 6 5 2 1 3 4]


- **Step 3:** Each server $j$ computes the inner product $A_j=\langle Q_j, D \rangle$ and send it to the user.

In [26]:
A_1 = np.dot(D, Q_1)
A_2 = np.dot(D, Q_2)
A_3 = np.dot(D, Q_3)
A_4 = np.dot(D, Q_4)

print("The Server 1 answers with", A_1)
print("The Server 2 answers with", A_2)
print("The Server 3 answers with", A_3)
print("The Server 4 answers with", A_4)

The Server 1 answers with 6
The Server 2 answers with 3
The Server 3 answers with 6
The Server 4 answers with 5


- **Step 4:** The user use all answers to decode $D_i = [\langle e_{2i-1}, D \rangle, \langle e_{2i}, D \rangle]^t=[D_{i_1},D_{i_2}]^t$.

In this step, note that we want to solve the system
$$
\begin{bmatrix}
    0 & 0 & 1 & 0 \\
    0 & 0 & 0 & 1 \\
    1 & 0 & 1 & 1 \\
    0 & 1 & 2 & 4
\end{bmatrix}

\begin{bmatrix}
    \langle e_{2i-1}, D \rangle \\
    \langle e_{2i}, D \rangle \\
    \langle R_1, D \rangle \\
    \langle R_2, D \rangle
\end{bmatrix}
=
\begin{bmatrix}
    A_1 \\
    A_2 \\
    A_3 \\
    A_4
\end{bmatrix}
.
$$

So,

In [27]:
b = GF7(np.array([A_1, A_2, A_3, A_4]))
M_coeff = GF7(np.array([[0, 0, 1, 0], [0, 0, 0, 1], [1, 0, 1, 1], [0, 1, 2, 4]]))

x = np.linalg.solve(M_coeff, b)

print('The desired message is', x[0:2])

The desired message is [4 2]


Since we download four symbols $A_1,A_2,A_3,A_4$ to retrieve two symbols $D_{i_1},D_{i_2}$, the download rate of this scheme is given by
\begin{align*}
    \mathcal{R}=\frac{2}{4}=\frac{1}{2}.
\end{align*}

Note that the download rate of $\frac{1}{2}$ is an improvement on the rate of $\frac{1}{3}$ determined in example 5.3.

In the original PIR formulation introduced by Chor, Goldreich and Kushilevitz [1,2], we consider that there will be no erasures. Therefore, we can always consider an $(N,N-1,T+1)$-Ramp Secret Sharing scheme that will retrieve $N-T$ symbols in an application to the PIR problem. 

Thus, before presenting a general formulation of the theoretical and computational model, let us consider a definition and a lemma that will help us in this process.

**Definition 5.1:** Let $a_1,...,a_n$ be distinct elements of a field $\mathbb{F}$ with at least $n$ elements. The Vandermonde matrix $V=V(a_1,...,a_n)$ is defined by
$$
V=V(a_1,...,a_n)=
\begin{bmatrix}
    1 & a_1 & a_1^2 & \cdots & a_1^{n-1} \\
    1 & a_2 & a_2^2 & \cdots & a_2^{n-1} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    1 & a_n & a_n^2 & \cdots & a_n^{n-1} \\
\end{bmatrix}
$$

The main advantage of the Vandermonde matrix is that it is always invertible for non-zero and distinct coefficients $a_1,a_2,...,a_n$. This result can be stated by the following lemma.

**Lemma 3.1:** If $a_1,...,a_n \in \mathbb{F}\backslash\{0\}$ are distinct elements, then the Vandermonde matrix $V=V(a_1,...,a_n)$ is invertible and
\begin{align*}
    \det(V) = \prod_{1 \leq i < j \leq n} (a_j - a_i).
\end{align*}

Through the formula, note that the determinant is always different from zero, because we are making the product of subtractions $(a_j-a_i)$ and, as $a_1,...,a_n$ are distinct, any $(a_j-a_i)$ cannot be zero, i.e., $\det(V)\neq 0$.

**Remark 5.4:** For the Vandermonde matrix $V(a_1,...,a_n)$ be invertible, we need that the field $\mathbb{F}$ have at least $n+1$ elements, because any square matrix with a row or column of zeros is not invertible.

**Remark 5.5:** The proof of the lemma can be obtained by doing Gauss elimination.

It is important to emphasize that there are many ways to define and generalize an $(N,N-1,T+1)$-Ramp Secret Sharing scheme for PIR, because the definition only needs to have the Decodability and Security properties. We will now present one of them, which will serve as the basis for presenting the Refine and Lift techniques in the next section.

Suppose that there are $N$ servers where $T$ of them can collude ($1\leq T<N$). Each server has a copy of database $D=[D_1,D_2,...,D_M]^t \in \text{GF}^{M(N-T)}(q)$ where each message is a vector of size $N-T$, i.e., $D_j=[D_{j_1},D_{j_2},...,D_{j_{N-T}}]^t \in \text{GF}^{(N-T)}(q)$ for $j \in [M]$ and $N-T<q$.

The user wants private access the message $D_i=[D_{i_1},D_{i_2}...,D_{i_{N-T}}]^t$ in the $i$-th entry without revealing any information about which message is being accessed to any server. The algotithm that solves the problem is as follows.

**Algorithm 1** $(N,N-1,T+1)$-Ramp Secret Sharing for PIR.

- **Step 1:** Generate $T$ uniformly at random vectors $R_1,R_2,...,R_T \in \text{GF}^{M(N-T)}(q)$.
- **Step 2:** The user transmits a query $Q_j^i=R_j$ for the servers $j \in [T]$ and $Q_{T+k}^i= e_{(N-T)(i-1)+k} + R_{T+k}$ for the servers $T+k$, $k \in [N-T]$, where
\begin{align*}
    R_{T+k} \doteq \sum_{l=0}^{T-1} k^l R_{l+1}.
\end{align*}
    
- **Step 3:** Each server $j$ computes the inner product $A_j=\langle Q_j, D \rangle$ and send it to the user.
- **Step 4:** The user decodes the $N-T$ symbols of $D_i$ defining $N-T$ combinations $Y_{T+k}$ such that
\begin{align*}
        \begin{bmatrix}
            1 & 1 & \cdots & 1^{T-1} \\
            1 & 2 & \cdots & 2^{T-1} \\
            \vdots & \vdots & \ddots & \vdots \\
            1 & N-T & \cdots & (N-T)^{T-1} \\
        \end{bmatrix}
        \begin{bmatrix}
            A_{1} \\
            A_{2} \\
            \vdots \\
            A_{T}
        \end{bmatrix}
        \doteq
        \begin{bmatrix}
            Y_{T+1} \\
            Y_{T+2} \\
            \vdots \\
            Y_{N}
        \end{bmatrix}
    \end{align*}
to subtract each $Y_{T+k}$ from $A_{T+k}$ and get each symbol $\langle e_{(N-T)(i-1)+k}, D \rangle=D_{i_k}$ for $k \in [N-T]$.

So, we need to prove that this scheme has the following properties:

- **(Decodabilty)** For $a_1,...,a_N \in \text{GF}(q)$, $d_i \in \text{GF}^{N-T}(q)$ and a deterministic function $f$ such that $f(a_1,...,a_N)=d_i$, it follows that 
\begin{align*}
    \text{Pr}[D_i=d_i|A_1=a_1,...,A_N=a_N]=
    \begin{cases}
        1, & \text{if} \ d_i=f(a_1,...,a_N). \\
        0, & \text{otherwise}.
        \end{cases}
\end{align*}
- **(Security)** For $m \in [M]$, $q_1,...,q_T \in \text{GF}^{M(N-T)}(q)$ and $\{j_1,j_2,...,j_T\} \subset [N]$, it follows that
\begin{align*}
    \text{Pr}[i=m|Q_{j_1}^i=q_1,...,Q_{j_T}^i=q_T]=\text{Pr}[i=m].
\end{align*}

**Proof:** Let us begin by demonstrating the Decodability property. Using **Step 4**, note that all $Y_{T+k}$ for $k \in [N-T]$ are functions of $A_1,A_2,...,A_T$ and $A_{T+k}-Y_{T+k}=D_{i_k}$. Thus, we can denote each $Y_{T+k}$ by $Y_{T+k}(A_1,...,A_T)$ and define a deterministic function $f$ such that $f(A_1,...,A_N)=D_i$ as follows:
\begin{align*}
    f(A_1,...,A_N)\doteq 
    \begin{bmatrix}
        A_{T+1}-Y_{T+1}(A_1,...,A_T)\\
        A_{T+2}-Y_{T+2}(A_1,...,A_T) \\
        \vdots \\
        A_{N}-Y_{N}(A_1,...,A_T)
    \end{bmatrix}
    =
    \begin{bmatrix}
        D_{i_1} \\
        D_{i_2} \\
        \vdots \\
        D_{i_{N-T}}
    \end{bmatrix}
    =D_i.
\end{align*}

Therefore, for $a_1,...,a_N \in \text{GF}(q)$ and $d_i \in \text{GF}^{N-T}(q)$, it follows that
\begin{align*}
    \text{Pr}[D_i=d_i|A_1=a_1,...,A_N=a_N]=
    \begin{cases}
        1, & \text{if} \ d_i=f(a_1,...,a_N). \\
        0, & \text{otherwise}.
        \end{cases}
\end{align*}

Now, we prove Security. First, note that any query can be written by a combination
\begin{align*}
    Q_j^i = \sum_{k=1}^{N-T} \lambda_{k}^j e_{(N-T)(i-1)+k} + \sum_{l=N-T+1}^{N} \lambda_{l}^j R_{l-N+T}
\end{align*}
where, for $\lambda^j \doteq [\lambda^j_1,\lambda^j_2,...,\lambda^j_N]^t \in \text{GF}^N(q)$, we have
\begin{align*}
    \lambda^j = e_{N-T+j} \in \text{GF}^N(q) \text{ for }j \in [T]
\end{align*}
and 
\begin{align*}
    \lambda^{T+j} = e_{j} + 
    \begin{bmatrix}
        \mathbf{0}_{N-T} \\
        j^0 \\
        j^1 \\
        \vdots \\
        j^{T-1} 
    \end{bmatrix}
    \in \text{GF}^N(q) \text{ for }j \in [N-T].
\end{align*}

Since $\{e_1,e_2,...,e_N\}$ is a basis of $\text{GF}^N(q)$, it follows that $\lambda^{1},\lambda^{2},..., \lambda^{N}$ are linearly independent. So, define
\begin{align*}
    X^j_p(i) = (\lambda_{l}^j)^{-1} (Q_j^i - \lambda_{p+N-T}^j R_{p})
\end{align*}
for $j \in [N]$ and $p \in [T]$. 

In this way, for $\{j_1,j_2,...,j_T\} \subset [N]$ and $q_1,q_2,...,q_T \in \text{GF}^{M(N-T)}(q)$, the joint probability of $Q_{j_1},Q_{j_2},...,Q_{j_T}$ is
\begin{align*}
    \text{Pr}[Q_{j_1}^i=q_1,...,Q_{j_T}^i=q_T]&=\text{Pr}[R_1=(\lambda_{N-T+1}^{j_1})^{-1}q_1-X_1^{j_1}(i),...,R_T=(\lambda_{N}^{j_T})^{-1}q_T-X_T^{j_T}(i)]\\
    &=\underbrace{\frac{1}{q^{M(N-T)}}...\frac{1}{q^{M(N-T)}}}_{\text{$T$ times}}=\left( \frac{1}{q^{M(N-T)}}\right)^T,
\end{align*}
because $R_1,R_2,...,R_T$ are uniformly at random in $\text{GF}^{M(N-T)}(q)$.

Similarly, for $m \in [M]$, the conditional probability of $Q_{j_1},Q_{j_2},...,Q_{j_T}$ given $i$ is
\begin{align*}
    \text{Pr}[Q_{j_1}^i=q_1,...,Q_{j_T}^i=q_T|i=m]&=\text{Pr}[R_1=(\lambda_{N-T+1}^{j_1})^{-1}q_1-X_1^{j_1}(m),...,R_T=(\lambda_{N}^{j_T})^{-1}q_T-X_T^{j_T}(m)]\\
    &=\underbrace{\frac{1}{q^{M(N-T)}}...\frac{1}{q^{M(N-T)}}}_{\text{$T$ times}}=\left( \frac{1}{q^{M(N-T)}}\right)^T.
\end{align*}

Therefore, using Bayes theorem, it follows that
\begin{align*}
    \text{Pr}[i=m|Q_{j_1}^i=q_1,...,Q_{j_T}^i=q_T]&=\frac{\text{Pr}[Q_{j_1}^i=q_1,...,Q_{j_T}^i=q_T|i=m]\text{Pr}[i=m]}{\text{Pr}[Q_{j_1}^i=q_1,...,Q_{j_T}^i=q_T]}\\
    &=\frac{\left( \frac{1}{q^{M(N-T)}}\right)^T \text{Pr}[i=m]}{\left( \frac{1}{q^{M(N-T)}}\right)^T}\\
    &=\text{Pr}[i=m].
\end{align*}



                                                                                                                                            
$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

Since we have $N-T$ desired symbols $D_{i_1},D_{i_2},...,D_{i_{N-T}}$ of $N$ symbols $A_1,A_2,...,A_N$ downloaded, the download rate $\mathcal{R}$ for such a scheme is
\begin{align*}
    \mathcal{R}=\frac{N-T}{N}=1-\frac{T}{N}.
\end{align*}

Note that for $1\leq T<N$, it follows that 
\begin{align*}
    \frac{1}{T+1} \leq \frac{N-T}{N}
\end{align*}
where the equality is satisfied only if $N=T+1$. In other words, this scheme is an improvement on the $(N,T+1)$-Secret Sharing scheme for PIR.


The general algorithm code for this case involves four functions, but there is no need to consider all four separately. The reason for separating them is mainly to explain the main points of the code.

First, we consider a function that will create all the vectors that will be attributed to the variables $e_{(N-T)(i-1)+k}$ for $k \in [N-T]$ and $R_j$ for $j \in [T]$ to be stored in a list. The function receives the number of servers $N$, the maximum number of servers that can collude $T$, the desired index $i$, the number of messages $M$ and the number of elements in the finite field $q$.

In [28]:
import galois
import numpy as np

#Ramp Secret Sharing

# Generate a list with the keys e_{(N-T)(i-1)+1},...,e_{i(N-T)}, R_1,R_2,...,R_T
def generate_keys_ramp(N, T, i, M, q):
    GF = galois.GF(q)                   # i is the desired message
    L = N-T
    l = []
    for j in range((i-1)*L, i*L):
        l.append(GF(np.eye(M*L, dtype=int)[j]))
    for j in range(1, T+1):
        l.append(GF.Random((M*L)))
    return l

For example, if $N=4$, $T=2$, $i=3$, $M=3$ and $q=19$:

In [29]:
print(generate_keys_ramp(4,2,3,3,19))

[GF([0, 0, 0, 0, 1, 0], order=19), GF([0, 0, 0, 0, 0, 1], order=19), GF([ 3, 18,  7, 12,  6, 18], order=19), GF([ 6,  9,  6, 11, 14,  5], order=19)]


Next, we created a function to compute the combinations of vectors $R_1,R_2,...,R_T$ based on the Vandermonde matrix $V(1,2,...,N-T)$. In this function we receive a parameter $x \in [N-T]$ which is the coefficient that defines a row of the Vandermonde matrix and the list of all keys defined by the previous function.

In [30]:
#Ramp Secret Sharing

# Generate the combinations the combinations of R_1,R_2,...,R_T in the queries
def evaluate_poly_ramp(N, T, M, vectors, x, q): #x i
    GF = galois.GF(q)
    L = N-T
    evaluate = GF(np.zeros(M*L, dtype=int))
    for degree, value in enumerate(vectors):
        evaluate += value * (GF(x) ** degree)
    return evaluate

With the above function, we define the function that will be used to generate the queries. In this function, we receive the input parameters $N,T,i,M,q$ and generate a list with each query labeled by the server number $j \in [N]$ as output.

In [31]:
#Ramp Secret Sharing

# Generate a tuple with all queries labeled by the server enumeration
def generate_queries_ramp(N, T, i, M, q):
    vectors = generate_keys_ramp(N, T, i, M, q)

    shares = []

    k = 0 # label
    for j in range(N-T, N):
        k += 1
        shares.append((k, vectors[j]))

    k = 0 
    for j in range(T+1, N+1):
        k += 1
        shares.append((j, evaluate_poly_ramp(N, T, M, vectors, k, q)))
    return tuple(shares)

In our previous example, we have

In [32]:
x = generate_queries_ramp(4, 2, 3, 3, 19)
print(x)

((1, GF([ 4, 14, 12, 12,  6,  2], order=19)), (2, GF([11, 13,  8,  4,  0,  7], order=19)), (3, GF([15,  8,  1, 16,  7, 10], order=19)), (4, GF([ 9,  8, 17,  4,  6,  9], order=19)))


Finally, we present a function that decodes the scheme through the answers that each server will compute using the previous list.

In [33]:
#Ramp Secret Sharing

# Compute the answers from each server and decodes the desired message if possible
def decodability_ramp(DB, qrylist, N, T, q):
    GF = galois.GF(q)

    A = []
    for j in range(N):
        query = qrylist[j][1]
        answ = np.dot(query, DB)
        A.append((j+1, answ))
    k = 0
    i = 1
    Mtrxcoeff = []
    for p in range(N):
        if k <= T-1:
            Mtrxcoeff.append(GF(np.eye(N, dtype=int)[N-T+k]))
        else:
            Mtrxcoeff.append([GF(i)**j for j in range(N)])
            i += 1
        k += 1
    evaluation_list = []
    for i in range(len(A)):
        evaluation_list.append(A[i][1])
    sol = np.linalg.solve(GF(np.array(Mtrxcoeff)), GF(np.array(evaluation_list)))
    desired_message = []
    for i in range(N-T):
        desired_message.append(sol[i])

    return print('Message reconstructed:', GF(np.array(desired_message)))

To continue with our example, let us define a random database:

In [34]:
GF = galois.GF(19)
DB = GF.Random((3*(4-2)))
print(DB)

[10  3 12 15 10  8]


Thus, we decode the message $D_3 \in \text{GF}^2(19)$ as follows:

In [35]:
decodability_ramp(DB, x, 4, 2, 19)

Message reconstructed: [10  8]


In 2016, Hua Sun and Syed A. Jafar[9, 10] determined the maximum possible download rate, called Capacity, that a PIR scheme can achieve for finite parameters $N,T,M$. This rate is given by 
\begin{align*}
    \mathcal{C}=\frac{(N-T)N^{M-1}}{N^M-T^M}.
\end{align*}

First, note that we can conclude that the $(N,N-1,T+1)$-Ramp Secret Sharing scheme is asymptotically optimal, because
\begin{align*}
    \lim_{\substack{M \to \infty}} \frac{(N-T)N^{M-1}}{N^M-T^M} = \frac{N-T}{N}
\end{align*}
where $N,T,M$ are positive integers.

In addition, observe that the capacity $\mathcal{C}$ depends on the number of $M$ messages from the database replicated on the $N$ servers and $\mathcal{R}=\frac{N-T}{N}$ does not. Thus, we still have the objective of presenting a scheme that has a download rate $\mathcal{R}=\mathcal{C}$. This objective will be achieved through the Refine and Lift techniques that we will present in the next section. In summary, the process involves using a Ramp Secret Sharing scheme as input, applying the Refine technique to optimize the download rate for $M=2$ messages and applying the Lift technique to generalize the improvement to the $M$ case.

## 6. Refine and Lift

The Refine and Lift techniques are part of a recursive process. In order, the steps are to take a Ramp Secret Sharing scheme that solves the problem with a rate $\frac{N-T}{N}$, applying the Refine technique to optimize the download rate for $M=2$ messages and applying the Lift technique to generalize the improvement to the $M$ case. We will see that the techniques are based on the following principles:
- Enforcing message symmetry by querying all the messages the same number of times.
- Making useful use of queries that do not query the desired message.

We will see in detail what these two principles are all about.

### 6.1. Refine

Let us start the presentation of Refine with a simple example.

**Example 6.1:** Consider $N = 2$ servers that do not collude ($T=1$) and each has a copy of the database $D=[D_1,D_2]^t \in \text{GF}^4(2)$ where $D_1,D_2 \in \text{GF}^2(2)$. The user wants private access to the message $D_i$ for $i \in [2]$.

One can directly apply the a $(2,1,2)$-Ramp Secret Sharing scheme representing the database in a similar way to example **4.2** to obtain a PIR scheme which solves the problem with a download rate of $\frac{1}{2}$. However, consider the refined scheme in the table below:

$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline a_1 & a_2+b_2  \\
 b_1 &  \\
\hline
\end{array}
\end{aligned}
$$

Without loss of generality, let us consider the desired index $i=1$, i.e., the user desires to retrieve the message $D_1$. In this case, the queries are chosen as follows.

- Choose two queries $a_1,a_2 \in \text{span}(e_1,e_2) \subset \text{GF}^4(2)$ uniformly at random and linearly independent.
- Choose a query $b_1 \in \text{span}(e_3,e_4) \subset \text{GF}^4(2)$ uniformly at random and linearly independent. In this case, it is enough to consider $b_1 \neq 0$.
- Choose $b_2=b_1$.

In this way, the previous table becomes as follows:
$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline a_1 & a_2+b_1  \\
 b_1 &  \\
\hline
\end{array}
\end{aligned}
$$

Note the similarity between the $(2,1,2)$-Ramp Secret Sharing scheme and its refined version. The way we choose the random key $R$ is to keep $e_i$ secure and the definition of the query $e_i+R$ is done in such a way that we can make use of $R$ for decoding.
$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline R & e_i+R  \\
\hline
\end{array}
\end{aligned}
$$

So, the user sends each of the above queries to the respective servers and the servers compute the inner products $A_1^1=\langle a_1, D\rangle$, $A_1^2=\langle b_1, D\rangle$ and $A_2^1=\langle a_2+b_1, D\rangle$ of the queries with the database and send it back to the user. 

The scheme has the Decodability property, because we can obtain two linearly independent combinations $A_1^1=\langle a_1, D\rangle$ and $A_2^1-A_1^2=\langle a_2+b_1, D\rangle-\langle b_1, D\rangle=\langle a_2, D\rangle$ of the two symbols of $D_1$ due to the fact that $a_1,a_2$ are linearly independent in $\text{span}(e_1,e_2) \subset \text{GF}^4(2)$. Thus, we can solve a linear system over $\text{GF}(2)$ which has a unique solution to determine the symbols of $D_1$.

Furthermore, the scheme has the Security property, because each server receives the same number of queries that query each message. Note that queries $a_1,a_2$ always query the first message, because when we do the inner product, due to the definition of queries on the space $\text{span}(e_1,e_2) \subset \text{GF}^4(2)$, we get two linearly independent combinations of the symbols of the first message. Similarly, $b_1$ query the second message because $b_1$ is linearly independent in $\text{span}(e_3,e_4) \subset \text{GF}^4(2)$. The fact that the vectors are linearly independent is important, because it guarantees that the user is trying to query the messages the same number of times. 

For example, suppose that $b_1 = 0$, i.e., $b_1$ is linearly dependent. In this case, we would not be able to obtain a useful combination of the symbols of $D_2$ which is equivalent to not consulting the $D_2$ symbols. This implies that we are not interested in $D_2$, so both servers would know that we are interested in the index $i=1$, which would compromise the security of the scheme. On the order hand, if we chose linearly dependent $a_1,a_2$ we would have a similar problem if $a_1=0$ and/or $a_2=0$ and if $a_1=ca_2$ for $c \in \text{GF}(2)$, we would have the problem that the combinations $A_1^1=\langle a_1, D\rangle$ and $A_2^1-A_1^2=\langle a_2, D\rangle$ would be linearly dependent, i.e., we would not be able to determine the symbols of $D_1$.

Similarly, for $i=2$, the scheme is given by the table below:
$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline a_1 & a_1+b_2  \\
 b_1 &  \\
\hline
\end{array}
\end{aligned}
$$

In this case, the queries are chosen as follows.

- Choose two queries $a_1 \in \text{span}(e_1,e_2) \subset \text{GF}^4(2)$ uniformly at random and linearly independent. In this case, it is enough to consider $a_1 \neq 0$.
- Choose a query $b_1,b_2 \in \text{span}(e_3,e_4) \subset \text{GF}^4(2)$ uniformly at random and linearly independent.

Again, let us demonstrate that the scheme has the properties of Decodability and Security, and show an example of how the process can be viewed computationally.

We begin by demonstrating the Decodability property. For $i=1$, we have $A_1^1=\langle a_1, D\rangle$ and $A_2^1-A_1^2=\langle a_2, D\rangle$. Thus, since $a_1,a_2$ are linearly independent in $\text{span}(e_1,e_2) \subset \text{GF}^4(2)$, the system
\begin{align*}
        \begin{bmatrix}
            a_1^1 & a_1^2\\
            a_2^1 & a_2^2
        \end{bmatrix}
        \begin{bmatrix}
            D_1^1 \\
            D_1^2
        \end{bmatrix}
        =
        \begin{bmatrix}
            \langle a_1, D\rangle \\
            \langle a_2, D\rangle
        \end{bmatrix}
    \end{align*}
has a unique solution where $a_j \doteq [a_j^1,a_j^2,0,0]^t$.

This implies that we can define a deterministic function $f_1$ such that
\begin{align*}
    f_1(A_1^1,A_1^2, A_2^1)=
    \begin{bmatrix}
        D_{1}^1 \\
        D_{1}^2 
    \end{bmatrix}
    =D_1.
\end{align*}

Therefore, for $r_1,r_2,r_3 \in \text{GF}(2)$ and $d_1 \in \text{GF}^{2}(2)$, it follows that
\begin{align*}
    \text{Pr}[D_1=d_1|A_1^1=r_1,A_1^2=r_2,A_2^1=r_3]=
    \begin{cases}
        1, & \text{if} \ d_1=f_1(r_1,r_2,r_3). \\
        0, & \text{otherwise}.
        \end{cases}
\end{align*}

Similarly, for $i=2$, we have $A_1^2=\langle b_1, D\rangle$ and $A_2^1-A_1^1=\langle b_2, D\rangle$. Thus, since $b_1,b_2$ are linearly independent in $\text{span}(e_3,e_4) \subset \text{GF}^4(2)$, the system
\begin{align*}
        \begin{bmatrix}
            b_1^1 & b_1^2\\
            b_2^1 & b_2^2
        \end{bmatrix}
        \begin{bmatrix}
            D_2^1 \\
            D_2^2
        \end{bmatrix}
        =
        \begin{bmatrix}
            \langle b_1, D\rangle \\
            \langle b_2, D\rangle
        \end{bmatrix}
    \end{align*}
has a unique solution where $b_j \doteq [0,0,b_j^1,b_j^2]^t$.

This implies that we can define a deterministic function $f_2$ such that
\begin{align*}
    f_2(A_1^1,A_1^2, A_2^1)=
    \begin{bmatrix}
        D_{2}^1 \\
        D_{2}^2 
    \end{bmatrix}
    =D_2.
\end{align*}

Therefore, for $s_1,s_2,s_3 \in \text{GF}(2)$ and $d_2 \in \text{GF}^{2}(2)$, it follows that
\begin{align*}
    \text{Pr}[D_2=d_2|A_1^1=s_1,A_1^2=s_2,A_2^1=s_3]=
    \begin{cases}
        1, & \text{if} \ d_2=f_2(s_1,s_2,s_3). \\
        0, & \text{otherwise}.
        \end{cases}
\end{align*}

Now, we prove Security. First, using the Iverson Bracket notation defined in section 2, define for a vector $v$ at any the following statements
\begin{align*}
        &[G_1(v)]\doteq
        \begin{cases}
            1, & \text{if }v \in \text{span}(e_1,e_2) \subset \text{GF}^4(2). \\
            0, & \text{otherwise}.
        \end{cases}
        \\
        &[G_2(v)]\doteq
        \begin{cases}
            1, & \text{if }v \in \text{span}(e_3,e_4) \subset \text{GF}^4(2). \\
            0, & \text{otherwise}.
        \end{cases}
        \\
        &[L(v)]\doteq
        \begin{cases}
            1, & \text{if $v$ is linearly independent}. \\
            0, & \text{otherwise}.
        \end{cases}
    \end{align*}

Note that since $\text{span}(e_1,e_2) \subset \text{GF}^4(2)$ and $\text{span}(e_3,e_4) \subset \text{GF}^4(2)$ only assume entries from $\text{GF}(2)$, the cardinality of each space is four. However, as the queries are linearly independent, they cannot assume the zero vector, i.e., the observation of each query separately has three possible realizations. With that in mind, let us consider each case separately.

Given $i=1$, Server 1 observe $a_1$ and $b_1$ with the following conditional distributions
\begin{align*}
    &\text{Pr}[a_1=r|i=1]=\frac{[L(r)]\cdot [G_1(r)]}{3},\\
    &\text{Pr}[b_1=s|i=1]=\frac{[L(s)]\cdot [G_2(s)]}{3}.
\end{align*}

On the other hand, if $i=2$, Server 1 observe $a_1$ and $b_1$ with the following conditional distributions
\begin{align*}
    &\text{Pr}[a_1=r|i=2]=\frac{[L(r)]\cdot [G_1(r)]}{3},\\
    &\text{Pr}[b_1=s|i=2]=\frac{[L(s)]\cdot [G_2(s)]}{3}.
\end{align*}

In this way, using the definition of conditional probability, we can determine the marginal probability of $a_1$ and $b_2$ separately as follows:
\begin{align*}
    \text{Pr}[a_1=r]&=\sum_{m \in \{1,2\}} \text{Pr}[a_1=r,i=m]\\
    &=\sum_{m \in [2]} \text{Pr}[a_1=r|i=m]\text{Pr}[i=m]\\
    &=\frac{[L(r)]\cdot [G_1(r)]}{3}\left( \sum_{m \in [2]} \text{Pr}[i=m]\right)\\
    &=\frac{[L(r)]\cdot [G_1(r)]}{3},\\
    \text{Pr}[b_1=s]&=\sum_{m \in [2]} \text{Pr}[b_1=s,i=m]\\
    &=\sum_{m \in [2]} \text{Pr}[b_1=s|i=m]\text{Pr}[i=m]\\
    &=\frac{[L(s)]\cdot [G_2(s)]}{3}\left( \sum_{m \in [2]} \text{Pr}[i=m]\right)\\
    &=\frac{[L(s)]\cdot [G_2(s)]}{3},
\end{align*}
where the passages follow due to the fact that $a_1$ and $b_1$ are uniformly at random in their respective vector spaces.

Thus, for $m \in [2]$, we have that
\begin{align*}
    &\text{Pr}[a_1=r|i=m]=\frac{[L(r)]\cdot [G_1(r)]}{3}=\text{Pr}[a_1=r]\\
    &\text{Pr}[b_1=s|i=m]=\frac{[L(s)]\cdot [G_2(s)]}{3}=\text{Pr}[b_1=s].
\end{align*}

Observe that we want the joint probability of $a_1$ and $b_1$, since Server 1 observes both at the same time. Thus, note that the vector spaces $\text{span}(e_1,e_2) \subset \text{GF}^4(2)$ and $\text{span}(e_3,e_4) \subset \text{GF}^4(2)$ are complementary, which implies that the choice of $a_1$ does not change the choice of $b_1$, i.e., $a_1$ and $b_1$ are statistically independent. So,
\begin{align*}
    \text{Pr}[a_1=r,b_1=s|i=m]=\frac{[L(r)]\cdot [L(s)] \cdot [G_1(r)] \cdot [G_2(s)]}{9}=\text{Pr}[a_1=r,b_1=s].
\end{align*}

Finally, using Bayes theorem, we conclude that
\begin{align*}
    \text{Pr}[i=m|a_1=r,b_1=s]&=\frac{\text{Pr}[a_1=r,b_1=s|i=m]\ \text{Pr}[i=m]}{\text{Pr}[a_1=r,b_1=s]}\\
    &=\frac{\text{Pr}[a_1=r,b_1=s]\ \text{Pr}[i=m]}{\text{Pr}[a_1=r,b_1=s]}\\
    &=\text{Pr}[i=m].
\end{align*}

Similarly, Server 2 observes the query $a_2+b_1$ when $i=1$ and $a_1+b_2$ when $i=2$. So, define $Q_2^i$ such that
\begin{align*}
    Q_2^1=\{a_2,b_1\} \text{ and }Q_2^2=\{a_1,b_2\}.
\end{align*}

The reason for considering the definition of the sums separately comes from the fact that servers understand $a_*+b_*$ as each query separately, because each one query has a different message.

In this way, we can make an analogous analysis to the case of Server 1 in order to obtain the following results
\begin{align*}
    \text{Pr}[Q_2^i=\{r,s\}|i=m]=\frac{[L(r)]\cdot [L(s)] \cdot [G_1(r)] \cdot [G_2(s)]}{9}=\text{Pr}[Q_2^i=\{r,s\}].
\end{align*}

Therefore, using Bayes theorem, we conclude that
\begin{align*}
    \text{Pr}[i=m|Q_2^i=\{r,s\}]&=\frac{\text{Pr}[Q_2^i=\{r,s\}|i=m]\ \text{Pr}[i=m]}{\text{Pr}[Q_2^i=\{r,s\}]}\\
    &=\frac{\text{Pr}[Q_2^i=\{r,s\}]\ \text{Pr}[i=m]}{\text{Pr}[Q_2^i=\{r,s\}]}\\
    &=\text{Pr}[i=m].
\end{align*}

Since we have two desired symbols $D_i^1,D_i^2 \in \text{GF}(2)$ of three downloaded $A^1_1,A^1_2,A^2_1 \in \text{GF}(2)$, the download rate $\mathcal{R}$ for such a scheme is
\begin{align*}
    \mathcal{R}=\frac{2}{3}
\end{align*}
This is an improvement on the Ramp Secret Sharing scheme that solves this problem, because $\frac{1}{2}<\frac{2}{3}$ and it is the capacity $\mathcal{C}$ for $N=2$, $T=1$ and $M=2$.

To exemplify the process computationally, consider $i=1$. Thus, a possible database would be

In [36]:
GF = galois.GF(2)
DB = GF(np.array([1,1,1,0]))
print(DB)

[1 1 1 0]


We will assume here that the queries assume specific values, but we will present a way of generating them all uniformly at random and linearly independent in their respective spaces later. Thus, for example, we can have the following queries in the scheme:

In [37]:
a_1 = GF(np.array([1,1,0,0]))
a_2 = GF(np.array([1,0,0,0]))
b_1 = GF(np.array([0,0,1,0]))

print('Server 1 receives', a_1,'and',b_1)
print('Server 2 receives', a_2+b_1)


Server 1 receives [1 1 0 0] and [0 0 1 0]
Server 2 receives [1 0 1 0]


Next, the servers compute the answers and send back to the user:

In [38]:
A_11 = np.dot(DB, a_1)
A_12 = np.dot(DB, b_1)
A_21 = np.dot(DB, a_2+b_1)

print('The user receives', A_11, 'and',A_12, 'from Server 1 and', A_21,'from Server 2')

The user receives 0 and 1 from Server 1 and 0 from Server 2


Finally, the user decodes the desired message $D_1=[1,1]^t$ by solving the linear system
\begin{align*}
        \begin{bmatrix}
            a_1^1 & a_1^2\\
            a_2^1 & a_2^2
        \end{bmatrix}
        \begin{bmatrix}
            D_1^1 \\
            D_1^2
        \end{bmatrix}
        =
        \begin{bmatrix}
            \langle a_1, D\rangle \\
            \langle a_2, D\rangle
        \end{bmatrix}
    \end{align*}.

In [39]:
M_coef = GF(np.array([[1,1],[1,0]])) 
b = GF(np.array([A_11, A_21-A_12]))

x = np.linalg.solve(M_coef, b)

print('The desired message is', x)

The desired message is [1 1]


We saw in the previous example that the proof of Security can be long for Refine. In general, they are if we analyze each case separately. So, we will introduce two definitions that will help us demonstrate the Security of the Refine and Lift schemes in a shorter way.

**Definition 6.1:** Let $\text{GF}(q)$ be a finite field with $q$ elements. Then, the General Linear Group of degree $n$, denoted by $(\text{GL}_n(q), \cdot)$, is the group of all $n \times n$ invertible matrices in $\text{GF}^{n \times n}(q)$ with multiplication operation, i.e.,
\begin{align*}
    \text{GL}_n(q)= \{M \in \text{GF}^{n \times n}(q): \det(M)\neq 0\}.
\end{align*}

The interesting fact about the General Linear Group $(\text{GL}_n(q), \cdot)$ is that its cardinality is finite and is given by the following lemma.

**Lemma 6.1:** The number of elements in $\text{GL}_n(q)$ is
\begin{align*}
    (q^n-1)(q^n-q)(q^n-q^2)\ ...\ (q^n-q^{n-1})=\prod_{j=0}^{n-1} (q^n-q^j).
\end{align*}

This is useful because we can choose uniform distributions of discrete random variables in $\text{GL}_n(q)$.

**Definition 6.2:** Let $M \in \text{GF}^{n \times n}(q)$ be expressed by
\begin{align*}
    M= 
    \begin{bmatrix}
        m_1 & m_2 & \cdots & m_n
    \end{bmatrix}
    ,
\end{align*}
where $m_j \in \text{GF}^{n}(q)$ is column vector for $j \in [n]$.

In addition, let a subset of indices $\mathcal{I} \doteq \{j_1,j_2,...,j_k\} \subseteq [n]$. So the column function with relation to $\mathcal{I}$ is a function $\text{col}_\mathcal{I}:\text{GF}^{n \times n}(q) \rightarrow \text{GF}^{k \times n}(q)$ such that
\begin{align*}
        M=
        \begin{bmatrix}
            m_1 & m_2 & \cdots & m_n
        \end{bmatrix}
        \mapsto 
        \text{col}_\mathcal{I}(M) = 
        \begin{bmatrix}
            m_{j_1} & m_{j_2} & \cdots & m_{j_k}
        \end{bmatrix}
        .
    \end{align*}

Now, let us consider an example with collusion.

**Example 6.2:** Suppose that there are $N = 3$ servers where any $T=2$ servers collude and each has a copy of database $D=[D_1,D_2]^t \in \text{GF}^6(3)$ where $D_1,D_2 \in \text{GF}^3(3)$. Furthermore, suppose that the user wants to retrieve the first file $D_i\doteq[D_i^1,D_i^2,D_i^3]^t$ for $i \in \{1,2\}$ without revealing interest in the $i$-th entry of the database.
One can directly apply the $(3,2,3)$-Ramp Secret Sharing scheme to obtain a PIR scheme which solves the problem with a download rate of $\frac{1}{3}$ as in the table below.

$$
\begin{aligned}
\begin{array}{|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 \\
\hline R_1 & R_2 & e_i+R_1+R_2  \\
\hline
\end{array}
\end{aligned}
$$

However, consider the refined scheme in the table below:

$$
\begin{aligned}
\begin{array}{|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 \\
\hline a_1 & a_2 & a_3+b_3  \\
 b_1 & b_2 & \\
\hline
\end{array}
\end{aligned}
$$

The queries are chosen as follows.

If $i=1$:
- Choose two queries $a_1,a_2,a_3 \in \text{span}(e_1,e_2,e_3) \subset \text{GF}^6(3)$ uniformly at random and linearly independent.
- Choose a query $b_1,b_2 \in \text{span}(e_4,e_5,e_6) \subset \text{GF}^6(3)$ uniformly at random and linearly independent.
- Choose $b_3=b_1+b_2$.

$$
\begin{aligned}
\begin{array}{|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 \\
\hline a_1 & a_2 & a_3+b_1+b_2  \\
 b_1 & b_2 & \\
\hline
\end{array}
\end{aligned}
$$

If $i=2$:
- Choose two queries $a_1,a_2 \in \text{span}(e_1,e_2,e_3) \subset \text{GF}^6(3)$ uniformly at random and linearly independent.
- Choose a query $b_1,b_2,b_3 \in \text{span}(e_4,e_5,e_6) \subset \text{GF}^6(3)$ uniformly at random and linearly independent.
- Choose $a_3=a_1+a_2$.

$$
\begin{aligned}
\begin{array}{|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 \\
\hline a_1 & a_2 & a_1+a_2+b_3  \\
 b_1 & b_2 & \\
\hline
\end{array}
\end{aligned}
$$

Again, note the similarity between the $(3,2,3)$-Ramp Secret Sharing scheme. The queries that do not query the desired message are used to reinforce the symmetry of the queries for each message and are useful for decoding the queries that do query the desired message. In this way, we can obtain all the combinations needed to decode the desired symbols through the answers.

In this way, the user sends each of the above queries to the respective servers and the servers compute the inner products $A_1^1=\langle a_1, D\rangle$, $A_1^2=\langle b_1, D\rangle$, $A_2^1=\langle a_2, D\rangle$, $A_2^2=\langle b_2, D\rangle$ and $A_3^1(i)=\langle a_3+b_3, D\rangle$ of the queries with the database and send it back to the user. We denote $A_3^1(i)$, because the $a_3+b_3$ depends on the value of the desired index as we defined earlier, i.e., $A_3^1(1)=\langle a_3+b_1+b_2, D\rangle$ and $A_3^1(2)=\langle a_1+a_2+b_3, D\rangle$.

Let us proof that such a scheme has the properties of Decodability and Security.

We begin by demonstrating the Decodability property. For $i=1$, we have $A_1^1=\langle a_1, D\rangle$ and $A_2^1=\langle a_2, D\rangle$ and $A_3^1(1)-A_1^2-A_2^2=\langle a_3, D\rangle$. Thus, since $a_1,a_2,a_3$ are linearly independent in $\text{span}(e_1,e_2,e_3) \subset \text{GF}^6(3)$, the system
\begin{align*}
        \begin{bmatrix}
            a_1^1 & a_1^2 & a_1^3\\
            a_2^1 & a_2^2 & a_2^3\\
            a_3^1 & a_3^2 & a_3^3
        \end{bmatrix}
        \begin{bmatrix}
            D_1^1 \\
            D_1^2 \\
            D_1^3 
        \end{bmatrix}
        =
        \begin{bmatrix}
            \langle a_1, D\rangle \\
            \langle a_2, D\rangle \\
            \langle a_3, D\rangle \\
        \end{bmatrix}
    \end{align*}
has a unique solution where $a_j \doteq [a_j^1,a_j^2,a_j^3,0,0,0]^t$.

This implies that we can define a deterministic function $f_1$ such that
\begin{align*}
    f_1(A_1^1,A_1^2, A_2^1,A_2^2,A_3^1(1))=
    \begin{bmatrix}
        D_{1}^1 \\
        D_{1}^2 \\
        D_{1}^3
    \end{bmatrix}
    =D_1.
\end{align*}

Therefore, for $r_1,r_2,...,r_5 \in \text{GF}(3)$ and $d_1 \in \text{GF}^{3}(3)$, it follows that
\begin{align*}
    \text{Pr}[D_1=d_1|A_1^1=r_1,A_1^2=r_2,...,A_3^1(1)=r_5]=
    \begin{cases}
        1, & \text{if} \ d_1=f_1(r_1,r_2,...,r_5). \\
        0, & \text{otherwise}.
        \end{cases}
\end{align*}

Similarly, for $i=2$, we have $A_1^2=\langle b_1, D\rangle$, $A_2^2=\langle b_2, D\rangle$ and $A_3^1(2)-A_1^1-A_2^1=\langle b_3, D\rangle$. Thus, since $b_1,b_2,b_3$ are linearly independent in $\text{span}(e_4,e_5,e_6) \subset \text{GF}^6(3)$, the system
\begin{align*}
        \begin{bmatrix}
            b_1^1 & b_1^2 & b_1^3\\
            b_2^1 & b_2^2 & b_2^3 \\
            b_3^1 & b_3^2 & b_3^3
        \end{bmatrix}
        \begin{bmatrix}
            D_2^1 \\
            D_2^2 \\
            D_2^3
        \end{bmatrix}
        =
        \begin{bmatrix}
            \langle b_1, D\rangle \\
            \langle b_2, D\rangle \\
            \langle b_3, D\rangle 
        \end{bmatrix}
    \end{align*}
has a unique solution where $b_j \doteq [0,0,0,b_j^1,b_j^2,b_j^3]^t$.

This implies that we can define a deterministic function $f_2$ such that
\begin{align*}
    f_2(A_1^1,A_1^2, A_2^1,A_2^2,A_3^1(2))=
    \begin{bmatrix}
        D_{2}^1 \\
        D_{2}^2 \\
        D_2^3
    \end{bmatrix}
    =D_2.
\end{align*}

Therefore, for $s_1,s_2,...,s_5 \in \text{GF}(3)$ and $d_2 \in \text{GF}^{3}(3)$, it follows that
\begin{align*}
    \text{Pr}[D_2=d_2|A_1^1=s_1,A_1^2=s_2,...,A_3^1(2)=s_5]=
    \begin{cases}
        1, & \text{if} \ d_2=f_2(s_1,s_2,s_3). \\
        0, & \text{otherwise}.
        \end{cases}
\end{align*}

Now, we prove Security. Initially, using Iverson Bracket notation, define for any matrix $M$ the following statemants:
\begin{align*}
        &[G_1(M)]\doteq
        \begin{cases}
            1, & \text{if all column vectors of $M$ are in }\text{span}(e_1,e_2,e_3) \subset \text{GF}^6(3). \\
            0, & \text{otherwise}.
        \end{cases}
        \\
        &[G_2(M)]\doteq
        \begin{cases}
            1, & \text{if all column vectors of $M$ are in } \text{span}(e_4,e_5,e_6) \subset \text{GF}^6(3). \\
            0, & \text{otherwise}.
        \end{cases}
        \\
        &[L(M)]\doteq
        \begin{cases}
            1, & \text{if all column vectors of $M$ are linearly independent}. \\
            0, & \text{otherwise}.
        \end{cases}
    \end{align*}

Note that any $T=2$ servers observe two queries of type $a_*$ and two queries of type $b_*$. Thus, we can interpret that any $T=2$ observes, for $\{k_1,k_2\} \subset [3]$ and $i \in [2]$, the following matrices

\begin{align*}
    \mathcal{A}^i \doteq
    \begin{bmatrix}
        a_{k_1}(i) & a_{k_2}(i)
    \end{bmatrix}
    \ \
    \mathcal{B}^i \doteq
    \begin{bmatrix}
        b_{k_1}(i) & b_{k_2}(i)
    \end{bmatrix}
\end{align*}
where
\begin{align*}
    &a_{1}(1)=a_1=a_{1}(2),\\
    &a_{2}(1)=a_2=a_{2}(2),\\
    &a_{3}(1)=a_3, \ a_{1}(2)=a_1+a_2,\\
    &b_{1}(1)=b_1=b_{1}(2),\\
    &b_{2}(1)=b_2=b_{2}(2),\\
    &b_{3}(1)=b_1+b_2, \ b_{1}(2)=b_3.
\end{align*}

In this way, since the queries are uniformly at random and linearly independent in their respective spaces based on the value that the desired index $i \in [2]$ assumes, any pair of queries $a_{k_1}(i),a_{k_2}(i)$ and $b_{k_1}(i),b_{k_2}(i)$ are uniformly at random and linearly independent. Therefore, since the vectors of type $a_*$ and $b_*$ assume zero in the last three entries or in the first three entries, we can interpret the distributions of $\mathcal{A}^i$ and $\mathcal{B}^i$ as the observation of a submatrix $3\times 2$ in $\text{GL}_3(3)$, i.e., for $\mathcal{I}\doteq \{k_1,k_2\}$,

\begin{align*}
    &\text{Pr}[\mathcal{A}^i=r]=\frac{[G_1(r)]\cdot [L(r)]}{|\text{col}(\text{GL}_3(3))|},\\
    &\text{Pr}[\mathcal{B}^i=s]=\frac{[G_2(s)]\cdot [L(s)]}{|\text{col}(\text{GL}_3(3))|}.
\end{align*}

Now, since the vector spaces $\text{span}(e_1,e_2,e_3) \subset \text{GF}^6(3)$ and $\text{span}(e_4,e_5,e_6) \subset \text{GF}^6(3)$ are complementary, the distributions of $\mathcal{A}^i$ and $\mathcal{B}^i$ do not influence each other, which implies that $\mathcal{A}^i$ and $\mathcal{B}^i$ are statistically independent. So,
\begin{align*}
    \text{Pr}[\mathcal{A}^i=r,\mathcal{B}^i=s]=\left( \frac{[G_1(r)]\cdot [G_2(s)]\cdot [L(r)] \cdot [L(s)]}{|\text{col}(\text{GL}_3(3))|} \right)^2.
\end{align*}

In addition, observe that the distribution of $\mathcal{A}^i$ and $\mathcal{B}^i$ only depend on the conditions $G_1(M), G_2(M)$ and $L(M)$ regardless of the value that $i$ assumes, so it follows that
\begin{align*}
    \text{Pr}[\mathcal{A}^i=r,\mathcal{B}^i=s|i=m]=\left( \frac{[G_1(r)]\cdot [G_2(s)]\cdot [L(r)] \cdot [L(s)]}{|\text{col}(\text{GL}_3(3))|} \right)^2=\text{Pr}[\mathcal{A}^i=r,\mathcal{B}^i=s].
\end{align*}
for $m \in [M]$.

Therefore, using Bayes theorem, we conclude that
\begin{align*}
    \text{Pr}[i=m|\mathcal{A}^i=r,\mathcal{B}^i=s]&=\frac{\text{Pr}[\mathcal{A}^i=r,\mathcal{B}^i=s|i=m]\ \text{Pr}[i=m]}{\text{Pr}[\mathcal{A}^i=r,\mathcal{B}^i=s]}\\
    &=\frac{\text{Pr}[\mathcal{A}^i=r,\mathcal{B}^i=s]\ \text{Pr}[i=m]}{\text{Pr}[\mathcal{A}^i=r,\mathcal{B}^i=s]}\\
    &=\text{Pr}[i=m].
\end{align*}

Since we have two desired symbols $D_i^1,D_i^2,D_i^3 \in \text{GF}(3)$ of five downloaded $A^1_1,A^1_2,A^2_1,A_2^2,A_3^1 \in \text{GF}(3)$, the download rate $\mathcal{R}$ for such a scheme is
\begin{align*}
    \mathcal{R}=\frac{3}{5}
\end{align*}
This is an improvement on the Ramp Secret Sharing scheme that solves this problem, because $\frac{1}{3}<\frac{3}{5}$ and it is the capacity $\mathcal{C}$ for $N=3$, $T=2$ and $M=2$.

To exemplify the process computationally, consider $i=2$. Thus, a possible database would be

In [40]:
GF = galois.GF(3)
DB = GF(np.array([1,1,1,0,2,2]))
print(DB)

[1 1 1 0 2 2]


Again, we will assume here that the queries assume specific values, but we will present a way of generating them all uniformly at random and linearly independent in their respective spaces later. Thus, for example, we can have the following queries in the scheme:

In [41]:
a_1 = GF(np.array([2,1,0,0,0,0]))
a_2 = GF(np.array([1,0,0,0,0,0]))
b_1 = GF(np.array([0,0,0,2,0,0]))
b_2 = GF(np.array([0,0,0,1,1,0]))
b_3 = GF(np.array([0,0,0,0,2,1]))

print('Server 1 receives', a_1,'and',b_1)
print('Server 2 receives', a_2,'and',b_2)
print('Server 3 receives', a_1+a_2+b_3)

Server 1 receives [2 1 0 0 0 0] and [0 0 0 2 0 0]
Server 2 receives [1 0 0 0 0 0] and [0 0 0 1 1 0]
Server 3 receives [0 1 0 0 2 1]


Next, the servers compute the answers and send back to the user:

In [42]:
A_11 = np.dot(DB, a_1)
A_12 = np.dot(DB, b_1)
A_21 = np.dot(DB, a_2)
A_22 = np.dot(DB, b_2)
A_31 = np.dot(DB, a_1+a_2+b_3)

print('The user receives', A_11, 'and',A_12, 'from Server 1,', A_21, 'and',A_22, 'from Server 2 and', A_31,'from Server 3')

The user receives 0 and 0 from Server 1, 1 and 2 from Server 2 and 1 from Server 3


Finally, the user decodes the desired message $D_2=[0,2,2]^t$ by solving the linear system
\begin{align*}
        \begin{bmatrix}
            b_1^1 & b_1^2 & b_1^3\\
            b_2^1 & b_2^2 & b_2^3 \\
            b_3^1 & b_3^2 & b_3^3
        \end{bmatrix}
        \begin{bmatrix}
            D_2^1 \\
            D_2^2 \\
            D_2^3
        \end{bmatrix}
        =
        \begin{bmatrix}
            \langle b_1, D\rangle \\
            \langle b_2, D\rangle \\
            \langle b_3, D\rangle
        \end{bmatrix}
    \end{align*}.

In [43]:
M_coef = GF(np.array([[2,0,0],[1,1,0],[0,2,1]])) 
b = GF(np.array([A_12, A_22, A_31-A_11-A_21]))

x = np.linalg.solve(M_coef, b)

print('The desired message is', x)

The desired message is [0 2 2]


In the proof of the Security property in the previous example, we used the argument that the distributions of the matrices $\mathcal{A}^i$ and $\mathcal{B}^i$ could be interpreted as a $3 \times 2$ submatrix of $\text{GL}_3(3)$. In this example, this arises from the fact that given a vector $v=[v^1,v^2,v^3]^t \in \text{GF}^3(3)$, we can create an equivalence between the vectors 
\begin{align*}
    v=
    \begin{bmatrix}
        v^1\\
        v^2\\
        v^3
    \end{bmatrix}
    \in \text{GF}^3(3), \
    v_a \doteq
    \begin{bmatrix}
        v^1\\
        v^2\\
        v^3 \\
        0 \\
        0 \\
        0
    \end{bmatrix}
    \in \text{span}(e_1,e_2,e_3) \subset \text{GF}^6(3)
    \text{ and }
    v_b \doteq
    \begin{bmatrix}
        0 \\
        0 \\
        0 \\
        v^1\\
        v^2\\
        v^3
    \end{bmatrix}
    \in \text{span}(e_4,e_5,e_6) \subset \text{GF}^6(3).
\end{align*}

Thus, when we place $v$ as a column vector of a submatrix of $\text{GL}_3(3)$, $v$ must be linearly independent, which implies that $v_a$ and $v_b$ will also be linearly independent.

We will always use analogous arguments to prove the Security property. Thus, it is necessary to establish this relation in an arbitrary way. In general, the Refine technique can retrieve $N$ symbols, which implies that vectors of type $a_*$ are in $\text{span}(e_1,... ,e_N) \subset \text{GF}^{2N}(q)$ and $b_*$ are in $\text{span}(e_{N+1},...,e_{2N}) \subset \text{GF}^{2N}(q)$, as we will see in Refine's generalization. Furthermore, we will prove later that the Lift technique can recover $N^{M-1}$ symbols, which implies que os vetores estarão em espaços $\text{span}(e_{(k-1)N^{M-1}+1},e_{(k-1)N^{M-1}+2},...,e_{kN^{M-1}}) \subset \text{GF}^{MN^{M-1}}(q)$ for $k \in [M]$. But, note that the Refine is the Lift particular case $M=2$. Therefore, we will state the lemma in a way that can be used in both cases.  

**Lemma 6.2:** Let $X$ be a matrix in $\text{GF}^{m\times n}$ and $X[i,j]$ be the entry of $X$ in row $i$ and column $j$. In addition, for $k \in [M]$, define the Iverson Bracket functions 
\begin{align*}
        &[G_k^M(X)]=
        \begin{cases}
            1, & \text{if all column vectors of $X$ are in }\text{span}(e_{(k-1)N^{M-1}+1},e_{(k-1)N^{M-1}+2},...,e_{kN^{M-1}}) \subset \text{GF}^{MN^{M-1}}(q). \\
            0, & \text{otherwise}.
        \end{cases}
        \\
        &[L(X)]=
        \begin{cases}
            1, & \text{if all column vectors of $X$ are linearly independent}. \\
            0, & \text{otherwise}.
        \end{cases}
    \end{align*}
    
and let
\begin{align*}
    C_k^n(M) \doteq \{X \in \text{GF}^{m\times n}: [G_k^M(X)]=1,[L(X)]=1\}
    .
\end{align*}

Then, for $\mathcal{I} \subseteq [N^{M-1}]$ such that $|\mathcal{I}|=n$, there is a bijective function $f:C_k^n(M) \rightarrow \text{col}_\mathcal{I}(\text{GL}_{N^{M-1}}(q))$ that maps any matrix $X \in C_k^n(M)$ into a matrix $Y \in \text{col}_\mathcal{I}(\text{GL}_{N^{M-1}}(q))$ with entries
\begin{align*}
    Y[i,j]=X[(k-1)N^{M-1}+i,j]
\end{align*}
for $i \in [N^{M-1}]$ and $j \in [n]$.

**Proof:** Let $X \in C_k^n(M)$ and the function $f:C_k^n(M) \rightarrow \text{col}_\mathcal{I}(\text{GL}_{N^{M-1}}(q))$ such that
\begin{align*}
    f(X)[i,j]=X[(k-1)N^{M-1}+i,j]
\end{align*}
for $i \in [N^{M-1}]$ and $j \in [n]$.

Since $G_k^M(X)=1$, all column vectors of $X$ are in $\text{span}(e_{(k-1)N^{M-1}+1},e_{(k-1)N^{M-1}+2},...,e_{kN^{M-1}}) \subset \text{GF}^{MN^{M-1}}(q)$. In other words, all entries of $X$ outside the block of rows $[(k-1)N^{M-1}+1, kN^{M-1}] \subseteq \mathbb{N}$ are zero. Consequently, since $L(X) = 1$ and these columns are fully contained within the block of rows $[(k-1)N^{M-1}+1, kN^{M-1}]$, it follows that the submatrix $Y$ formed by the rows $[(k-1)N^{M-1}+1, kN^{M-1}]$ also has linearly independent columns, i.e., $L(Y)=1$.

But, note that
\begin{align*}
    Y[i,j]=X[(k-1)N^{M-1}+i,j]=f(X)[i,j] \implies Y=f(X).
\end{align*}

Thus, using the definition of the general linear group $\text{GL}_{N^{M-1}}(q)$,
\begin{align*}
    L(f(X))=1 \implies f(X) \in \text{col}_\mathcal{I}(\text{GL}_{N^{M-1}}(q)),
\end{align*}
i.e., $f$ is surjective.

Now, suppose two matrices $X_1,X_2 \in C_k^n(M)$ such that $f(X_1)=f(X_2)$. This implies that 
\begin{align*}
    X_1[(k-1)N^{M-1}+i,j]=f(X_1)[i,j]=f(X_2)[i,j]=X_2[(k-1)N^{M-1}+i,j].
\end{align*}
for $i \in [N^{M-1}]$ and $j \in [n]$.

But, since $G_k(X_1)=G_k(X_2)=1$ and $L(X_1)=L(X_2)=1$, we have that $X_1=X_2$, i.e., $f$ is injective.

Therefore, since $f$ is surjective and injective, we can conclude that $f$ is bijective.

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

With this lemma established, let us present the general case of Refine.

A user wants to retrieve a message $D_i=[D_i^1,...,D_i^N]^t \in \text{GF}^{N}(q)$ privately from a database $D=[D_1,D_2]^t \in \text{GF}^{2N}(q)$ replicated on $N$ servers with any $T$ colluding $(T< q)$. The user proceeds as follows.

**Algorithm 2:** $(N,T)$-Refined Scheme.

- **Step 1:** Take a $(N,N-1,T+1)$-Ramp Secret Sharing scheme as input:
    $$
    \begin{aligned}
    \begin{array}{|c|c|c|c|c|c|}
    \hline \text{Server } 1 & ... & \text{Server } T &\text{Server } T+1 & ... & \text{Server } N \\
    \hline R_1 & ... & R_T & e_1+R_{T+1} & ... & e_{N-T}+R_N  \\
    \hline
    \end{array}
    \end{aligned}
    $$
where, for $k \in [N-T]$,
\begin{align*}
    R_{T+k} \doteq \sum_{l=0}^{T-1} k^l R_{l+1}.
\end{align*}
    
- **Step 2:** If $i=1$:
    - Choose $N$ queries $a_1,...,a_N \in \text{span}(e_1,...,e_N) \subset \text{GF}^{2N}(q)$ uniformly at random and linearly independent.
    - Choose $T$ queries $b_1,...,b_T \in \text{span}(e_{N+1},...,e_{2N}) \subset \text{GF}^{2N}(q)$ uniformly at random and linearly independent.
    
    If $i=2$:
    - Choose $T$ queries $a_1,...,a_T \in \text{span}(e_1,...,e_N) \subset \text{GF}^{2N}(q)$ uniformly at random and linearly independent.
    - Choose $N$ queries $b_1,...,b_N \in \text{span}(e_{N+1},...,e_{2N}) \subset \text{GF}^{2N}(q)$ uniformly at random and linearly independent.
    
- **Step 3:** The user send the queries according to the table below:
    $$
    \begin{aligned}
    \begin{array}{|c|c|c|c|c|c|}
    \hline \text{Server } 1 & ... & \text{Server } T &\text{Server } T+1 & ... & \text{Server } N \\
    \hline a_1 & ... & a_T & a_{T+1}+b_{T+1} & ... & a_{N}+b_{N}  \\
     b_1 & ... & b_T &  &  &  \\
    \hline
    \end{array}
    \end{aligned}
    $$
    where, if $i=1$, $b_{T+1},b_{T+2},...,b_N$ are defined by 
\begin{align*}
    b_{T+k} \doteq \sum_{l=0}^{T-1} k^l b_{l+1}
\end{align*}
    and, if $i=2$, $a_{T+1},a_{T+2},...,a_N$ are defined by
\begin{align*}
    a_{T+k} \doteq \sum_{l=0}^{T-1} k^l a_{l+1},
\end{align*}    
for $k \in [N-T]$.

- **Step 4:** For $j \in \{1,2,...,T\}$, server $j$ answers with $A_j^1=\langle a_j, D \rangle$ and $A_j^2=\langle b_j, D \rangle$. For $l \in \{T+1,T+2,...,N\}$, server $l$ answers with $A_l^1=\langle a_l+b_l, D \rangle$.

Again, note that any $T$ servers observe $T$ queries of type $a_*$ the first message and two queries of type $b_*$ the second message. Thus, we can interpret that any $T$ observes, for $\{k_1,k_2,...,k_T\} \subset [N]$ and $i \in [2]$, the matrices

\begin{align*}
    \mathcal{A}^i \doteq
    \begin{bmatrix}
        a_{k_1}(i) & a_{k_2}(i) & ... & a_{k_T}(i)
    \end{bmatrix}
    \ \text{and  }\ 
    \mathcal{B}^i \doteq
    \begin{bmatrix}
        b_{k_1}(i) & b_{k_2}(i) & ... & b_{k_T}(i)
    \end{bmatrix}
\end{align*}
where the value that the desired index $i \in [2]$ determines the definition of each query as in Step 2 and 3.

Thus, using the definitions of $G_k^M(X),L(X)$ and $C_k^n(M)$ in **Lemma 6.2**, we need to prove the following results:

- **(Decodabilty)** For $j \in [2]$, $r_1^j,...,r_T^j, r_{T+1}^1, ...,r_N^1 \in \text{GF}(q)$, $d_i \in \text{GF}^{N}(q)$ and a deterministic function $f$ such that $f(r_1^j,...,r_N^1)=d_i$,
\begin{align*}
    \text{Pr}[D_i=d_i|A_1^j=r_1^j,...,A_N^1=r_N^1]=
    \begin{cases}
        1, & \text{if} \ d_i=f(r_1^j,...,r_N^1). \\
        0, & \text{otherwise}.
        \end{cases}
\end{align*}

- **(Security)** For $m \in [2]$, $q_1 \in C_1^T(2) \subset \text{GF}^{2N\times T}(q)$ and $q_2 \in C_2^T(2) \subset \text{GF}^{2N\times T}(q)$, it follows that
\begin{align*}
    \text{Pr}[i=m|\mathcal{A}^i=q_1,\mathcal{B}^i=q_2]=\text{Pr}[i=m].
\end{align*}

**Proof:** If $i=1$, note that the user has the value of $A_1^1=\langle a_1,D\rangle$,$A_2^1=\langle a_2,D\rangle$,...,$A_T^1=\langle a_T,D\rangle$. Furthermore, the user can decode $\langle a_{T+k},D\rangle$ for $k \in [N-T]$ by doing
\begin{align*}
    A_{T+k}^1-\sum_{l=0}^{T-1} k^l A_{l+1}^2=\langle a_{T+k}+b_{T+k},D\rangle-\sum_{l=0}^{T-1} k^l \langle b_{l+1},D\rangle=\langle a_{T+k},D\rangle.
\end{align*}

Thus, since $a_1,a_2,...,a_n$ are linearly independent in $\text{span}(e_1,...,e_N) \subset \text{GF}^{2N}(q)$, the system
\begin{align*}
        \begin{bmatrix}
            a_1^1 & a_1^2 & \cdots & a_1^N\\
            a_2^1 & a_2^2 & \cdots & a_2^N\\
            \vdots & \vdots & \ddots & \vdots \\
            a_N^1 & a_N^2 & \cdots & a_N^N
        \end{bmatrix}
        \begin{bmatrix}
            D_1^1 \\
            D_1^2 \\
            \vdots \\
            D_1^N 
        \end{bmatrix}
        =
        \begin{bmatrix}
            \langle a_1, D\rangle \\
            \langle a_2, D\rangle \\
            \vdots \\
            \langle a_N, D\rangle \\
        \end{bmatrix}
    \end{align*}
has a unique solution, where $a_j \doteq [a_j^1,a_j^2,...,a_j^N,0,0,...,0]^t$.

This implies that we can define a deterministic function $f_1$ such that
\begin{align*}
    f_1(A_1^j,A_2^j,...,A_{T+1}^1,...,A_{N}^1)=
    \begin{bmatrix}
        D_{1}^1 \\
        D_{1}^2 \\
        \vdots \\
        D_{1}^N
    \end{bmatrix}
    =D_1,
\end{align*}
for $j \in [2]$.

Therefore, for $j \in [2]$, $r_1^j,...,r_T^j, r_{T+1}^1, ...,r_N^1 \in \text{GF}(q)$, $d_1 \in \text{GF}^{N}(q)$, it follows that
\begin{align*}
    \text{Pr}[D_1=d_1|A_1^j=r_1^j,...,A_N^1=r_N^1]=
    \begin{cases}
        1, & \text{if} \ d_1=f_1(r_1^j,...,r_N^1). \\
        0, & \text{otherwise}.
        \end{cases}
\end{align*}

If $i=2$, note that the user has the value of $A_1^2=\langle b_1,D\rangle$,$A_2^2=\langle b_2,D\rangle$,...,$A_T^2=\langle b_T,D\rangle$. Furthermore, the user can decode $\langle b_{T+k},D\rangle$ for $k \in [N-T]$ by doing
\begin{align*}
    A_{T+k}^1-\sum_{l=0}^{T-1} k^l A_{l+1}^1=\langle a_{T+k}+b_{T+k},D\rangle-\sum_{l=0}^{T-1} k^l \langle a_{l+1},D\rangle=\langle b_{T+k},D\rangle.
\end{align*}

Thus, since $b_1,b_2,...,b_n$ are linearly independent in $\text{span}(e_{N+1},...,e_{2N}) \subset \text{GF}^{2N}(q)$, the system
\begin{align*}
        \begin{bmatrix}
            b_1^1 & b_1^2 & \cdots & b_1^N\\
            b_2^1 & b_2^2 & \cdots & b_2^N\\
            \vdots & \vdots & \ddots & \vdots \\
            b_N^1 & b_N^2 & \cdots & b_N^N
        \end{bmatrix}
        \begin{bmatrix}
            D_2^1 \\
            D_2^2 \\
            \vdots \\
            D_2^N 
        \end{bmatrix}
        =
        \begin{bmatrix}
            \langle b_1, D\rangle \\
            \langle b_2, D\rangle \\
            \vdots \\
            \langle b_N, D\rangle \\
        \end{bmatrix}
    \end{align*}
has a unique solution, where $b_j \doteq [0,0,...,0,b_j^1,b_j^2,...,b_j^N]^t$.

Therefore, for $j \in [2]$, $r_1^j,...,r_T^j, r_{T+1}^1, ...,r_N^1 \in \text{GF}(q)$, $d_2 \in \text{GF}^{N}(q)$, it follows that
\begin{align*}
    \text{Pr}[D_2=d_2|A_1^j=r_1^j,...,A_N^1=r_N^1]=
    \begin{cases}
        1, & \text{if} \ d_2=f_2(r_1^j,...,r_N^1). \\
        0, & \text{otherwise}.
        \end{cases}
\end{align*}

Now, we prove Security.

Since the queries are uniformly at random and linearly independent in their respective spaces based on the value that the desired index $i \in [2]$ assumes, any $T$ queries $a_{k_1}(i),...,a_{k_T}(i)$ and $b_{k_1}(i),...,b_{k_T}(i)$ are uniformly at random and linearly independent. Thus, using **Lemma 6.2**, we can interpret the distributions of $\mathcal{A}^i$ and $\mathcal{B}^i$ as the observation of a submatrix $2N \times T$ in $\text{GL}_{2N}(q)$. So, for $\mathcal{I}\doteq \{k_1,k_2,...,k_T\} \subset [N]$,
\begin{align*}
    &\text{Pr}[\mathcal{A}^i=r]=\frac{[G_1^2(r)]\cdot [L(r)]}{|\text{col}(\text{GL}_{2N}(q))|},\\
    &\text{Pr}[\mathcal{B}^i=s]=\frac{[G_2^2(s)]\cdot [L(s)]}{|\text{col}(\text{GL}_{2N}(q))|}.
\end{align*}
for $m \in [M]$.

Now, since the vector spaces $\text{span}(e_1,...,e_N) \subset \text{GF}^{2N}(q)$ and $\text{span}(e_{N+1},...,e_{2N}) \subset \text{GF}^{2N}(q)$ are complementary, the distributions of $\mathcal{A}^i$ and $\mathcal{B}^i$ do not influence each other, which implies that $\mathcal{A}^i$ and $\mathcal{B}^i$ are statistically independent. Then,
\begin{align*}
    \text{Pr}[\mathcal{A}^i=r,\mathcal{B}^i=s]=\left( \frac{[G_1^2(r)]\cdot [G_2^2(s)]\cdot [L(r)]\cdot [L(s)]}{|\text{col}(\text{GL}_{2N}(q))|} \right)^2.
\end{align*}

In addition, observe that the distribution of $\mathcal{A}^i$ and $\mathcal{B}^i$ only depend on the conditions $G_1^2(X), G_2^2(X)$ and $L(X)$ regardless of the value that $i$ assumes, so it follows that
\begin{align*}
    \text{Pr}[\mathcal{A}^i=r,\mathcal{B}^i=s|i=m]=\left( \frac{[G_1^2(r)]\cdot [G_2^2(s)]\cdot [L(r)] \cdot [L(s)]}{|\text{col}(\text{GL}_{2N}(q))|} \right)^2=\text{Pr}[\mathcal{A}^i=r,\mathcal{B}^i=s],
\end{align*}
for $m \in [M]$.

Therefore, using Bayes theorem, we conclude that
\begin{align*}
    \text{Pr}[i=m|\mathcal{A}^i=r,\mathcal{B}^i=s]&=\frac{\text{Pr}[\mathcal{A}^i=r,\mathcal{B}^i=s|i=m]\ \text{Pr}[i=m]}{\text{Pr}[\mathcal{A}^i=r,\mathcal{B}^i=s]}\\
    &=\frac{\text{Pr}[\mathcal{A}^i=r,\mathcal{B}^i=s]\ \text{Pr}[i=m]}{\text{Pr}[\mathcal{A}^i=r,\mathcal{B}^i=s]}\\
    &=\text{Pr}[i=m].
\end{align*}


$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

Since the first $T$ servers answers with $2T$ symbols and the last $N-T$ servers answers with $N-T$ symbols, the download rate for the general refined scheme is
\begin{align*}
    \mathcal{R}=\frac{N}{2T+N-T}=\frac{N}{N+T}.
\end{align*}
Note that this download rate is equal to the PIR capacity $\mathcal{C}$ for $M=2$ because
\begin{align*}
    \mathcal{C}=\frac{(N-T)N}{N^2-T^2}=\frac{(N-T)N}{(N+T)(N-T)}=\frac{N}{N+T}.
\end{align*}

The general algorithm code for this case involves six functions, but there is no need to consider all six separately. The reason for separating them is mainly to explain the main points of the code.

First, we consider a function that generates any linearly independent vectors of $\text{GF}^N(q)$ to be added in a tuple. The process involves creating a random matrix $N\times N$ and checking that its determinant is non-zero, because all rows will be are linearly independent. 

In [44]:
# Generate a tuple with N vectos linearly independent

def generate_vectors(N, q):
    GF = galois.GF(q)

    vector_size = N
    m = GF.Random((vector_size, vector_size))
    d = np.linalg.det(m)
    while d == GF(0):                                  
        m = GF.Random((vector_size, vector_size))
        d = np.linalg.det(m)
        
    rows_list = []
    for row in m:
        rows_list.append(row)

    return tuple(rows_list)

But, the queries of type $a_*$ and $b_*$ are vectors of, respectively, $\text{span}(e_1,...,e_N) \subset \text{GF}^{2N}(q)$ and $\text{span}(e_{N+1},...,e_{2N}) \subset \text{GF}^{2N}(q)$. Thus, we create a function to modify the vectors of the tuple created by the previous function so that the zeros are added to the correct blocks.

In [45]:
# Add the zeros entries in the appropriate positions and returns a tuples of vectors to be used in the scheme

def add_zero_entries(vectors_list, N, j, q): #j is block of the vector that will be equal to the L.I. vector of the vector list
    GF = galois.GF(q)
    new_vector_list = []

    block_size = N
    total_size = 2*N

    for vector in vectors_list:
        new_vector = GF(np.zeros(total_size, dtype=int))
        start_index = (j - 1) * block_size
        end_index = j * block_size
        new_vector[start_index:end_index] = vector
        new_vector_list.append(new_vector)

    return tuple(new_vector_list)

Now, we create a function that define the queries $a_{T+1},a_{T+2},...,a_N$ when $i=2$ and $b_{T+1},b_{T+2},...,b_N$ when $i=1$:

In [46]:
# Generates a linear combination through the (N-T) x T Vandermonde matrix V(1,2,...,N-T) of the first T queries that do not query the desired message

def evaluate_poly_ref(N, T, vectors, x, q):             # x is the Vandermonde scalar
    GF = galois.GF(q)

    evaluate = GF(np.zeros(2*N, dtype=int))
    for degree, value in enumerate(vectors):
        evaluate += value * (GF(x) ** degree)
        if degree + 1 >= T:
            break
        
    return evaluate

With these functions, we have enough to define the set of queries that will be sent to each server:

In [47]:
# Generate a tuple with all queries labeled by the server enumeration

def generate_queries_ref(N, T, i, q):
    vectors_a = generate_vectors(N, q)
    vectors_a = add_zero_entries(vectors_a, N, 1, q)

    vectors_b = generate_vectors(N, q)
    vectors_b = add_zero_entries(vectors_b, N, 2, q)

    shares = []

    # Send to the first T servers
    for k in range(T):
        shares.append((k + 1, vectors_a[k], vectors_b[k]))

    x = 1 # First Vandermonde coefficient
    
    # Send to the remaining (N-T) servers
    for k in range(T, N):
        if i == 1:
            combination_b = evaluate_poly_ref(N, T, vectors_b, x, q)
            shares.append((k + 1, vectors_a[k] + combination_b))
        elif i == 2:
            combination_a = evaluate_poly_ref(N, T, vectors_a, x, q)
            shares.append((k + 1, combination_a + vectors_b[k]))
        x += 1

    return tuple(shares)

Since the user sends the queries to the servers, each server responds with the inner product between the queries and the database. So, we define a function that computes all the answers:

In [48]:
# Generate a tuple labeled by the server enumeration with the answers that the servers send back to the user

def servers_answ(DB, qry_list, N, T):
    A = qry_list
    A = list(A)
    for i in range(len(A)):
        A[i] = list(A[i])

    for j in range(T):
        for k in range(1, 3):
            query = qry_list[j][k]
            A[j][k] = np.dot(query, DB)
    for j in range(T, N):
        query = qry_list[j][1]
        A[j][1] = np.dot(query, DB)

    for i in range(len(A)):
        A[i] = tuple(A[i])

    return tuple(A)

Finally, note that, when $i=1$, we need to solve the linear system
\begin{align*}
        \begin{bmatrix}
            a_1^1 & a_1^2 & \cdots & a_1^N\\
            a_2^1 & a_2^2 & \cdots & a_2^N\\
            \vdots & \vdots & \ddots & \vdots \\
            a_N^1 & a_N^2 & \cdots & a_N^N
        \end{bmatrix}
        \begin{bmatrix}
            D_1^1 \\
            D_1^2 \\
            \vdots \\
            D_1^N 
        \end{bmatrix}
        =
        \begin{bmatrix}
            \langle a_1, D\rangle \\
            \langle a_2, D\rangle \\
            \vdots \\
            \langle a_N, D\rangle \\
        \end{bmatrix}
    \end{align*}
where $a_j \doteq [a_j^1,a_j^2,...,a_j^N,0,0,...,0]^t$.

On the other hand, when $i=2$, we need to solve the linear system
\begin{align*}
        \begin{bmatrix}
            b_1^1 & b_1^2 & \cdots & b_1^N\\
            b_2^1 & b_2^2 & \cdots & b_2^N\\
            \vdots & \vdots & \ddots & \vdots \\
            b_N^1 & b_N^2 & \cdots & b_N^N
        \end{bmatrix}
        \begin{bmatrix}
            D_2^1 \\
            D_2^2 \\
            \vdots \\
            D_2^N 
        \end{bmatrix}
        =
        \begin{bmatrix}
            \langle b_1, D\rangle \\
            \langle b_2, D\rangle \\
            \vdots \\
            \langle b_N, D\rangle \\
        \end{bmatrix}
    \end{align*}
where $b_j \doteq [0,0,...,0,b_j^1,b_j^2,...,b_j^N]^t$.


This implies that we need the definition of the queries and answers that are stored, respectively, in the lists of queries and answers. Thus, we define a function that decodes the desired message $i$ as follows:

In [49]:
# Decodes the desired message

def decodability(DB, qry_list, answ_list, N, T, i, q):
    GF = galois.GF(q)
    A = answ_list
    A = list(A)
    for k in range(len(A)):
        A[k] = list(A[k])

    x = 1
    for j in range(T,N):
        if i == 1:
            b = GF(np.zeros(2*N, dtype=int))
            degree = 0
            for k in range(T):
                b += qry_list[k][2] * (GF(x) ** degree)
                degree += 1
            x += 1
            A[j][1] = A[j][1] - np.dot(DB, b)
        elif i == 2:
            a = GF(np.zeros(2*N, dtype=int))
            degree = 0
            for k in range(T):
                a += qry_list[k][1] * (GF(x) ** degree)
                degree += 1
            x += 1
            A[j][1] = A[j][1] - np.dot(DB, a)
                    
    Mtrxcoeff = [] 
    if i == 1:
        for j in range(len(A)):
            q = qry_list[j][1]
            Mtrxcoeff.append(q[:N])
    elif i == 2:
        for j in range(T):
            q = qry_list[j][2]
            Mtrxcoeff.append(q[-N:])
        for j in range(T,N):
            q = qry_list[j][1]
            Mtrxcoeff.append(q[-N:])

    evaluation_list = []
    
    if i == 1:
        for j in range(len(A)):
            evaluation_list.append(A[j][1])
    elif i == 2:
        for j in range(T):
            evaluation_list.append(A[j][2])
        for j in range(T,N):
            evaluation_list.append(A[j][1])

    sol = np.linalg.solve(GF(np.array(Mtrxcoeff)), GF(np.array(evaluation_list)))

    return print('Message reconstructed:', sol)

To explain the use of the previous codes, consider $N=4$, $T=2$, $i=1$ and $q= 5$. In this way, we can define a database

In [50]:
# Example

N, T, i, q = 4, 2, 1, 5

GF = galois.GF(q)
DB = GF.Random((2*N))
print(DB)

[0 1 0 2 0 2 3 1]


and generate de list of queries that each server will receive using the function generate_queries_ref:

In [51]:
# Example
g = generate_queries_ref(N, T, i, q)
print(g)

((1, GF([3, 1, 0, 0, 0, 0, 0, 0], order=5), GF([0, 0, 0, 0, 0, 2, 4, 3], order=5)), (2, GF([1, 2, 4, 1, 0, 0, 0, 0], order=5), GF([0, 0, 0, 0, 3, 3, 1, 4], order=5)), (3, GF([3, 1, 0, 4, 3, 0, 0, 2], order=5)), (4, GF([2, 3, 3, 1, 1, 3, 1, 1], order=5)))


Now, we compute all answers using the function servers_answ:

In [52]:
# Example

answers = servers_answ(DB, g, N, T)
print(answers)

((1, GF(1, order=5), GF(4, order=5)), (2, GF(4, order=5), GF(3, order=5)), (3, GF(1, order=5)), (4, GF(0, order=5)))


Finally, we decode the desired message using the function decodability:

In [53]:
# Example

print('The database is', DB)
decodability(DB, g, answers, N, T, i, q)

The database is [0 1 0 2 0 2 3 1]
Message reconstructed: [0 1 0 2]


### 6.2. Lift

Finally, we introduce the Lift technique. Remember that in Refine, we always use queries that do not query the desired message in a useful way for decoding and to reinforce the symmetry of queries for each message, i.e., for the number of queries that consult the first and second message be the same, which guarantees the security of the scheme. In Lift, we do the same process in a recursive way from $M=2$ to $M=3$, $M=3$ to $M=4$, up to an arbitrary $M$. So, let us illustrate this process showing how we can lift the number of messages from the schemes in examples **6.1** and **6.2**.

**Example 6.3:** Consider the same configuration as in example **6.1** where $N=2$, $T=1$ and let us introduce the recursive process to lift the scheme from $M=2$ to $M=3$ messages. The idea is always to maintain the symmetry of the queries and use the queries that do not query the desired message in a useful way. In this case, we consider that each server has a copy of the database $D=[D_1,D_2,D_3]^t \in \text{GF}^{12}(q)$ where each message $D_i \in \text{GF}^{4}(q)$. Note that the size of each message is four instead of two. In general, the more we lift, the larger the messages should be, as we are interested in configurations involving large messages, because this is the case where the download rate dominates (see Section **4**, examples **4.1** and **4.2**).

Remember that the general scheme when $M=2$ is given by the following table:

$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline a_1 & a_2+b_2  \\
 b_1 &  \\
\hline
\end{array}
\end{aligned}
$$

Without loss of generality, suppose that $i=1$, i.e., the scheme is given by 

$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline a_1 & a_2+b_1  \\
 b_1 &  \\
\hline
\end{array}
\end{aligned}
$$

So, we will use this table to construct the scheme with $M=3$ messages.

First, we must remember that we now have three messages instead of two. Therefore, to reinforce the symmetry with the first server, the user must send a query that query the third message $c_1$ as below:

$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline a_1 & a_2+b_1  \\
 b_1 &  \\
 c_1 & \\
\hline
\end{array}
\end{aligned}
$$

Similarly, the user reinforces the symmetry of the queries with the second server by sending $a_3+c_2$ and $b_2+c_3$:
$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline a_1 & a_2+b_1  \\
 b_1 & a_3+c_2 \\
 c_1 & b_2+c_3\\
\hline
\end{array}
\end{aligned}
$$

But, note that we always want to take advantage of queries that do not query the desired message $i=1$ in queries that query the desired message, as $a_2+b_1$ is defined to make $\langle a_2+b_1\rangle -\langle b_1\rangle=\langle a_2\rangle$ and obtain linearly independent combinations of the symbols of the desired message $i=1$. In this way, we can take advantage of the query $c_1$ by making $c_2=c_1. Thus, the table is updated as follows:
$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline a_1 & a_2+b_1  \\
 b_1 & a_3+c_1 \\
 c_1 & b_2+c_2\\
\hline
\end{array}
\end{aligned}
$$

Note that the new $b_2$ and $c_2$ in the table above must be linearly independent of, respectively, $b_1$ and $c_1$, because if they are linearly dependent, we will have the equivalent of a single query that query messages 2 and 3, and two queries that query message 1, i.e., Server 2 will discover that the user is interested in message $i=1$.

Again, note that the query $b_2+c_2$ used to enforce query symmetry between messages is not being used in a useful way for decoding. This implies that we can send a new symmetric query to the first server that depends on $b_2+c_2$, such as $a_4+b_2+c_2$. Thus, the table is updated as follows:

$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline a_1 & a_2+b_1  \\
 b_1 & a_3+c_1 \\
 c_1 & b_2+c_2\\
\hline a_4+b_2+c_2 & \\
\hline 
\end{array}
\end{aligned}
$$

For $i=1$, the queries are defined as follows:
- Choose queries $a_1,a_2,a_3,a_4 \in \text{span}(e_1,...,e_4) \subset \text{GF}^{12}(q)$ uniformly at random and linearly independent.
- Choose queries $b_1,b_2 \in \text{span}(e_5,...,e_8) \subset \text{GF}^{12}(q)$ uniform at random and linearly independent.
- Choose queries $c_1,c_2 \in \text{span}(e_9,...,e_{12}) \subset \text{GF}^{12}(q)$ uniform at random and linearly independent.

In general, the scheme is given in the table
$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline a_1 & a_2+b_2  \\
 b_1 & a_3+c_2 \\
 c_1 & b_3+c_3\\
\hline a_4+b_4+c_4 & \\
\hline 
\end{array}
\end{aligned}
$$
and queries are defined in a similar way to the previous definition, based on the value of the desired index $i \in [3]$.

The scheme has the Decodability property, because we can always obtain four linearly independent combinations of the four symbols of the desired message $D_i \in \text{GF}^{4}(q)$ using all answers. In addition, the scheme has the Security property due to the symmetry of queries between messages. The demonstrations follow the same logic as Refine, so we will prove it formally when we present the general case of Lift to focus on presenting the process behind the technique that will provide us with optimal PIR schemes for an arbitrary number of $M$ messages.

Since we have four desired symbols of seven answers downloaded, the download rate $\mathcal{R}$ for such a scheme is
\begin{align*}
    \mathcal{R}=\frac{4}{7}.
\end{align*}
Again, this is an improvement on the one-shot scheme for $N=2$ and $T=1$, because $\frac{N-T}{T}=\frac{2-1}{2}=\frac{1}{2}<\frac{4}{7}$ and it is the capacity $\mathcal{C}$ for $N=2$, $T=1$ and $M=3$.

**Example 6.4:** Again, without loss of generality, consider $i=1$, $N=2$ and $T=1$. In this case, we consider that each server has a copy of the database $D=[D_1,D_2,D_3,D_4]^t \in \text{GF}^{32}(q)$ where each message $D_j \in \text{GF}^{8}(q)$. So, to lift the number of messages in the previous scheme from $M=3$ to $M=4$, the user proceeds as follows: 

- **Step 1:** Reinforce the symmetry by introducing a new query type $d_*$ that queries the fourth message and using the table
$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline a_1 & a_2+b_1  \\
 b_1 & a_3+c_1 \\
 c_1 & b_2+c_2\\
\hline a_4+b_2+c_2 & \\
\hline 
\end{array}
\end{aligned}
$$
to obtain the following table:
$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline a_1 & a_2+b_1  \\
 b_1 & a_3+c_1 \\
 c_1 & b_2+c_2\\
 d_1 & a_4+ d_2 \\
 & b_3+d_3 \\
 & c_3+d_4 \\
\hline a_5+b_2+c_2 & \\
a_6+b_4+d_5 & \\
a_7+c_4+d_6 & \\
b_5+c_5+d_7 & \\
\hline 
\end{array}
\end{aligned}
$$

- **Step 2:** Use the queries that do not query the desired message in useful way to decode the desired message as follows:

$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline a_1 & a_2+b_1  \\
 b_1 & a_3+c_1 \\
 c_1 & b_2+c_2\\
 d_1 & a_4+ d_1 \\
 & b_3+d_2 \\
 & c_3+d_3 \\
\hline a_5+b_2+c_2 & \\
a_6+b_3+d_2 & \\
a_7+c_3+d_3 & \\
b_4+c_4+d_4 & \\
\hline 
\end{array}
\end{aligned}
$$

- **Step 3:** Create a new symmetric query, i.e. a query that queries all $M=4$ messages to be sent to Server 2 so that $b_4+c_4+d_4$ can be used for decodability, as $a_8+b_4+c_4+d_4$:
$$
\begin{aligned}
\begin{array}{|c|c|}
\hline \text{Server } 1 & \text{Server } 2 \\
\hline a_1 & a_2+b_1  \\
 b_1 & a_3+c_1 \\
 c_1 & b_2+c_2\\
 d_1 & a_4+ d_1 \\
 & b_3+d_2 \\
 & c_3+d_3 \\
\hline a_5+b_2+c_2 & a_8+b_4+c_4+d_4\\
a_6+b_3+d_2 & \\
a_7+c_3+d_3 & \\
b_4+c_4+d_4 & \\
\hline 
\end{array}
\end{aligned}
$$

Again, the scheme has the Decodability property, because we can obtain eight linearly independent combinations of the eight symbols of the desired message $D_i \in \text{GF}^{8}(q)$ using all answers. In addition, the scheme has the Security property due to the symmetry of queries between messages. For example, each server observes four queries to each different message, which implies that the server cannot tell which message is desired because the user declares the same “amount” of interest in all the messages. 

In this scheme, since we will have eight desired symbols of fifteen answers downloaded, the download rate $\mathcal{R}$ for such a scheme is
\begin{align*}
    \mathcal{R}=\frac{8}{15}.
\end{align*}
This is the capacity $\mathcal{C}$ for $N=2$, $T=1$ and $M=4$.

To avoid saying “the query that does not query the desired message” all the time, we will give this type of query a name.

**Definition 6.1:** Let $x^1,x^2,...,x^M$ be the queries that queries, respectively, the messages $1,2,...,M$ and a desired index $i \in [M]$. A query $Q$ is called a *Leftover* if $Q \doteq \sum_{j=1}^M c_j x^j$ has $c_i=0$. 

In other words, a query that do not query the desired message is called a leftover. So, let us present an example with collusion.

**Example 6.5:** Consider the same configuration as in example **6.2** where $N=3$, $T=2$. We present, the lift scheme for $M=3$ messages. In this case, we consider that each server has a copy of the database $D=[D_1,D_2,D_3]^t \in \text{GF}^{27}(q)$ where each message $D_i \in \text{GF}^{9}(q)$. So, remember that the general Refine scheme ($M=2$) is given by

$$
\begin{aligned}
\begin{array}{|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 \\
\hline a_1 & a_2 & a_3+b_3  \\
 b_1 & b_2 & \\
\hline
\end{array}
\end{aligned}
$$
where the leftovers are defined based on the value of desired index.

One could extend the scheme in the table above as follows:
$$
\begin{aligned}
\begin{array}{|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 \\
\hline a_1 & a_2 & a_3+b_3  \\
 b_1 & b_2 & a_4+c_3 \\
 c_1 & c_2 & b_4+c_4\\
\hline a_5+b_5+c_5 & & \\
\hline 
\end{array}
\end{aligned}
$$

However, as $T=2$ servers collude, if $i=1$ and we set $b_5=b_4$ and $c_5=c_4$, servers 1 and 3 together would be able to discover that the desired message is $D_1$, because the queries $a_5+b_5+c_5$ and $b_4+c_4$ would be equivalent to two queries for the first message and only one for each other message. Therefore, the scheme would not be secure. Therefore, we need to define another leftover of type $b_*+c_*$ to define a query of type $a_*+b_*+c_*$. This logic arises because of the $(3,2,3)$-Ramp Secret Sharing scheme given by 
$$
\begin{aligned}
\begin{array}{|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 \\
\hline R_1 & R_2 & e_i+R_1+R_2  \\
\hline 
\end{array}
\end{aligned}
$$
where we use the $T=2$ "leftovers" $R_1,R_2$ to define the the query that query the desired message $e_i+(R_1+R_2)$.

In this way, the new leftover of type $b_*+c_*$ needs to be defined and sent to another server to ensure security. Then, we can repeat the first part of the scheme shifted to the left and obtain the following general scheme for $M=3$:

$$
\begin{aligned}
\begin{array}{|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 \\
\hline a_1 & a_2 & a_3+b_3  \\
 b_1 & b_2 & a_4+c_3 \\
 c_1 & c_2 & b_4+c_4\\
\hline a_6 & a_7+b_7 & a_5  \\
 b_6 & a_8+c_7 & b_5 \\
 c_6 & b_8+c_8 & c_5\\
\hline a_9+b_9+c_9 & & \\
\hline 
\end{array}
\end{aligned}
$$

Note that any $T=2$ servers observe six different (linearly independent) queries for each message. Therefore, the scheme is symmetric, which implies Security. To make it clear how queries are defined as a function of the desired index $i \in [3]$, consider $i=1$. In this case, the scheme is given by the following table:
$$
\begin{aligned}
\begin{array}{|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 \\
\hline a_1 & a_2 & a_3+b_1+b_2  \\
 b_1 & b_2 & a_4+c_1+c_2 \\
 c_1 & c_2 & b_3+c_3\\
\hline a_6 & a_7+b_4+b_5 & a_5  \\
 b_5 & a_8+c_4+c_5 & b_4 \\
 c_5 & b_6+c_6 & c_4\\
\hline a_9+b_3+b_6+c_3+c_6 & & \\
\hline 
\end{array}
\end{aligned}
$$

For $i=1$, the queries are defined as follows:
- $a_1,a_2,...,a_9 \in \text{span}(e_1,...,e_{9}) \subset \text{GF}^{27}(q)$ are uniformly at random and linearly independent.
- $b_1,b_2,...,b_6 \in \text{span}(e_{10},...,e_{18}) \subset \text{GF}^{27}(q)$ are uniformly at random and linearly independent.
- $c_1,c_2,...,c_6 \in \text{span}(e_{19},...,e_{27}) \subset \text{GF}^{27}(q)$ are uniformly at random and linearly independent.

Now, consider $i=2$. In this case, the scheme is as follows:

$$
\begin{aligned}
\begin{array}{|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 \\
\hline a_1 & a_2 & a_1+a_2+b_3  \\
 b_1 & b_2 & a_3+c_3 \\
 c_1 & c_2 & b_4+c_1+c_2\\
\hline a_5 & a_4+a_5+b_7 & a_4  \\
 b_6 & a_6+c_6 & b_5 \\
 c_5 & b_8+c_4+c_5 & c_4\\
\hline a_3+a_6+b_9+b_6+c_3+c_6 & & \\
\hline 
\end{array}
\end{aligned}
$$

Thus, for $i=2$, the queries are defined as follows:
- $a_1,a_2,...,a_6 \in \text{span}(e_1,...,e_{9}) \subset \text{GF}^{27}(q)$ are uniformly at random and linearly independent.
- $b_1,b_2,...,b_9 \in \text{span}(e_{10},...,e_{18}) \subset \text{GF}^{27}(q)$ are uniformly at random and linearly independent.
- $c_1,c_2,...,c_6 \in \text{span}(e_{19},...,e_{27}) \subset \text{GF}^{27}(q)$ are uniformly at random and linearly independent.

Finally, consider $i=3$. In this case, the scheme is as follows:

$$
\begin{aligned}
\begin{array}{|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 \\
\hline a_1 & a_2 & a_3+b_3  \\
 b_1 & b_2 & a_1+a_2+c_3 \\
 c_1 & c_2 & b_1+b_2+c_4\\
\hline a_5 & a_6+b_6 & a_4  \\
 b_5 & a_4+a_5+c_7 & b_4 \\
 c_6 & b_4+b_5+c_8 & c_5\\
\hline a_3+a_6+b_3+b_6+c_9 & & \\
\hline 
\end{array}
\end{aligned}
$$

So, for $i=3$, the queries are defined as follows:
- $a_1,a_2,...,a_6 \in \text{span}(e_1,...,e_{9}) \subset \text{GF}^{27}(q)$ are uniformly at random and linearly independent.
- $b_1,b_2,...,b_6 \in \text{span}(e_{10},...,e_{18}) \subset \text{GF}^{27}(q)$ are uniformly at random and linearly independent.
- $c_1,c_2,...,c_9 \in \text{span}(e_{19},...,e_{27}) \subset \text{GF}^{27}(q)$ are uniformly at random and linearly independent.

In this scheme, since we will have nine desired symbols of nineteen answers downloaded, the download rate $\mathcal{R}$ for such a scheme is
\begin{align*}
    \mathcal{R}=\frac{9}{19}.
\end{align*}
And, it is the capacity $\mathcal{C}$ for $N=3$, $T=2$ and $M=3$.

This process can be generalized and schematized in an algorithm that we will present theoretically and computationally. So, let us define some concepts and functions that will be part of our algorithm.

**Definition 6.2:** A d-query is a sum of d queries, each querying a specific message.

**Example 6.6:** $a_1,b_1,c_1$ are 1-queries, $a_2+c_2$ is a 2-query and $a_3+b_3+c_3$ is a 3-query.

With this notion, we present a structure called symbolic matrix $\mathcal{S}_M$. Each column of $S_M$ corresponds to a server and each $k$ entry in a $j$ column corresponds to sending all $k$-query combinations to the $j$ server. For example, in Refine ($M=2$), the symbolic matrix is
\begin{align*}
    \mathcal{S}_2=
    \begin{bmatrix}
        1 & 1 & \cdots & 1 & 2 & 2 & \cdots & 2
    \end{bmatrix}
\end{align*}
A value of 1 in the first $T$ columns represents sending all possible combinations of 1-query for the first $T$ servers and a value of 2 in the lastly $N-T$ columns represents sending all combinations of 2-query of every message to the lastly $N-T$ servers.

**Example 6.7:** For $N=2$ and $T=1$ the symbolic matrix $\mathcal{S}_2$, $\mathcal{S}_3$ and $\mathcal{S}_4$ are 
\begin{align*}
    \mathcal{S}_2=
    \begin{bmatrix}
        1 & 2
    \end{bmatrix}
    , \
    \mathcal{S}_3=
    \begin{bmatrix}
        1 & 2\\
        3 &
    \end{bmatrix}
    , \
    \mathcal{S}_4=
    \begin{bmatrix}
        1 & 2\\
        3 & 4
    \end{bmatrix}
    .
\end{align*}
We omit the zeros to prevent confusion.

Note that these matrices define the general schemes discussed in examples **6.1**, **6.3** and **6.4**, but they do not say anything about how to relate the leftovers. We will see later how to make it clear the way to relate them.

**Example 6.8:** For $N=3$ and $T=2$ the symbolic matrix $\mathcal{S}_2$ and $\mathcal{S}_3$ are
\begin{align*}
    \mathcal{S}_2=
    \begin{bmatrix}
        1 & 1 & 2
    \end{bmatrix}
    , 
    \mathcal{S}_3=
    \begin{bmatrix}
        1 & 1 & 2\\
        1 & 2 & 1\\
        3 & &
    \end{bmatrix}
    .
\end{align*}

Observe that these matrices define the general schemes discussed in examples **6.2** and **6.5**.

Now, we begin the definitions for the combinatorial process.

**Definition 6.3:** Let $\mathcal{S}$ be a matrix with $N$ columns. The left shift operation $\sigma(\mathcal{S})$ is a matrix such that
\begin{align*}
    \sigma(\mathcal{S})[i,j]=
        \begin{cases}
            \mathcal{S}[i,j+1] &\text{, if }1\leq j\leq N-1.\\
            \mathcal{S}[i,1] &\text{, if }j=N.\\
        \end{cases}
        ,
\end{align*}
where $\mathcal{S}[i,j]$ represents the entry of $\mathcal{S}$ in the row $i$ and column $j$.

**Example 6.9:** For $\mathcal{S}=\begin{bmatrix} 1 & 2 & 3 \\ 0 & 5 & 3 \end{bmatrix}$, $\sigma(\mathcal{S})=\begin{bmatrix} 2 & 3 & 1 \\ 5 & 3 & 0 \end{bmatrix}$.

This function can be implemented by following code:

In [54]:
# Shifts the columns of a matrix to the left

def left_shift(S):
    return np.roll(S, shift=-1, axis=1)

Thus, in our example, we have:

In [55]:
S = np.array([[1,2,3],[0,5,3]])
print(S)
print(left_shift(S))

[[1 2 3]
 [0 5 3]]
[[2 3 1]
 [5 3 0]]


**Definition 6.4:** Let $\mathcal{S}$ be a matrix. The set of entries in $\mathcal{S}$ with value $k$ is defined by
\begin{align*}
    [k, \mathcal{S}]=\{(i,j):S[i,j]=k\}.
\end{align*}

**Example 6.10:** For $\mathcal{S}=\begin{bmatrix} 1 & 2 & 3 \\ 0 & 5 & 3 \end{bmatrix}$, $[3,\mathcal{S}]=\{(1,3),(2,3)\}$.

**Definition 6.5:** Let $k \in \mathbb{N}$ and a PIR scheme with $N$ servers. We define a function $\tau_k: \mathbb{N}^2 \rightarrow \mathbb{N}^2$ in a PIR situation as
\begin{align*}
    \tau_k(i,j)=
        \begin{cases}
            (i+k,j-1), &\text{ if }2\leq j\leq N-1.\\
            (i+k, N), &\text{ if }j=1.\\
        \end{cases}
        .
\end{align*}

**Example 6.11:** $\tau_1(1,1)=(2,2)$.

This function can be implemented by following code:

In [56]:
def tau_k(k, pair, N):
    i, j = pair[0], pair[1]
    if 2 <= j <= N:
        return (i + k, j - 1)
    elif j == 1:
        return (i + k, N)

Thus, in our example, we have:

In [57]:
print(tau_k(1, (1, 1), 2))

(2, 2)


**Definition 6.6:** Let $S \subseteq \mathbb{N}^2$. We define a function $\pi: 2^{\mathbb{N}^2} \rightarrow 2^\mathbb{N}$ such that $\pi(S)=\{j \in \mathbb{N}: (i,j) \in S\}.$

**Example 6.11:** For $S=\{(1,4),(2,3),(4,5)\}$, $\pi(S)=\{3,4,5\}$.

This function can be implemented by following code:

In [58]:
def pi(S):
    return {j for (i, j) in S}

Thus, in our example, we have:

In [59]:
S = {(1,4),(2,3),(4,5)}
print(pi(S))

{3, 4, 5}


**Definition 6.7:** The lexicographical order $\prec$ is the order on the set $\mathbb{N}^2$ such that $(i,j) \prec (k,l)$ if $i<k$ or $i=k$ and $j<l$.

**Example 6.12:** $(1,1)\prec(2,1)$, because $1<2$. In addition, $(1,1)\prec(1,4)$, because $1=1$ and $1<4$.

In Python, we can use the “sorted” function to sort a set or list lexigraphically:

In [60]:
# Example 

print(sorted([(1,4),(1,3),(2,2),(2,1)]))

[(1, 3), (1, 4), (2, 1), (2, 2)]


**Definition 6.8:** (Symbolic Matrix) Let $N,T,M \in \mathbb{N}$ the symbolic matrix $\mathcal{S}_M$ is defined recursively as below.
\begin{align*}
    &\mathcal{S}_2=[1,...,1,2,...,2]\\
    &\mathcal{S}_{M+1}=\text{lift}(\mathcal{S}_M)
\end{align*}
where $\mathcal{S}_2$ has $N$ columns, $T$ entries 1 in the first $T$ entries, $N-T$ entries 2 in the last $N-T$ entries and, for $M(\mathbb{N})$ being the set of all matrices with entries in $\mathbb{N}$, the function $\text{lift}:M(\mathbb{N}) \rightarrow M(\mathbb{N})$ is defined as
\begin{align*}
    \text{lift}(\mathcal{S}_M)=
    \begin{bmatrix}
        S_M\\
        \sigma(S_M)\\
        \sigma^2(S_M)\\
        \vdots \\
        \sigma^{T-1}(S_M)\\
        A_M
    \end{bmatrix}
    .
\end{align*}

The matrix $A_M$ is constructed as follows.

Let $B^M=[M,\mathcal{S}_M]=\{h_1,h_2,...,h_{\#[M,\mathcal{S}_M]}\}$ where $h_i \prec h_j$ if $i<j$. In addition, for every $i \in \mathbb{N}$ such that $1 \leq i \leq \#[M,\mathcal{S}_M]$, let $l$ be the number of rows in $\mathcal{S}_M$ and $B^M_i=\{h_i,\tau_l(h_i),...,\tau^{T-1}_l(h_i)\}$.

Then, the matrix $A_M$ has $\#[M,\mathcal{S}_M]$ rows and $N$ columns and is defined by
\begin{align*}
    A_M[i,j]=
        \begin{cases}
            0, &\text{ if }j \in \pi(B^M_i).\\
            M+1, &\text{ if }j \notin \pi(B^M_i).\\
        \end{cases}
\end{align*}

**Example 6.13:** Let us construct the symbolic matrix of $\mathcal{S}_3$ for $N=4$, $T=2$ and $M=3$. Inicialmente, a matriz simbolica $\mathcal{S}_2$ é dada por 
\begin{align*}
    \mathcal{S}_2=
    \begin{bmatrix}
        1 & 1 & 2 & 2
    \end{bmatrix}
    ,
\end{align*}
where $\mathcal{S}_2$ has $N=4$ columns, $T=2$ entries 1 in the first $T=2$ entries and $N-T=2$ entries 2 in the last $N-T=2$ entries.

In this way, the symbolic matrix $\mathcal{S_3}$ will be defined as follows:
\begin{align*}
    \mathcal{S}_3=
    \begin{bmatrix}
        \mathcal{S}_2 \\
        \sigma(\mathcal{S}_2) \\
        A_2
    \end{bmatrix}
    .
\end{align*}

Using the definition of the left shift operation $\sigma(\mathcal{S})$, we have that
\begin{align*}
    \sigma(\mathcal{S}_2)=
    \begin{bmatrix}
        1 & 2 & 2 & 1
    \end{bmatrix}
    .
\end{align*}

Finally, to define $A_2$, we have
\begin{align*}
    B^2=[2,\mathcal{S}_2]=\{(1,3),(1,4)\}.
\end{align*}

With this, we define $B_1^2$ and $B_2^2$, respectively, by considering $h_1=(1,3)$ and $h_2=(1,4)$:
\begin{align*}
    B_1^2=\{h_1,\tau_1(h_1)\}=\{(1,3),(2,2)\},\\
    B_2^2=\{h_2,\tau_1(h_2)\}=\{(1,4),(2,3)\},\\
\end{align*}
where we consider $l=1$ in $\tau_l(i,j)$ because $\mathcal{S}_2$ have one row.

Then, the matrix $A_2$ will have $\#[2,\mathcal{S}_2]$ rows and $N=4$ columns and is defined by
\begin{align*}
    A_2[i,j]=
        \begin{cases}
            0, &\text{ if }j \in \pi(B^M_i).\\
            3, &\text{ if }j \notin \pi(B^M_i).\\
        \end{cases}
        ,
\end{align*} 
i.e.,
\begin{align*}
    A_2=
    \begin{bmatrix}
        3 & 3 & 0 & 0 \\
        3 & 0 & 0 & 3
    \end{bmatrix}
    .
\end{align*}

Therefore, the symbolic matrix $\mathcal{S_3}$ will be defined by 
\begin{align*}
    \mathcal{S}_3=
    \begin{bmatrix}
        1 & 1 & 2 & 2 \\
        1 & 2 & 2 & 1 \\
        3 &  &  & 3 \\
        3 & 3 &  & 
    \end{bmatrix}
    ,
\end{align*}
where we omit the zeros.

This symbolic matrix generate de following PIR scheme:

$$
\begin{aligned}
\begin{array}{|c|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 & \text{Server } 4\\
\hline a_1 & a_2 & a_3+b_3 & a_5+b_5   \\
 b_1 & b_2 & a_4+c_3 & a_6+c_5 \\
 c_1 & c_2 & b_4+c_4 & b_6+c_6\\
\hline a_8 & a_9+b_9 & a_{11}+b_{11} & a_7\\
 b_8 & a_{10}+c_9 & a_{12}+c_{11} & b_7\\
 c_8 & b_{10}+c_{10} & b_{12}+c_{12} & c_7\\
\hline 
a_{13}+b_{13}+c_{13} &  &  & a_{14}+b_{14}+c_{14}\\
\hline
a_{15}+b_{15}+c_{15} & a_{16}+b_{16}+c_{16} & &  \\
\hline
\end{array}
\end{aligned}
$$

Again, remember that the leftovers will be defined based on the $(4,3,4)$-Ramp Secret Sharing scheme that solves the problem with a download rate of $\frac{1}{2}$, i.e.,
$$
\begin{aligned}
\begin{array}{|c|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 & \text{Server } 4\\
\hline R_1 & R_2 & e_i+R_1+R_2 & e_j+R_1+2R_2   \\
\hline
\end{array}
\end{aligned}
$$
where $i,j$ are the entries that contain the desired symbols in the database.

So, for example, if $i \in [3]$ is the desired index, since we have to choose $T=2$ leftovers of $T=2$ servers to define $N-T=2$ useful queries that query the desired message of $N-T=2$ different servers, a possible way to define the leftovers, for $x^1_*\doteq a_*$, $x^2_*\doteq b_*$, $x^3_*\doteq c_*$ and $k \in [3]\backslash \{i\}$, could be
\begin{align*}
    &x^k_3=x^k_1+x^k_2, \ x^k_4=x^k_1+2x^k_2, \\
    &x^k_9=x^k_7+x^k_8, \ x^k_{11}=x^k_7+2x^k_8, \\
    &x^k_{13}=x^k_4+x^k_{10}, \ x^k_{14}=x^k_4+2x^k_{10}, \\
    &x^k_{15}=x^k_6+x^k_{12}, \ x^k_{16}=x^k_6+2x^k_{12}.
\end{align*}

This choice is interesting because it guarantees the security of the scheme (as we will demonstrate in the general case) and arises as a function of $B_i^M$ which through $T$ entries $M$ in $B_i^M$ are used to define $N-T$ entries $M+1$ in the matrix $A_M$. In this way, it would be useful if we had a tool that could provide us with the information we need to relate the leftovers and achieve the capacity of the scheme, because the symbolic matrix
\begin{align*}
    \mathcal{S}_3=
    \begin{bmatrix}
        1 & 1 & 2 & 2 \\
        1 & 2 & 2 & 1 \\
        3 &  &  & 3 \\
        3 & 3 &  &
    \end{bmatrix}
\end{align*}
does not specify how to define them.

There are several ways of specifying how to relate leftovers. We will present one of these ways. The idea is to create sets labeled by each row of the symbolic matrix. For exemple, consider the set
\begin{align*}
    \{1,(1,1),(1,2),(1,3),(1,4)\}.
\end{align*}

This set tells us the following: “In row 1 of the symbolic matrix $\mathcal{S}_3$, define the queries generated by the entries $(1,3)$ and $(1,4)$ that query the desired message through the leftovers generated by $(1,1)$ and $(1,2)$”, which gives us the following relations
\begin{align*}
    &x^k_3=x^k_1+x^k_2 \text{ and } x^k_4=x^k_1+2x^k_2.
\end{align*}

In this case, note that we used 1-queries to define 2-queries, because the set lists entries 1 and 2 present in line 1 of $\mathcal{S}_3$. In general, a labeled set lists two entries, one entry $k_1$ and another entry $k_2$ such that $k_1<k_2$. In addition, we need $T$ entries $k_1$ to define $N-T$ entries $k_2$. Therefore, it is essential that we create the queries for the smaller $k_1$ entries first and then generate the $k_2$ entries using the leftovers from the $k_1$ entries.

So, back to our example, as the symbolic matrix $\mathcal{S}_3$ has four rows, the four sets below
\begin{align*}
    &\{1,(1,1),(1,2),(1,3),(1,4)\}, \\
    &\{2,(2,4),(2,1),(2,2),(2,3)\}, \\
    &\{3,(1,3),(2,2),(3,1),(3,4)\}, \\
    &\{4,(1,4),(2,3),(4,1),(4,2)\}, \\
\end{align*}
completely define the relations between the leftovers
\begin{align*}
    &x^k_3=x^k_1+x^k_2, \ x^k_4=x^k_1+2x^k_2, \\
    &x^k_9=x^k_7+x^k_8, \ x^k_{11}=x^k_7+2x^k_8, \\
    &x^k_{13}=x^k_4+x^k_{10}, \ x^k_{14}=x^k_4+2x^k_{10}, \\
    &x^k_{15}=x^k_6+x^k_{12}, \ x^k_{16}=x^k_6+2x^k_{12}.
\end{align*}

In this scheme, we will have sixteen desired symbols of twenty-eight answers downloaded, the download rate $\mathcal{R}$ for such a scheme is
\begin{align*}
    \mathcal{R}=\frac{16}{28}=\frac{4}{7}.
\end{align*}
And, it is the capacity $\mathcal{C}$ for $N=4$, $T=2$ and $M=3$.

Before we present the general Lift algorithm, let us characterize some results to confirm that Lift always achieves the capacity of PIR schemes for finite parameters $N$, $T$ and $M$.

**Lemma 6.3:**  Let $\mathcal{S}_M$ be the symbolic matrix of a $(N,T+1)$-secret sharing scheme. Then, the order, i.e., the number of elements of the set $[k, \mathcal{S}_M]=\{(i,j):S_M[i,j]=k\}$ is
\begin{align*}
    \#[k, \mathcal{S}_M]=\left(N-T\right)^{k-1} T^{M-k}, \ 1\leq k \leq M.
\end{align*}

**Proof:** We will prove the result by induction on $M$. The base case is given by $M=2$ where the symbolic matrix $\mathcal{S}_2=[1,...,1,2,...,2]$ has $T$ entries $1$ and $N-T$ entries 2. Thus, 
\begin{align*}
    &\#[1, \mathcal{S}_2]=\left(N-T\right)^{1-1} T^{2-1}=T,\\
    &\#[2, \mathcal{S}_2]=\left(N-T\right)^{2-1} T^{2-2}=N-T.
\end{align*}

Now, suppose that the result is valid for $M-1$, i.e, 
\begin{align*}
    \#[k, \mathcal{S}_{M-1}]=\left(N-T\right)^{k-1} T^{M-k-1}, \ 1\leq k \leq M-1.
\end{align*}
Observe that $\mathcal{S}_M=\text{lift}(S_{M-1})$ is given by
\begin{align*}
    S_M=
    \begin{bmatrix}
    S_{M-1}\\
    \sigma(S_{M-1})\\
    \sigma^2(S_{M-1})\\
    \vdots \\
    \sigma^{T-1}(S_{M-1})\\
    A_{M-1}
\end{bmatrix}
.
\end{align*} 

Since the left shift operation $\sigma^{i}(S_{M-1})$ for $0 \leq i \leq T-1$ keeps the number of each entry type, we have
\begin{align*}
    \#[k, \mathcal{S}_{M}]=\#[k, \mathcal{S}_{M-1}]\cdot T=\left(N-T\right)^{k-1} T^{M-k}, \ 1\leq k \leq M-1.
\end{align*}

Finally, we construct the matrix $A_{M-1}$ using the definition of the symbolic matrix. Note that $A_{M-1}$ is a matrix that only have entries with the value $M$. 

Note that, by the induction hypothesis, $\#[M-1,S_{M-1}]=(N-T)^{M-2}$. So,
\begin{align*}
    B^{M-1}=[M-1,S_{M-1}]=\{h_1,h_2,...,h_{(N-T)^{M-2}}\}, \ h_i \prec h_j \text{ if } i<j.
\end{align*}
Then, we define the sets $B_j^{M-1}$ for $1\leq j \leq (N-T)^{M-2}$ with order $T$ as
\begin{align*}
    &B_1^{M-1}=\{h_1,\tau_l(h_1),...,\tau^{T-1}_l(h_1)\},\\
    &\vdots\\
    &B^{M-1}_{(N-T)^{M-2}}=\{h_{(N-T)^{M-2}},\tau_l(h_{(N-T)^{M-2}}),...,\tau^{T-1}_l(h_{(N-T)^{M-2}})\}.
\end{align*}
        
In this way, each set $\pi(B^{M-1}_j)$ has order $T$ as well, which implies that $A_{M-1}$ will have $\#[M-1,S_{M-1}]=(N-T)^{M-2}$ rows with each having $T$ zeros and $(N-T)$ entries $M$. Therefore,
\begin{align*}
    \#[M,S_{M}]=(N-T)^{M-2} \cdot (N-T)=(N-T)^{M-1},
\end{align*}
and the result below follows
\begin{align*}
    \#[k, \mathcal{S}_M]=\left(N-T\right)^{k-1} T^{M-k}, \ 1\leq k \leq M.
\end{align*}

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

**Lemma 6.4:** Let $\mathcal{S}_M$ be the symbolic matrix of a $(N,N-1,T+1)$-Ramp Secret Sharing scheme. Then, the number $L$ of queries that query the message $D_i$ in database $D=[D_1,D_2,...,D_M]^t$ for $1\leq i \leq M$ is 
\begin{align*}
    L = N^{M-1}.
\end{align*}

**Proof:** Note that each entry $k \in \mathbb{N}$ in the symbolic matrix $S_M$ represents $\binom{M}{k}$ queries. So, if we want to know how many of these queries query the message $D_i$, just fix the query $M^i_*$ that queries $D_i$ in the sum, i.e,
\begin{align*}
    \underbrace{{M^i_*}}_{\text{fixed}}+\underbrace{M^{i_1}_*+M^{i_2}_*+...+M^{i_{k-1}}_*}_{\text{$k-1$ options from $M-1$}}.
\end{align*}
Then, the number of k-queries which query $D_i$ is 
\begin{align*}
    \binom{M-1}{k-1}.
\end{align*}

Thus, by **Lemma 6.3**, as number of each k-query in the symbolic matrix is $\#[k, \mathcal{S}_M]=\left(N-T\right)^{k-1} T^{M-k}, \ 1\leq k \leq M$, we have that
\begin{align*}
    L&=\sum_k \#[k, \mathcal{S}_M] \binom{M-1}{k-1} \cdot [1\leq k\leq M]\\
    &=\sum_k \left(N-T\right)^{k-1} T^{M-k}  \binom{M-1}{k-1}\cdot  [1\leq k\leq M]\\
    &=\sum_k \left(N-T\right)^{k} T^{M-1-k}  \binom{M-1}{k}\cdot  [0\leq k\leq M-1]\\
    &= ((N-T)+T)^{M-1}\\
    &= N^{M-1}.
\end{align*}

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

**Lemma 6.5:** Let $\mathcal{S}_M$ be the symbolic matrix of a $(N,N-1,T+1)$-Ramp Secret Sharing scheme. Then, the total number $Q$ of queries generated according to the symbolic matrix $S_M$ is 
\begin{align*}
    Q=\frac{N^M-T^M}{N-T}.
\end{align*}

**Proof:** Since each entry $k$ in the symbolic matrix represents $\binom{M}{k}$ k-queries and, by **Lemma 6.3**, $\#[k, \mathcal{S}_M]=\left(N-T\right)^{k-1} T^{M-k}, \ 1\leq k \leq M,$ we need to compute
\begin{align*}
    Q&=\sum_k \#[k, \mathcal{S}_M] \binom{M}{k} \cdot [1\leq k\leq M]\\
    &= \sum_k \left(N-T\right)^{k-1} T^{M-k} \binom{M}{k} \cdot [1\leq k\leq M].
\end{align*}

In this way, we can note that
\begin{align*}
    N^M&=((N-T)+T)^M\\
    &=\sum_k (N-T)^k T^{M-k} \binom{M}{k} \cdot [0\leq k\leq M]\\
    &=T^M+\sum_k (N-T)^k T^{M-k} \binom{M}{k} \cdot [1\leq k\leq M]\\
    &\implies N^M-T^M = \sum_k (N-T)^{k} T^{M-k} \binom{M}{k} \cdot [1\leq k\leq M]\\
    &\implies \frac{N^M-T^M}{N-T} = \sum_k (N-T)^{k-1} T^{M-k} \binom{M}{k} \cdot [1\leq k\leq M].
\end{align*}

Therefore, we conclude that
\begin{align*}
    \frac{N^M - T^M}{N-T}&=\sum_k (N-T)^k T^{M-k} \binom{M}{k} \cdot [1\leq k\leq M]\\
    &=\sum_k \#[k, \mathcal{S}_M] \binom{M}{k} \cdot [1\leq k\leq M]\\
    &=Q.
\end{align*}

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

Using lemmas **6.4** and **6.5**, we can immediately conclude that we will have $N^{M-1}$ desired symbols $D_i^1,D_i^2,...,D_i^{N^{M-1}} \in \text{GF}(q)$ and $\frac{N^M-T^M}{N-T}$ answers downloaded. Then, the download rate $\mathcal{R}$ for such a scheme is
\begin{align*}
    \mathcal{R}=\frac{N^{M-1}}{\frac{N^M-T^M}{N-T}}=\frac{(N-T)N^{M-1}}{N^M-T^M}.
\end{align*}

Which is exactly the capacity $\mathcal{C}$ known for the case where the database is replicated on $N$ servers with any $T$ of them being able to collude. However, we still need to characterize the general algorithm and demonstrate the Decodability and Security properties of the scheme. So, let us present the general algorithm.

A user wants to retrieve a message $D_i=[D_i^1,...,D_i^{N^{M-1}}]^t \in \text{GF}^{N^{M-1}}(q)$ privately from a database $D=[D_1,D_2,...,D_M]^t \in \text{GF}^{MN^{M-1}}(q)$ replicated on $N$ servers which any $T$ of them can collude $(T< q)$. The user proceeds as follows:

**Algorithm 3:** $(N,T,M)$-Lift Scheme.

**Step 1:** Take a $(N,N-1,T+1)$-Ramp Secret Sharing scheme as input:
    $$
    \begin{aligned}
    \begin{array}{|c|c|c|c|c|c|}
    \hline \text{Server } 1 & ... & \text{Server } T &\text{Server } T+1 & ... & \text{Server } N \\
    \hline R_1 & ... & R_T & e_1+R_{T+1} & ... & e_{N-T}+R_N  \\
    \hline
    \end{array}
    \end{aligned}
    $$
where, for $k \in [N-T]$,
\begin{align*}
    R_{T+k} \doteq \sum_{l=0}^{T-1} k^l R_{l+1}.
\end{align*}

**Step 2:** Construct the Refine Scheme to obtain the symbolic matrix $\mathcal{S}_2$ with $T$ 1's and $(N-T)$ 2's as below:
\begin{align*}
    &\mathcal{S}_2=
    \begin{bmatrix}
        1 & 1 & \cdots & 1 & 2 & 2 & \cdots & 2
    \end{bmatrix}
\end{align*}
And create a the set of the labeled sets
\begin{align*}
    \text{LF}\doteq \{\{1,(1,1),(1,2),...,(1,N)\}\}.
\end{align*}

**Step 3:** Apply the lift function to $\mathcal{S}_2$ $(M-2)$ times to construct $\mathcal{S}_M$, i.e., 
\begin{align*}
    \mathcal{S}_M=\text{lift}^{M-2}(\mathcal{S}_2),
\end{align*}
attributing a label $l$ to each set $B^m_l$ merged with the N-T ordered pairs of the N-T entries $m+1$ generated by $B^m_l$ to be added to $\text{LF}$.

**Step 4:** For $i \in [M]$ being the desired index, start all the sets of queries that each server will receive as empty sets, i.e.,
\begin{align*}
    Q_1^i\doteq\{\},\ Q_2^i\doteq\{\}, ...,\ Q_N^i\doteq\{\},
\end{align*}
where the upper index $i$ is used, because the definition of the queries change in function of $i$.

**Step 5:** For each entry $k$ in the column $j$ in each row of $\mathcal{S}_M$, add to $Q_j^i$ the queries from the set of $k$-queries
\begin{align*}
    \left\{\sum_k x^k_*\cdot  [k \in K]: K \subseteq [M], |K| =k \right\}
\end{align*}
where each $x^k_* \in \text{span}(e_{(k-1)N^{M-1}+1},e_{(k-1)N^{M-1}+2},...,e_{kN^{M-1}}) \subset \text{GF}^{MN^{M-1}}(q)$.

In this step, for $i \in [M]$ being the desired index, choose $x^i_1,x^i_2,...,x^i_{N^{M-1}}$ uniformly at random and linearly independent in $ \text{span}(e_{(i-1)N^{M-1}+1},e_{(i-1)N^{M-1}+2},...,e_{iN^{M-1}}) \subset \text{GF}^{MN^{M-1}}(q)$. In addition, choose all leftovers generated by the entry $k$ uniformly at random and linearly independent in their respective space.

The way that we do this has two cases:

1) The row has entries 1 and 2: In this case, we generate the entries $1$ first to define the entries $2$ using the leftovers related by the set of sets labeled $\text{LF}$.

2) The row has entries 0 and k such that $k \geq 3$: In this case, by the recursive process, the $k-1$-entries are already created, which implies that the $k-1$-query leftovers that will be used to define the $k$-queries of the $k$-entry are already defined. Thus, we generate the entries $k$ using the leftovers related by the set of sets labeled $\text{LF}$.

In this way, considering $i\neq j \in [M]$ and $x^j_{i_1},x^j_{i_2},...,x^j_{i_T}$ the $T$ queries generated uniformly at random and linearly independent by the smallest entry $k-1$, we define, through the p-th entry $k$ of the $N-T$, the queries that consult the desired message as a combination  
\begin{align*}
    x^i_*+\sum_k x^k_{i_{T+p}}\cdot  [k \in K]
\end{align*}    
such that $K \subseteq [M]\backslash \{i\}, |K| =k-1, p \in [N-T]$ and
\begin{align*}
    x^k_{i_{T+p}} \doteq \sum_{l=0}^{T-1} p^l x^j_{i_{l+1}}.
\end{align*}

**Step 6:** The user sends $Q^i_j$ for each server $j \in [N]$.

**Step 7:** Each server $j \in [N]$ answers with 
\begin{align*}
    A_j = \{\langle x, D \rangle: x \in Q^i_j\}.
\end{align*}

**Step 8:** The user decodes $D_i^1,...,D_i^{N^{M-1}} \in \text{GF}^{N^{M-1}}(q)$ using the answers of the queries related by the labeled sets in $\text{LF}$ to obtain all the inner products $\langle x^i_1,D\rangle,\langle x^i_2,D\rangle,...,\langle x^i_{N^{M-1}},D\rangle$ and solving the linear system
\begin{align*}
    \begin{bmatrix}
        x_1^i(1) & x_1^i(2) & \cdots & x_1^i(N^{M-1})\\
        x_2^i(1) & x_2^i(2) & \cdots & x_2^i(N^{M-1})\\
        \vdots & \vdots & \ddots & \vdots \\
        x_{N^{M-1}}^i(1) & x_{N^{M-1}}^i(2) & \cdots & x_{N^{M-1}}^i(N^{M-1})
    \end{bmatrix}
    \begin{bmatrix}
        D_i^1 \\
        D_i^2 \\
        \vdots \\
        D_i^{N^{M-1}} 
    \end{bmatrix}
    =
    \begin{bmatrix}
        \langle x^i_1,D\rangle \\
        \langle x^i_2,D\rangle \\
        \vdots \\
        \langle x^i_{N^{M-1}}, D\rangle \\
    \end{bmatrix}
\end{align*}
where $x^i_*(j) \in \text{GF}(q)$ is the entry $(i-1)N^{M-1}+j$ of the vector $x^i_* \in \text{span}(e_{(i-1)N^{M-1}+1},e_{(i-1)N^{M-1}+2},...,e_{iN^{M-1}}) \subset \text{GF}^{MN^{M-1}}(q)$.

To prove that this scheme has the Decodabilty property, we need to show that given $r_j \in \text{GF}^{|Q^i_j|}(q)$, $d_i \in \text{GF}^{N^{M-1}}(q)$ and a deterministic function $f$ such that $f(r_1,...,r_N)=d_i$, we have that
\begin{align*}
    \text{Pr}[D_i=d_i|A_1=r_1,...,A_N=r_N]=
    \begin{cases}
        1, & \text{if} \ d_i=f(r_1,...,r_N). \\
        0, & \text{otherwise}.
    \end{cases}
\end{align*}

To prove Security, we observe that, since we have a total of $N^{M-1}$ queries of each type $x^k_*$, $k \in [M]$, by **Lemma 6.4**, each server observe
\begin{align*}
    \frac{N^{M-1}}{N}=N^{M-2}
\end{align*}
queries of each type. Thus, this implies that $T$ servers observe $TN^{M-2}$ queries of each type $x^k_*$.

In this way, let $\{t_1,t_2,...,t_{TN^{M-2}}\} \subset \{1,2,...,N^{M-1}\}$ be the set of indices that the $T$ servers observe. Then, the $T$ servers observe $M$ matrices of type
\begin{align*}
    \mathcal{M}^i_k= 
    \begin{bmatrix}
        x^k_{t_1} & x^k_{t_2} & \cdots & x^k_{t_{TN^{M-2}}}
    \end{bmatrix}
\end{align*}
where we denote upper index $i$ is used to explicit the fact that the queries depend on the choice of the desired index $i \in [M]$.

Thus, using the definitions of $G_k^M(X),L(X)$ and $C_k^n(M)$ in **Lemma 6.2**, we need that given $m \in [M]$ and $q_k \in C_k^{TN^{M-2}}(M) \subset \text{GF}^{(MN^{M-1})\times (TN^{M-2})}(q)$, it follows that
\begin{align*}
    \text{Pr}[i=m|\mathcal{M}_1^i=q_1,...,\mathcal{M}_M^i=q_M]=\text{Pr}[i=m].
\end{align*}

**Proof:** Initially, we prove Decodability.

Note that for the 1 entries in the symbolic matrix, according to **Lemma 6.3**, we will always have 1-queries that query only the desired message, which implies that we will have 
\begin{align*}
    \#[1, \mathcal{S}_M]=T^{M-1}
\end{align*}
answers of the type $\langle x^i_*, D \rangle$.

In addition, observe that every $k$-query ($k\geq 2$) that queries the desired message is written as the linear combination of a query of type $x^i_*$ and $k-1$ queries of type $x^j_*$ ($i \neq j$), i.e.,
\begin{align*}
    \sum_{p} x^p_* \cdot [p \in P]
\end{align*}
such that $P \subseteq [M]$, $|P|=k$ and $i \in P$. 

Thus, by **Step 5**, we can obtain the other $(N-T)^{M-1}$ answers of the type $\langle x^i_*, D \rangle$ noting that
\begin{align*}
    \sum_{p} x^p_* \cdot [p \in P \backslash \{i\}]
\end{align*}
is linear combination $T$ leftovers of type $k-1$-query, i.e., we can decode $\langle x^i_*, D \rangle$ by doing
\begin{align*}
    \langle x^i_*, D \rangle= \langle\sum_{p} x^p_* \cdot [p \in P]\rangle - \sum_{l=1}^T\left(\langle \sum_{p} x^p_{p_{l}}\rangle \cdot [p \in P \backslash \{i\}]\right)
\end{align*}
where the subindices $p_l$ are the indices that appear in the leftovers for each $p \in P \backslash \{i\}$.

In other words, using all answers, we can get all combinations
\begin{align*}
    \langle x^i_1, D \rangle, \langle x^i_2, D \rangle,...,\langle x^i_{N^{M-1}}, D \rangle.
\end{align*}

In this way, since $x^i_1, x^i_2,..., x^i_{N^{M-1}}$ are linearly independent in $\text{span}(e_{(i-1)N^{M-1}+1},e_{(i-1)N^{M-1}+2},$ $...,e_{iN^{M-1}}) \subset \text{GF}^{MN^{M-1}}(q)$, we have $N^{M-1}$ linearly independent combinations of the $N^{M-1}$ unknown symbols of the message $D_i$. Thus, $D_i^1,D^2_i,...,D_i^{N^{M-1}}\in \text{GF}(q)$ are deterministic and we can define a deterministic function such that
\begin{align*}
    f(A_1,A_2,...,A_N)= [D_i^1,D_i^2,...,D_i^{N^{M-1}}]^t
    =D_i.
\end{align*}

Therefore, for $r_j \in \text{GF}^{|Q^i_j|}(q)$ and $d_i \in \text{GF}^{N^{M-1}}(q)$, we have that
\begin{align*}
    \text{Pr}[D_i=d_i|A_1=r_1,...,A_N=r_N]=
    \begin{cases}
        1, & \text{if} \ d_i=f(r_1,...,r_N). \\
        0, & \text{otherwise}.
    \end{cases}
\end{align*}

Now, we prove Security.

First, by **Step 3** and **Step 5**, the labeled sets use $T$ uniformly at random and linearly independent queries that do not query the desired message from $T$ different servers $k_1,k_2,...,k_T$, such as $x^j_{k_1},x^j_{k_2},...,x^j_{k_T}$, to define $N-T$ queries that do not query the desired message from the remaining $N-T$ servers $k_{T+1},k_{T+2},...,k_N$. This implies that every query related by a labeled set can be defined by a linear combination
\begin{align*}
    x^j_{k_p}= \sum_{l=1}^T c_l x^j_{k_l},
\end{align*}
where, if $p \in [T]$,
\begin{align*}
    x^j_{k_p}= \sum_{l=1}^T c_l x^j_{k_l}\cdot [l=p]
\end{align*}
and, if $p \in \{T+1,T+2,...,N\}$,
\begin{align*}
    x^j_{k_p}= \sum_{l=1}^T p^{l-1} x^j_{k_l}.
\end{align*}

However, since only $T$ servers collude and the Vandermonde matrix defines linearly independent combinations, any quantity less than or equal to $T$ of these queries is linearly independent, which implies that all columns of $\mathcal{M}_j^i$ are uniformly at random and linearly independent. Therefore, using **Lemma 6.2**, we can interpret the distribution of $\mathcal{M}_j^i$ as submatrix $(N^{M-1}) \times (TN^{M-2})$ of $\text{GL}_{N^{M-1}}(q)$ to obtain that 
\begin{align*}
    \text{Pr}[\mathcal{M}_j^i=q]=\frac{[G_k^M(q)]\cdot [L(q)]}{|\text{col}_\mathcal{I}(\text{GL}_{N^{M-1}}(q))|}
\end{align*}
for $\mathcal{I}\subset [MN^{M-1}]$ such that $|\mathcal{I}|=TN^{M-2}$.

In this way, since the vector spaces $\text{span}(e_{(k-1)N^{M-1}+1},e_{(k-1)N^{M-1}+2},...,e_{kN^{M-1}}) \subset \text{GF}^{MN^{M-1}}(q)$ for $k \in [M]$ are complementary, the distributions of each $\mathcal{M}_k^i$ do not influence each other, which implies that they are statistically independent. Then,
\begin{align*}
    \text{Pr}[\mathcal{M}_1^i=q_1,...,\mathcal{M}_M^i=q_M]=\left( \frac{\prod_{k=1}^M [G_k^M(q_k)]\cdot [L(q_k)]}{|\text{col}(\text{GL}_{N^{M-1}}(q))|} \right)^M.
\end{align*}

In addition, observe that the distribution of $\mathcal{M}_k^i$ only depend on the conditions $G_k^M(X)$ and $L(X)$ regardless of the value that $i$ assumes, so it follows that
\begin{align*}
    \text{Pr}[\mathcal{M}_1^i=q_1,...,\mathcal{M}_M^i=q_M|i=m]=\left( \frac{\prod_{k=1}^M [G_k^M(q_k)]\cdot [L(q_k)]}{|\text{col}(\text{GL}_{N^{M-1}}(q))|} \right)^M=\text{Pr}[\mathcal{M}_1^i=q_1,...,\mathcal{M}_M^i=q_M],
\end{align*}
for $m \in [M]$.

Therefore, using Bayes theorem, we conclude that
\begin{align*}
    \text{Pr}[i=m|\mathcal{M}_1^i=q_1,...,\mathcal{M}_M^i=q_M]&=\frac{\text{Pr}[\mathcal{M}_1^i=q_1,...,\mathcal{M}_M^i=q_M|i=m]\ \text{Pr}[i=m]}{\text{Pr}[\mathcal{M}_1^i=q_1,...,\mathcal{M}_M^i=q_M]}\\
    &=\frac{\text{Pr}[\mathcal{M}_1^i=q_1,...,\mathcal{M}_M^i=q_M]\ \text{Pr}[i=m]}{\text{Pr}[\mathcal{M}_1^i=q_1,...,\mathcal{M}_M^i=q_M]}\\
    &=\text{Pr}[i=m].
\end{align*}


$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

Finally, we give a possible implementation of the Lift algorithm. For this case, the code involves seven functions, but there is no need to consider all eight separately. The reason for separating them is mainly to explain the main points of the code.

Initially, we give a function that constructs the symbolic matrix and provides the set of the labeled sets $\text{LF}$:

In [61]:
def lift(N, T, M):
    S_2 = np.array([1] * T + [2] * (N - T)).reshape((1, N))
    lifted_S = S_2.copy()

    # Start of the recursive process
    B_dict = {}
    BM_i = {}
    comb_set = {'1': [(1, j) for j in range(1,N+1)]}

    for k in range(2, M):
        # Number of keys in the original dictionary
        j = int(list(comb_set.keys())[-1])

        # Number of rows of S_{k} to finish defining the labeled sets when creating the k+1 input matrix
        count = 0
        for row in lifted_S:
            for element in row:
                if element == k:
                    count += 1
        
        # Left-Shift
        for t in range(1, T):
            lifted_S = np.vstack((lifted_S, left_shift(lifted_S[-S_2.shape[0]:])))
        # Create a copy of the original dictionary so as not to modify the original directly
        new_comb_set = comb_set.copy()
        # Calculate the new key
        for i in range(0, (T-1) * j):
            nova_chave = i + j +1
            nova_lista = [tau_k(j, pair, N) for pair in comb_set[f'{i+1}']]
            new_comb_set[f'{nova_chave}'] = nova_lista
##          
        B_dict[f'B^{k}'] = sorted([(i+1, j+1) for i in range(len(S_2)) for j in range(N) if S_2[i, j] == k]) 

        l = len(S_2)
        A_M = np.zeros((len(B_dict[f'B^{k}']), N), dtype=int)

        m = 1
        for idx, h_i in enumerate(B_dict[f'B^{k}']):
            B_M_i = [h_i]
            h = h_i
            for t in range(1, T):
                h = tau_k(l, (h[0], h[1]), N)
                B_M_i.append(h)
            BM_i[f'B^{k}_{m}'] = B_M_i
        
            pi_H_M_i = pi(BM_i[f'B^{k}_{m}'])
            for j in range(N):
                if j + 1 in pi_H_M_i:
                    A_M[idx, j] = 0
                else:
                    A_M[idx, j] = k + 1
            m += 1

        # Number of keys in the original dictionary
        jj = int(list(new_comb_set.keys())[-1])
        for p in range(count): #p is the row of A_M
            list_of_new_entries = []
            for q in range(N):
                if A_M[p][q] == k+1:
                    list_of_new_entries.append((len(lifted_S)+p+1, q+1))
            nova_chave = jj + p + 1
            nova_lista = BM_i[f'B^{k}_{p+1}'] + list_of_new_entries
            new_comb_set[f'{nova_chave}'] = nova_lista

        comb_set = new_comb_set.copy()
        lifted_S = np.vstack((lifted_S, A_M))
        S_2 = lifted_S.copy()

    return lifted_S, new_comb_set


For example, if $N=4$, $T=2$ and $M=3$, we can check that 
\begin{align*}
    \mathcal{S}_3=
    \begin{bmatrix}
        1 & 1 & 2 & 2 \\
        1 & 2 & 2 & 1 \\
        3 &  &  & 3 \\
        3 & 3 &  & 
    \end{bmatrix}
    ,
\end{align*}
and obtain 
\begin{align*}
    \text{LF}=\{&\{1,(1,1),(1,2),(1,3),(1,4)\}, \\
    &\{2,(2,4),(2,1),(2,2),(2,3)\}, \\
    &\{3,(1,3),(2,2),(3,1),(3,4)\}, \\
    &\{4,(1,4),(2,3),(4,1),(4,2)\}\} \\
\end{align*}

by doing

In [62]:
mtrx, lab = lift(4, 2, 3)
print(mtrx)
print(lab)

[[1 1 2 2]
 [1 2 2 1]
 [3 0 0 3]
 [3 3 0 0]]
{'1': [(1, 1), (1, 2), (1, 3), (1, 4)], '2': [(2, 4), (2, 1), (2, 2), (2, 3)], '3': [(1, 3), (2, 2), (3, 1), (3, 4)], '4': [(1, 4), (2, 3), (4, 1), (4, 2)]}


**Remark 6.1:** We consider lists instead of sets in the code, but both have the same purpose for our goals.

Now, let us move on to the functions that will help us define the set of queries:

Again, we consider a function that generates any linearly independent vectors of $\text{GF}^{N^{M-1}}(q)$ to be added in a tuple and a function to modify the vectors of to the space $\text{GF}^{MN^{M-1}}(q)$ the tuple created by the previous function so that the zeros are added to the correct blocks:

In [63]:
# Lift

def generate_vectors_lift(N, M, q):
    GF = galois.GF(q)

    vector_size = N ** (M-1)
    m = GF.Random((vector_size, vector_size))
    d = np.linalg.det(m)
    while d == GF(0):
        m = GF.Random((vector_size, vector_size))
        d = np.linalg.det(m)
        
    rows_list = []
    for row in m:
        rows_list.append(row)

    return tuple(rows_list)

def add_zero_entries_lift(vectors_list, N, M, i, q):
    GF = galois.GF(q)
    new_vector_list = []

    block_size = N ** (M - 1)
    total_size = M * block_size

    for vector in vectors_list:
        new_vector = GF(np.zeros(total_size, dtype=int))
        start_index = (i - 1) * block_size
        end_index = i * block_size
        new_vector[start_index:end_index] = vector
        new_vector_list.append(new_vector)

    return tuple(new_vector_list)

Now, let us introduce the main idea behind the code. We created an object/class called “Query” which has three characteristics:
1) The entry of the symbolic matrix that generated it.
2) The type of the query given by a set.
3) The ordered pair of the entry $k$ that generated the query.

The second feature tells us which messages the query is querying. For example, query $q$ will receive the set 
\begin{align*}
    \{1,2\},
\end{align*}
if the query queries the symbols of messages 1 and 2 in the database.

This idea is useful because we can think of the problem in terms of sets. For example, if we were to generate the first line of 
\begin{align*}
    \mathcal{S}_3=
    \begin{bmatrix}
        1 & 1 & 2 & 2 \\
        1 & 2 & 2 & 1 \\
        3 &  &  & 3 \\
        3 & 3 &  & 
    \end{bmatrix}
\end{align*}
we could think of the problem as follows:
$$
\begin{aligned}
\begin{array}{|c|c|c|c|}
\hline \text{Server } 1 & \text{Server } 2 & \text{Server } 3 & \text{Server } 4\\
\hline \{1\} & \{1\} & \{1,2\} & \{1,2\}   \\
 \{2\} & \{2\} & \{1,3\} & \{1,3\} \\
 \{3\} & \{3\} & \{2,3\} & \{2,3\}\\
\hline
\end{array}
\end{aligned}
$$

In this way, since the labeled set for this row is $\{1,(1,1),(1,2),(1,3),(1,4)\}$, pick the queries defined by the sets $S$ in which $i$ belongs such that $|S|=2$, because the largest entry is 2 in this case and take leftovers of the entries 1 defined by sets of the type
\begin{align*}
    S-\{i\}.
\end{align*}

In other words, for each entry $k$ of the symbolic matrix, we generate all combinations $\binom{M}{k}$ of sets with elements in $[M]$ and define the queries based in this combinations with the labeled sets.


In [64]:
class Query:
    def __init__(self):
        self.data = {}

    def __call__(self, x: int, y: set, z: tuple):
        return self.data.get((x, frozenset(y), z))

    def __setitem__(self, key, value):
        x, y, z = key
        self.data[(x, frozenset(y), z)] = value

    def __repr__(self):
        return f"Query(data={self.data})"

Q = Query()

from itertools import combinations

def gen_sets(M, k):
    elementos = list(range(1, M + 1))
    return list(combinations(elementos, k))

Now, we are prepared to provide a possible implementation, using all the previous functions for Lift:

In [65]:
# The function reads the symbolic matrix and the labeled sets to create the query sets

def gen_queries(S, labeled_sets, desired_index, N, T, M, q):
    GF = galois.GF(q)

    # Generating all the queries of each type x^1_*,x^2_*,...,x^M_*
    queries_types = {}
    for i in range(1, M+1):
        queries = generate_vectors_lift(N, M, q)
        queries_types[i] = add_zero_entries_lift(queries, N, M, i, q)

    # Reading the symbolic matrix with the labeled sets to create the queries
    usage = {i: 0 for i in range(1, M+1)}            #We defined this dictionary to avoid repeating the use of queries when atributting the choices
    queries_list = {i: [] for i in range(1, N+1)}
    for i in range(1, int(list(labeled_sets.keys())[-1]) + 1):
        if labeled_sets[f'{i}'][0][0] == i:    #it only happens when the first T ordered pairs of the labeled sets are 1's 
            for j in range(T):
                x, y = labeled_sets[f'{i}'][j][0], labeled_sets[f'{i}'][j][1]
                entry = S[x-1, y-1]
                comb_types = gen_sets(M, entry)
                for ty_pe in comb_types:
                    for k in set(ty_pe):
                        Q[entry, frozenset(ty_pe), (x, y)] = queries_types[k][usage[k]]
                        queries_list[y].append(Q(entry, frozenset(ty_pe), (x, y)))
                        usage[k] += 1
            
            # Generating the larger entries and defining the leftovers
            c = 1 #Vandermonde coefficient
            for j in range(T, N):
                x, y = labeled_sets[f'{i}'][j][0], labeled_sets[f'{i}'][j][1]
                entry = S[x-1, y-1]
                comb_types = gen_sets(M, entry)
                for ty_pe in comb_types:
                    v = GF(np.zeros(M*N**(M-1), dtype=int))
                    if desired_index in ty_pe: #LEFTOVER
                        for k in set(ty_pe):
                            if k == desired_index:
                                v += queries_types[k][usage[k]]
                                usage[k] += 1
                            elif k != desired_index: 
                                degree = 0
                                for l in range(T):
                                    z, w = labeled_sets[f'{i}'][l][0], labeled_sets[f'{i}'][l][1]
                                    v += Q(S[z-1, w-1], frozenset(set(ty_pe)-{desired_index}), (z, w)) * (GF(c)** degree) 
                                    degree += 1
                                Q[entry, frozenset(ty_pe), (x, y)] = v

                    elif desired_index not in ty_pe: #Define a leftover that will be used for a query that asks for one more message
                        for k in set(ty_pe):
                            v += queries_types[k][usage[k]]
                            usage[k] += 1
                        Q[entry, frozenset(ty_pe), (x, y)] = v
                    queries_list[y].append(Q(entry, frozenset(ty_pe), (x, y)))
                
                c += 1
        
        if labeled_sets[f'{i}'][0][0] != i: #Occurs when the first T ordered pairs of the labeled sets are different than 0 and 1 
            # Generation of the larger entries with definition of the leftovers
            c = 1
            for j in range(T, N):
                x, y = labeled_sets[f'{i}'][j][0], labeled_sets[f'{i}'][j][1]
                entry = S[x-1, y-1]
                comb_types = gen_sets(M, entry)
                for ty_pe in comb_types:
                    v = GF(np.zeros(M*N**(M-1), dtype=int))
                    if desired_index in ty_pe: # define a query that asks for the desired message via the leftovers defined by the first T entries
                        v += queries_types[desired_index][usage[desired_index]]
                        usage[desired_index] += 1
                        degree = 0
                        for l in range(T):
                            z, w = labeled_sets[f'{i}'][l][0], labeled_sets[f'{i}'][l][1]
                            v += Q(S[z-1, w-1], frozenset(set(ty_pe)-{desired_index}), (z, w)) * (GF(c)** degree) 
                            degree += 1
                        Q[entry, frozenset(ty_pe), (x, y)] = v
                        
                    elif desired_index not in ty_pe: #create a query that does not ask for the desired message
                        for k in set(ty_pe):
                            v += queries_types[k][usage[k]]
                            usage[k] += 1
                        Q[entry, frozenset(ty_pe), (x, y)] = v

                    queries_list[y].append(Q(entry, frozenset(ty_pe), (x, y)))
                
                c += 1

    return queries_list, queries_types

In the function we also return the list of all queries given by the “queries_types” dictionary, as it is essential that we have access to this dictionary in order to construct the linear system 
\begin{align*}
    \begin{bmatrix}
        x_1^i(1) & x_1^i(2) & \cdots & x_1^i(N^{M-1})\\
        x_2^i(1) & x_2^i(2) & \cdots & x_2^i(N^{M-1})\\
        \vdots & \vdots & \ddots & \vdots \\
        x_{N^{M-1}}^i(1) & x_{N^{M-1}}^i(2) & \cdots & x_{N^{M-1}}^i(N^{M-1})
    \end{bmatrix}
    \begin{bmatrix}
        D_i^1 \\
        D_i^2 \\
        \vdots \\
        D_i^{N^{M-1}} 
    \end{bmatrix}
    =
    \begin{bmatrix}
        \langle x^i_1,D\rangle \\
        \langle x^i_2,D\rangle \\
        \vdots \\
        \langle x^i_{N^{M-1}}, D\rangle \\
    \end{bmatrix}
\end{align*}
.

For example, let us consider $N=2$, $T=1$ and $M=4$. In this case, the symbolic matrix is given by 

\begin{align*}
    \mathcal{S}_3=
    \begin{bmatrix}
        1 & 2\\
        3 & 
    \end{bmatrix}
\end{align*}
where the labeled sets are 
\begin{align*}
    \text{LF}=\{&\{1,(1,1),(1,2)\}, \\
    &\{2,(1,2),(2,1)\}\}.
\end{align*}

In [66]:
Sym, ls = lift(2, 1, 3)
print(Sym)
print(ls)

[[1 2]
 [3 0]]
{'1': [(1, 1), (1, 2)], '2': [(1, 2), (2, 1)]}


If we are interested in the index $i=2$, the set of queries to be sent to each server by our implementation is given by

In [67]:
g, h = gen_queries(Sym, ls, 2, 2, 1, 3, 2)

print(g)

{1: [GF([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], order=2), GF([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], order=2), GF([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], order=2), GF([1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0], order=2)], 2: [GF([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0], order=2), GF([1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0], order=2), GF([0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1], order=2)]}


where we consider $q=2$ to examplify.

Similarly, to define a function that generates all the answers, it would be interesting to build an “Answer” object/class that has the same characteristics as the Query object. In this way, through the labeled sets and the list of all queries, we can access the entries $x_*^i(j)$ of each vector of type $x_*^i$ to contruct the linear system 
\begin{align*}
    \begin{bmatrix}
        x_1^i(1) & x_1^i(2) & \cdots & x_1^i(N^{M-1})\\
        x_2^i(1) & x_2^i(2) & \cdots & x_2^i(N^{M-1})\\
        \vdots & \vdots & \ddots & \vdots \\
        x_{N^{M-1}}^i(1) & x_{N^{M-1}}^i(2) & \cdots & x_{N^{M-1}}^i(N^{M-1})
    \end{bmatrix}
    \begin{bmatrix}
        D_i^1 \\
        D_i^2 \\
        \vdots \\
        D_i^{N^{M-1}} 
    \end{bmatrix}
    =
    \begin{bmatrix}
        \langle x^i_1,D\rangle \\
        \langle x^i_2,D\rangle \\
        \vdots \\
        \langle x^i_{N^{M-1}}, D\rangle \\
    \end{bmatrix}
\end{align*}
.

Thus, using the symbolic matrix and the set of labeled sets, we generate all the combinations of sets $\binom{M}{k}$ defined by an entry $k$ and access the queries using their three characteristics to compute the inner product with the database:

In [68]:
class Answer:
    def __init__(self):
        self.data = {}

    def __call__(self, x: int, y: set, z: tuple):
        return self.data.get((x, frozenset(y), z))

    def __setitem__(self, key, value):
        x, y, z = key
        self.data[(x, frozenset(y), z)] = value

    def __repr__(self):
        return f"Answer(data={self.data})"
    
A = Answer()

def gen_answers(S, labeled_sets, DB, N, T, M):
    # Determining the server answers 
    answers_list = {i: [] for i in range(1, N+1)}
    for i in range(1, int(list(labeled_sets.keys())[-1]) + 1):
        if labeled_sets[f'{i}'][0][0] == i:    #it only happens when the first T ordered pairs are 1's
            for j in range(N):
                x, y = labeled_sets[f'{i}'][j][0], labeled_sets[f'{i}'][j][1]
                entry = S[x-1, y-1]
                comb_types = gen_sets(M, entry)
                for ty_pe in comb_types:
                    A[entry, frozenset(ty_pe), (x, y)] = np.dot(Q(entry, frozenset(ty_pe), (x, y)), DB)
                    answers_list[y].append(A(entry, frozenset(ty_pe), (x, y)))
        
        if labeled_sets[f'{i}'][0][0] != i: #it only happens when the first T ordered pairs are different than 0 and 1
            for j in range(T, N):
                x, y = labeled_sets[f'{i}'][j][0], labeled_sets[f'{i}'][j][1]
                entry = S[x-1, y-1]
                comb_types = gen_sets(M, entry)
                for ty_pe in comb_types:
                    A[entry, frozenset(ty_pe), (x, y)] = np.dot(Q(entry, frozenset(ty_pe), (x, y)), DB)
                    answers_list[y].append(A(entry, frozenset(ty_pe), (x, y)))

    return answers_list


For example, if $N=2$, $T=1$ and $M=3$, we can define a database as follows:

In [69]:
GF = galois.GF(2)
N = 2
T = 1
M = 3

DB = GF.Random((M*N**(M-1)))
print(DB)

[1 1 0 0 1 1 1 1 1 1 0 0]


Thus, the answers are

In [70]:
gen_answers(Sym, ls, DB, N, T, M)

{1: [GF(0, order=2), GF(1, order=2), GF(0, order=2), GF(1, order=2)],
 2: [GF(1, order=2), GF(1, order=2), GF(0, order=2)]}

Finally, we move on to define the decoding function for the desired message. Again, we use the same idea of using the symbolic matrix and the set of labeled sets to generate all the combinations of sets $\binom{M}{k}$ defined by an entry $k$, obtaining the values of $\langle x^i_{1}, DB\rangle,\langle x^i_{2}, D\rangle,. ...,\langle x^i_{N^{M-1}}, D\rangle$ by subtracting combinations defined by the Vandermonde matrix and constructs the linear system
\begin{align*}
    \begin{bmatrix}
        x_1^i(1) & x_1^i(2) & \cdots & x_1^i(N^{M-1})\\
        x_2^i(1) & x_2^i(2) & \cdots & x_2^i(N^{M-1})\\
        \vdots & \vdots & \ddots & \vdots \\
        x_{N^{M-1}}^i(1) & x_{N^{M-1}}^i(2) & \cdots & x_{N^{M-1}}^i(N^{M-1})
    \end{bmatrix}
    \begin{bmatrix}
        D_i^1 \\
        D_i^2 \\
        \vdots \\
        D_i^{N^{M-1}} 
    \end{bmatrix}
    =
    \begin{bmatrix}
        \langle x^i_1,D\rangle \\
        \langle x^i_2,D\rangle \\
        \vdots \\
        \langle x^i_{N^{M-1}}, D\rangle \\
    \end{bmatrix}
\end{align*}
through the list of queries returned by the “gen_queries()” function.

In [71]:
def decodability(S, labeled_sets, query_type_list, desired_index, N, T, M, q):
    GF = galois.GF(q)

    b = []
          
    for i in range(1, int(list(labeled_sets.keys())[-1]) + 1):
        if labeled_sets[f'{i}'][0][0] == i:    #it only happens when the first T ordered pairs are 1's
            for j in range(T):
                x, y = labeled_sets[f'{i}'][j][0], labeled_sets[f'{i}'][j][1]
                entry = S[x-1, y-1]
                comb_types = gen_sets(M, entry)
                for ty_pe in comb_types:
                    for k in set(ty_pe):
                        if k == desired_index:
                            b.append(A(entry, frozenset(ty_pe), (x, y)))
            
            # Generation of the larger entries with definition of the leftovers
            c = 1 #Vandermonde coefficient
            for j in range(T, N):
                x, y = labeled_sets[f'{i}'][j][0], labeled_sets[f'{i}'][j][1]
                entry = S[x-1, y-1]
                comb_types = gen_sets(M, entry)
                for ty_pe in comb_types:
                    v = A(entry, frozenset(ty_pe), (x, y))
                    if desired_index in ty_pe:
                        for k in set(ty_pe):
                            if k != desired_index: 
                                degree = 0
                                for l in range(T):
                                    z, w = labeled_sets[f'{i}'][l][0], labeled_sets[f'{i}'][l][1]
                                    v -= A(S[z-1, w-1], frozenset(set(ty_pe)-{desired_index}), (z, w)) * (GF(c)** degree) 
                                    degree += 1
                                b.append(v)
                
                c += 1
        
        if labeled_sets[f'{i}'][0][0] != i: #it only happens when the first T ordered pairs are different than 0 and 1
            # Generation of the larger entries with definition of the leftovers
            c = 1
            for j in range(T, N):
                x, y = labeled_sets[f'{i}'][j][0], labeled_sets[f'{i}'][j][1]
                entry = S[x-1, y-1]
                comb_types = gen_sets(M, entry)
                for ty_pe in comb_types:
                    v = A(entry, frozenset(ty_pe), (x, y))
                    if desired_index in ty_pe:
                        degree = 0
                        for l in range(T):
                            z, w = labeled_sets[f'{i}'][l][0], labeled_sets[f'{i}'][l][1]
                            v -= A(S[z-1, w-1], frozenset(set(ty_pe)-{desired_index}), (z, w)) * (GF(c)** degree) 
                            degree += 1
                        b.append(v)
                
                c += 1

    a = GF(np.array([vec[(desired_index-1)*N**(M-1): (desired_index-1)*N**(M-1) + N**(M-1)] for vec in query_type_list[desired_index]]))
    b = GF(np.array(b))
    message = np.linalg.solve(a, b)

    return message

In our example, we can have that

In [72]:
print("The database is", DB)
print('The desired message is', decodability(Sym, ls, h, 2, N, T, M, 2))

The database is [1 1 0 0 1 1 1 1 1 1 0 0]
The desired message is [1 1 1 1]


## 7. Final considerations

We presented a possible solution to the PIR problem with replicated databases using the Secret Sharing cryptographic algorithm, which uses polynomials over finite fields. We saw that when the database has arbitrarily large messages, the download rate dominates, which led us to focus on contructing PIR schemes that optimize the download rate. In addition, we saw that the direct application of Secret Sharing schemes does not achieve the highest known download rate for the problem, called Capacity, which motivated us to present Refine and Lift techniques to obtain optimal PIR schemes for finite parameters through the original formulation of the problem where there are no errors. 

The Refine and Lift techniques transform any Secret Sharing scheme into an optimal scheme for databases replicated on $N$ servers where $T$ collude. However, due to the errors and erasures that naturally occur in any database, there is a great deal of interest in PIR schemes that are resilient to errors and erasures. In this case, the Refine and Lift techniques can also be used to build PIR schemes where an MDS error-correcting code is used, typically a Reed-Solomon code. Therefore, if the reader is interested, we strongly recommend reading the references [11] and [12].

## References

[1] B. Chor, O. Goldreich, E. Kushilevitz, and M. Sudan, “Private information retrieval,” in IEEE Symposium on Foundations of Computer Science,
pp. 41–50, 1995.

[2] B. Chor, E. Kushilevitz, O. Goldreich, and M. Sudan, “Private information retrieval,” Journal of the ACM (JACM), vol. 45, no. 6, pp. 965–981, 1998.

[3] A. Shamir, “How to share a secret,” Commun. ACM, vol. 22, no. 11, pp. 612–613, 1979.

[4] G. R. Blakley, "Safeguarding Cryptographic Keys", Proceedings of the National Computer Conference, 48, 313-317, 1979.

[5] I. S. Reed and G. Solomon, “Polynomial codes over certain finite fields,” Journal of the society for industrial and applied mathematics, vol. 8, no. 2, pp. 300–304, 1960.

[6] R. J. McEliece and D. V. Sarwate, “On sharing secrets and reed-solomon codes,” Communications of the ACM, vol. 24, no. 9, pp. 583–584, 1981.

[7] G. R. Blakley and C. Meadows, “Security of ramp schemes,” in Workshop on the Theory and Application of Cryptographic Techniques. Springer, 1984, pp. 242–268.

[8] C. E. Shannon, “Communication theory of secrecy systems,” The Bell system technical journal, vol. 28, no. 4, pp. 656–715, 1949.

[9] Sun, Hua, and Syed Ali Jafar. "The capacity of private information retrieval." IEEE Transactions on Information Theory 63.7 (2017): 4075-4088.

[10] H. Sun and S. A. Jafar, “The capacity of robust private information retrieval with colluding databases”, 2016.

[11] R. G. L. D’Oliveira and S. El Rouayheb, "One-Shot PIR: Refinement and Lifting", in IEEE Transactions on Information Theory, vol. 66, no. 4, 2020.

[12] R. G. L. D’Oliveira and S. El Rouayheb, "A Guided Walk Through Coded Private Information Retrieval", in IEEE BITS the Information Theory Magazine, 2024.

